In [56]:
!pip install langdetect
!pip install googletrans==4.0.0-rc1
!pip install langid translate
!pip install fasttext
!pip install emoji

In [57]:
import pandas as pd
import string
import re
from langdetect import detect
from googletrans import Translator
import fasttext
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Or PorterStemmer for stemming
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import emoji
import spacy # For tokenization task
from sklearn.preprocessing import LabelEncoder


# Download necessary NLTK data (if not already downloaded)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
df = pd.read_csv('/content/fakeReviewData.csv')
df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"You can wear the hood by itself, wear it with ..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...


In [59]:
# Function to detect the language

def detect_language(text):
    try:
        return detect(text)
    except:
        pass

In [60]:
df['text_'].apply(detect_language).unique()

# Total 15 Difference languages are detected

array(['en', 'ca', 'de', 'af', 'es', 'fr', 'da', 'so', 'et', 'pt', 'nl',
       'vi', 'no', 'sw', 'it', 'tl', None], dtype=object)

In [61]:
# Function to detect and translate the text
model = fasttext.load_model('lid.176.ftz')  # Download the model from https://fasttext.cc/docs/en/language-identification.html

def detect_and_translate(text):
    try:
        predictions = model.predict(text, k=1)
        detected_language = predictions[0][0].replace("__label__", "")
        # print(f"Detected language: {detected_language}")

        # Translate only if the text is not in English
        if detected_language != "en":
            translator = Translator()
            translation = translator.translate(text, src=detected_language, dest='en')
            return translation.text
        else:
            return text  # Return the original text if it's already in English
    except Exception as e:
        return text  # Return the original text if there's an error

In [62]:
df['text_updated'] = df['text_'].apply(detect_and_translate)

In [63]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

Number of rows affected: 64


In [64]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"You can wear the hood by itself, wear it with ..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...


In [65]:
chat_words = {
"BBC": "British Broadcasting Corporation",
"BBB": "Bugging Beyond Belief",
"BBE": "Babe",
"GLOTR": "Good Luck On The Rebuild",
"TWSS": "That's What She Said",
"BBG": "Best Be Going",
"BBK": "Boy Better Know",
"TFRM": "Thanks For Reminding Me",
"BBM": "Blackberry Messaging",
"BBL": "Be Back Later",
"BBO": "Being Bored Online",
"BBN": "Be Back Never",
"BBQ": "Barbecue| Better Be Quick",
"10-4": "Understood, OK",
"SDA": "Seventh-Day Adventist",
"BBS": "Be Back Soon| Bulletin Board System",
"U.O.E.N.O": "You Don't Even Know",
"YEAP": "Yes",
"10-2": "Radio code for 'Receiving well'",
"BBR": "Blonde, Brunette, Redhead",
"10-1": "Radio code for 'Receiving poorly'",
"BBT": "Be Back Tomorrow",
"BBW": "Big Beautiful Woman",
"BBV": "Brags Beats and Variance",
"BBY": "Baby",
"BBZ": "Babes",
"SDK": "Software Development Kit",
"BCD": "Behind Closed Doors",
"BCC": "Blind Carbon Copy",
"$": "Dollar| money",
"BCF": "Best Cousin Forever",
"BCE": "Before Common Era",
"&": "And",
"BCG": "Birth Control Glasses",
"GTBOSH": "Glad To Be Of Some Help",
"*": "Indicating a spelling correction",
"SEC": "Second",
"2": "To",
"SCNR": "Sorry, Could Not Resist",
"BCS": "Bowl Championship Series",
"SED": "Said",
"TASTY": "Good looking| Cool| awesome",
"SKEWL": "School",
"SEM": "Search Engine Marketing",
"IWTD": "I Want To Die",
"SEO": "Search Engine Optimization",
"": "(:):Pig",
"NATTY LIGHT": "Natural Light beer",
"SEP": "Somebody Else's Problem",
"@": "At",
"SES": "Marijuana",
"TFTI": "Thanks For The Info",
"B": "Be",
"C": ":Very happy",
"PKIT": "Please Keep In Touch",
"SET": "Division of a gang",
"E": "Ecstasy",
"TFTD": "Thought For The Day",
"TFTC": "Thanks For The (Geo)Cache",
"G": "Grin",
"H": "Hug",
"J": "Joking",
"K": "OK",
"L": "Laugh",
"BDN": "Busy Doing Nothing",
"TFTL": "Thanks for the Link",
"N": "And",
"SCOFF": "Food| Eat fast| Sneer",
"O": "):Innocent| angelic",
"STUNNER": "Good looking person",
"P": "Penny| pence",
"Q": "Thank You",
"BDU": "Brain Dead User| Battle Dress Uniform",
"R": "Are",
"U": "You",
"V": "Very",
"X": "Kiss",
"STSP": "Same Time Same Place",
"Y": "Why?",
"JACK WAGON": "Loser",
"SFP": "Sorry for Partying",
"SFO": "San Francisco International Airport",
"BEB": "Babe",
"SFR": "Smiling For Real",
"^": "Refer To The Above Posted In The Chat",
"MEATHEAD": "Muscular, but dumb person",
"CEEB": "CBB",
"STR8": "Straight",
"C&C": "Command and Conquer (game)",
"BEF": "Best Enemies Forever| Best Email Friend",
"ICWUDT": "I See What You Did There",
"BEH": "disinterest| IDC",
"SFX": "Sound Effects",
"BEG": "Big Evil Grin",
"SFW": "Safe For Work",
"30STM": "30 Seconds To Mars (band)",
"C&P": "Copy & Paste",
"BER": "Beyond Economical Repair",
"BF2": "Battlefield 2 (game)",
"MUPPET": "Stupid person| idiot",
"BET": "Black Entertainment Television",
"BBAM": "Be Back After Meal",
"ROXOR": "Rocks| Awesome",
"ONE OFF": "Happened Once",
"TFTA": "Thanks For The Add",
"SGO": "Let's Go",
"BFA": "Best Friends Always",
"JUNKIE": "Addict",
"~": "Denotes Sarcasm| Put At The End Of A Word To Make It Longer",
"BFB": "Better From Behind",
"WTHRU": "Who The Hell Are You?",
"SCSI": "Small Computer System Interface",
"SGT": "Sergeant",
"NEET": "Not in Education, Employment or Training",
"BFF": "Best Friends Forever",
"BFK": "Big Fat Kiss",
"BBBW": "Big Beautiful Black Woman",
"BFN": "Bye For Now",
"SHB": "Super Hot Babe",
"BFP": "Bad For Pics",
"JROTC": "Junior Reserve Officer Training Corps",
"SHH": "Be quiet, shut up",
"MSDN": "Microsoft Developer Network",
"SHK": "Should Have Known",
"OHIM": "Oh Hell, It's Monday",
"OLDFAG": "Long established member of an online community",
"SHM": "Simple Harmonic Motion",
"SHO": "Sure| Super High Output",
"BGD": "Black Gangster Disciples",
"DYKT": "Did You Know That?",
"BGF": "Best Girl/Guy Friend",
"BSER": "Back Stabber",
"SHU": "Secure Housing Unit",
"BONKERS": "Crazy",
"NEDM": "Not Even Doom Music",
"BGL": "Big Game License| likes fat people",
"HTTR": "Hail To The Redskins",
"BGM": "Background Music",
"HTTP": "Hyper-Text Transfer Protocol",
"SIA": "Sorry I Asked",
"TABOO": "Forbidden| not talked about",
"SIC": "Latin sicut| apparent mistake",
"BGT": "Britain's Got Talent (TV show)",
"SID": "Acid| LSD",
"SIG": "Signature",
"SIF": "As If",
"SIK": "SICK",
"SIM": "Subscriber Identity Module",
"SIL": "Son-In-Law",
"SIO": "Bye",
"NEEK": "A cross between a Nerd and a Geek",
"SIP": "YEP in Spanish",
"SDRAM": "Synchronous Dynamic Random Access Memory",
"SIS": "Sister",
"STTN": "Sleeping Through the Night",
"BHD": "Bad Hair Day",
"SIT": "Stay In Touch",
"ISTATOY": "I Saw This And Thought Of You",
"SIW": "Someone Is Watching",
"NEGU": "Never Ever Give Up",
"SIV": "Bad goalie in hockey",
"BHH": "Bless His/Her Heart",
"YTMND": "You're The Man Now, Dog",
"BHM": "Big Handsome Man",
"BHO": "Barak Hussein Obama",
"8": "::D:A Penis",
"BHS": "Be Home Soon",
"MSFT": "Microsoft (stock symbol)",
"OHKO": "One Hit Knock Out",
"BOOMERS": "Psilocybin (magic) mushrooms",
"BIC": "Believe It Comrade",
"BIG": "Great| really good",
"SK8": "Skate",
"C++": "Programming language",
"BIL": "Brother In-Law",
"SCOTUS": "Supreme Court Of The United States",
"BIN": "Buy It Now",
"BIM": "Barbados",
"BIO": "Biological break| toilet",
"BIT": "Binary digIT - 1 or 0",
"BIS": "Best In Slot",
"SKI": "Spend Kids' Inheritance| Snort cocaine",
"TWYL": "Talk With You Later",
"BIZ": "Business",
"SKL": "School",
"DYLM": "Do You Love/Like Me?",
"SKO": "Let's Go",
"SKN": "OK, cool, whatever",
"NEGL": "Not Even Gonna Lie",
"PKMN": "Pokemon",
"SKU": "Stock Keeping Unit",
"BBEG": "Big Bad Evil Guy",
"CD-RW": "Compact Disk Rewritable",
"PL0X": "Please",
"IHYDM": "I Hope You Don't Mind",
"SLA": "Service Level Agreement",
"DUECES": "DEUCES",
"SLD": "Second Level Domain",
"DHMU": "Don't Hit Me Up",
"SLF": "Sounds Like Fun",
"SLH": "Santa's Little Helper",
"SWIDT": "See What I Did There?",
"SLI": "Scalable Link Interface",
"SMACKED": "High on marijuana",
"STYL": "Speak To You Later",
"&APOS;S": "Wrongly displayed 's",
"SLM": "Salaam| hello",
"BBFN": "Bye Bye For Now",
"DYOR": "Do Your Own Research",
"SLR": "Single Lens Reflex| Camera",
"BBFL": "Best Buds For Life",
"BKA": "Better Known As",
"TCP/IP": "Transmission Control Protocol/Internet Protocol",
"SLT": "Salut| hello",
"T-T": "Crying",
"SLY": "Still Love You",
"HK": "Hong Kong| Hostile Kids",
"SME": "Small/Medium Enterprises",
"SMG": "Sub-Machine Gun",
"SMH": "Shaking My Head",
"SML": "Screw My Life| Story of My Life",
"SMN": "Shoot Me Now",
"SN1": "Spare No One",
"SMS": "Short Message Service",
"BLD": "Bad Life Decision",
"SMT": "Sucking My Teeth",
"T.T": "Crying",
"BLG": "Boys Like Girls (band)",
"BLK": "Black",
"DWTTAI": "Don't Want To Talk About It",
"NOOPS": "Backward spoon",
"BLL": "Bud Light Lime",
"IMWTK": "Inquiring Minds Want To Know",
"SND": "Search And Destroy",
"SNC": "Social Network Check",
"SWIFT": "Awesome",
"NVMS": "No Visible Means of Support",
"SNF": "So Not Funny",
"BLT": "Bacon, Lettuce and Tomato sandwich",
"SNH": "Sarcasm Noted Here",
"SNK": "Video game company",
"SNM": "Say No More",
"BMB": "Bebo Me Back",
"SNR": "Signal To Noise Ratio",
"SHEILA": "Woman",
"CMIIW": "Correct Me If I'm wrong.",
"TRDMF": "Tears Running Down My Face",
"TRDMC": "Tears Running Down My Cheeks",
"BME": "Body Modification Ezine",
"SNU": "What's New?",
"BMG": "Be My Guest",
"BMI": "Body Mass Index",
"BML": "Bless My Life",
"BMP": "Image file format",
"SOA": "State Of Alert (band)",
"BMT": "Before My Time| Basic Military Training",
"SOE": "Sony Online Entertainment",
"BMS": "Bitchy Men Syndrome",
"BMV": "Be My Valentine",
"SOG": "Sea Of Green| cannabis growing",
"SOF": "Soldier Of Fortune (Game)",
"BMX": "Bicycle Motocross",
"SOI": "Statement of Intent",
"BMW": "Bavarian Motor Works",
"SOH": "Sense Of Humor",
"SOK": "That's OK",
"SPURS": "US Basketball team| UK football team",
"SOO": "So",
"SON": "Close Friend",
"SP2": "Service Pack 2",
"SOP": "Standard Operating Procedure",
"SOS": "Save Our Souls",
"BNB": "Bad News Bears| Bed 'N' Breakfast| Bread 'N' Butter",
"C/D": "Confirm/Deny",
"BNG": "Blow And Go",
"SOW": "Female police officer",
"BNF": "Big Name Fan",
"NVMD": "Never Mind",
"BNI": "Batteries Not Included",
"SOY": "Shame On You",
"CELL": "Cellphone",
"SOX": "Socks",
"SOZ": "Sorry",
"C/O": "Class Of ...",
"BNP": "British National Party",
"BNR": "But Not Really",
"LIITA": "Love Is In The Air",
"SPF": "Strictly Platonic Friend",
"SPG": "Special Patrol Group",
"SPN": "Supernatural (TV show)",
"SPM": "South Park Mexican (rapper)",
"WAREZ": "Cracked/pirate software",
"BOD": "Beginning Of Day| Person| DUDE",
"BOF": "Whatever",
"BOH": "Gunshot sound",
"BOG": "Toilet",
"BOI": "Boy",
"BOL": "Best Of Luck",
"BOM": "Bro| dude| friend",
"BOO": "Boyfriend/Girlfriend",
"NVNG": "Nothing Ventured, Nothing Gained",
"BOT": "Back On Topic",
"BOS": "Boyfriend Over Shoulder",
"MBFAM": "My Brother From Another Mother",
"T2M": "Talk To Me",
"SQL": "Structured Query Language",
"CANDYMAN": "Drug Dealer",
"T2T": "Time To Talk",
"KMFC": "Keeping My Fingers Crossed",
"BPM": "Beats Per Minute",
"BBML": "Be Back Much Later",
"BPS": "Bits/Bytes Per Second",
"SLICE": "Eighth of an ounce of marijuana",
"HABIBI": "My darling",
"SRH": "Supporting Radical Habits",
"BSNS": "Business",
"SLICK": "Sharp| cool| smooth",
"SRM": "Stark Raving Mad",
"SRO": "Standing Room Only",
"SRT": "Street and Racing Technology",
"C2C": "Consumer to Consumer| Cam to Cam",
"SRS": "Serious",
"SRY": "Sorry",
"RTFAQ": "Read The FAQ",
"SSA": "Subject Says All",
"SSC": "Safe, Sane, Consensual",
"SHOTGUN": "Front passenger seat",
"SSD": "Solid State Drive",
"SSH": "Secure Shell",
"PHONY": "Fake object or person",
"ALL- NIGHTER": "Stay Up All Night",
"SSL": "Secure Socket Layer| See Subject Line",
"T4P": "Thanks For Posting",
"BRB": "Be Right Back",
"BRE": "Sexy girl",
"BICBW": "But I Could Be Wrong",
"BRD": "Black Rock Depths",
"T4T": "Tit For Tat| Thanks For Trade",
"BSOF": "Big Smile On Face",
"BRH": "Be Right Here",
"BSOD": "Blue Screen Of Death",
"CEPT": "Except",
"TGAL": "Think Globally, Act Locally",
"BRO": "Brother| buddy| friend",
"STB": "Soon To Be",
"STA": "Surveillance and Target Acquisition",
"STC": "Subject to Change",
"BRU": "Brother",
"POCKO": "Pocket",
"BRT": "Be Right There",
"STH": "Something",
"NINJA": "Japanese warrior",
"STG": "Swear To God",
"STK": "Shoot To Kill",
"STN": "Spend The Night",
"NERD": "Intelligent but socially inept person",
"STM": "Short Term Memory",
"NERF": "Weaken",
"BSB": "Backstreet Boys",
"STR": "Strength| Short Term Relationship",
"BSA": "Boy Scouts of America",
"BSC": "Bachelor of Science",
"C4C": "Comment for Comment| Cam for Cam",
"STS": "Smirk To Self",
"TI AMO": "I Love You",
"BSF": "But Seriously Folks",
"SUBS": "Subwoofers",
"BSG": "Battlestar Galactica (TV show)",
"STW": "Search The Web",
"STY": "Same To You",
"BSL": "British Sign Language",
"2DAY": "Today",
"BST": "But Seriously Though",
"SUAD": "Shut Up And Drive",
"BSU": "Boring Status Update",
"NVRM": "Never Mind",
"SUH": "Sir",
"SUM": "Some",
"CLARET": "Blood",
"SUL": "See You Later",
"BTA": "But Then Again....",
"SUP": "What's Up",
"SUS": "Suspect| suspicious",
"BTB": "By The By",
"BTE": "Best Time Ever",
"BTD": "Bored To Death",
"SUV": "Sport Utility Vehicle",
"SUX": "Sucks",
"BTK": "Back To Keyboard",
"BTM": "Be There Moment",
"BTL": "Between The Lines",
"BTO": "Bachman Turner Overdrive (band)",
"JANOE": "I Know",
"BTN": "Better Than Nothing",
"BTS": "Be There Soon",
"HUZZAH": "triumph| joy",
"MSRP": "Manufacturer's Suggested Retail Price",
"BTR": "Better",
"HOLD UP": "Wait a minute",
"BTU": "British Thermal Unit",
"BTT": "Bump To Top",
"BTW": "By The Way",
"SNOUT": "Cigarette",
"BTY": "Back To You",
"HDCP": "High-bandwidth Digital Content Protection",
"YESH": "Yes",
"ASDFGHJKL;'": "When A Feeling Cannot Be Expressed In Words| Home Keys On Standard Keyboard",
"SVP": "S'il Vous Plait| please",
"POT HEAD": "Person who smokes WEED",
"BUB": "Buddy| friend| Champange",
"BUD": "Marijuana",
"SVS": "Someone Very Special",
"TGFI": "Thank God For the Internet",
"WIIFM": "What's In It For Me",
"SVU": "Special Victims Unit",
"TGFG": "Thank God For Google",
"BUH": "dismay| disgust",
"BUG": "Bother| annoy",
"BUL": "Man| dude| boy",
"BUN": "Smoke marijuana| Ugly",
"YWSYLS": "You Win Some, You Lose Some",
"BUR": "LOL",
"TGFL": "Thank God For Life",
"SWG": "Star Wars Galaxies (game)",
"SWF": "Single White Female",
"YERP": "Yes| what's up| ok",
"SWK": "Star Wars Kid",
"SWM": "Single White Male",
"BW": "Beautiful Woman| Black and White",
"APPS": "Applications (software)",
"HYPER": "Over energetic",
"NICKER": "Pound| quid",
"BVB": "Black Veil Brides (band)",
"IITYWTMWYKM": "If I Tell You What This Means Will You Kiss Me?",
"GRAF": "Graffiti| Paragraph",
"SWT": "Sweat| Sweet",
"SWW": "Slippery When Wet",
"1AAT": "One At A Time",
"SWY": "Sowwy| sorry",
"BVM": "Blessed Virgin Mary",
"MBPS": "MegaBits/Bytes Per Second",
"NICKED": "Stolen| Arrested",
"TWITTER": "A social networking site",
"SXE": "Straightedge",
"SUFI": "Super Finger",
"IGHT": "Alright",
"WEBINAR": "Web based seminar",
"1UP": "Extra life in a game",
"BSTS": "Better Safe Than Sorry",
"BWC": "But Who Cares?",
"FOAF": "Friend Of A Friend",
"BWG": "Big Wide Grin",
"BSTL": "Bam, Said The Lady",
"BWL": "Bursting With Laughter",
"BWM": "Bisexual White Male| Bear With Me",
"SYA": "See You",
"BWT": "Misspelling of BTW",
"SYG": "Shut Your Gob",
"SYF": "Shut Your Face",
"BWW": "Buffalo Wild Wings",
"CACHAI": "Do you understand?",
"SYK": "So You Know",
"SYM": "Shut Your Mouth| So You Mad?",
"TGFT": "Thank God For That",
"SYL": "See You Later",
"*<": "-):Clown",
"CEWL": "Cool",
"DDMMYY": "Day Month Year",
"SYS": "See Ya Soon",
"DHYB": "Don't Hold Your Breath",
"HUFF": "Low grade weed",
"SYT": "See You Tomorrow| See You There",
"SYWISY": "See You When I See You",
"FADED": "High| stoned| drunk",
"FLUFF": "Humorous/romantic fan fiction",
"IGGY": "Ignore",
"MAN U": "Manchester United Football Club",
"NORCAL": "Northern California",
"IHYSM": "I Hate You So Much",
"FOCL": "Falling Off Chair Laughing",
"SORTA": "Sort of",
"BYP": "Beg Your Pardon",
"BYO": "Bring Your Own",
"BYT": "Bright Young Thing| Before Your Time",
"NEXY": "Nerdy And Sexy",
"LASER": "Light Amplification by Stimulated Emission of Radiation",
"NEWB": "Newbie",
"TSGTM": "That SGTM",
"TGIA": "Thank God I'm Alive",
"DUB SACK": "A $20 bag of marijuana",
"TGIM": "Thank God It's Monday",
"TGIF": "Thank God It's Friday",
"FOBR": "Fall Out Boy Rock",
"TGIT": "Thank God It's Thursday",
"TGIO": "Thank God It's Over",
"GAFM": "Get Away From Me",
"NO1CURR": "No-One Cares",
"BBWL": "Be Back Way Later",
"PEMDAS": "Parenthesis, Exponents, Multiply, Divide, Add, Subtract",
"YEYO": "Cocaine",
"RAFL": "ROFL",
"GAGA": "Crazy| infatuated",
"IGMC": "I'll Get My Coat",
"KMSL": "Killing Myself Laughing",
"POTUS": "President of the United States",
"FANFIC": "Fanfiction",
"SUIT": "Businessman| boss",
"LURKER": "Forum follower who never posts",
"GAFF": "House",
"SUM1": "Someone",
"DIAF": "Die In A Fire",
"TWTMC": "To Whom This May Concern",
"BBYG": "Baby Girl",
"WPWW": "White Pride World Wide",
"DIAO": "Cool| hip",
"CUPPA": "Cup of tea",
"KOOLIO": "Cool",
"RAGE": "Party very hard| get wasted",
"TWITTER JAIL": "Having reached your tweeting limit",
"FOFL": "Falling On Floor Laughing",
"IGNB": "I'm Going Now, Bye",
"SULA": "Sweaty Upper Lip Alert",
"RAGO": "Whatever, OK",
"FOFO": "Fourty Four caliber gun",
"MAMAW": "Grandmother",
"TAB": "Acid| LSD| Cigarette",
"TAD": "Small amount",
"TAC": "Cannabis resin",
"TAF": "Tell A Friend",
"TAE": "Trial And Error",
"TAG": "Personal signature",
"CHILLAX": "Chill and relax",
"TAI": "Think About It",
"TAL": "Thanks A Lot| Try Again Later",
"FOIA": "Freedom of Information Act",
"RUDEBOY": "Gangster| Ska fan",
"TAR": "Not very pure heroin",
"TAT": "Tattoo| Rubbish| junk",
"TAV": "Marijuana",
"TAW": "Teachers Are Watching",
"SDMF": "Strength, Determination, Merciless, Forever| Society Dwelling Mother F***er",
"TAY": "Thinking About You",
"TBA": "To Be Announced",
"TBC": "To Be Continued",
"APYR": "As Per Your Request",
"TBE": "To be Edited",
"TBD": "To Be Decided",
"TWITCON": "Profile picture on Twitter",
"TBF": "To Be Fair",
"TBH": "To Be Honest",
"TBM": "Tactical Boyfriend Mention",
"TBS": "Tablespoon| To Be Specified",
"CAB": "Misspelling of CBA",
"TBR": "To Be Real",
"TBU": "To Be Updated",
"CAD": "Computer Aided Design",
"TBT": "Truth Be Told",
"IN THE BAG": "Definite",
"TBX": "To Be Exact",
"CAM": "Computer-Aided Manufacturing",
"CAL": "Cyberathlete Amateur League (gaming)",
"TGOD": "Taylor Gang Or Die",
"TCB": "Taking Care of Business",
"CAP": "Shoot",
"CAS": "Close Air Support| Confidence, Appearance, Swagger| Creativity, Action, Service",
"RAIL": "Line of cocaine",
"DIBS": "A claim",
"CAT": "Cool person| Computed Axial Tomography",
"TCH": "disagreement| frustration",
"HDMI": "High Definition Multimedia Interface",
"TCG": "Trading Card Game",
"PLEB": "Commoner| inferior person",
"TCL": "Tool Command Language",
"24/7": "Twenty-four hours a day, seven days a week",
"CAZ": "Crazy in a nice way",
"TCK": "Third Culture Kid",
"WA GWAN": "What's Going On",
"TCP": "Transmission Control Protocol",
"CBB": "Can't Be Bothered",
"CBD": "Cute But Dumb",
"CBC": "Canadian Broadcasting Corporation",
"CBH": "Child Bearing Hips",
"CBI": "Californian Bureau of Investigation| Confederation of British Industry",
"CBR": "Captive Bead Ring",
"ELI5": "Explain Like I'm 5",
"CBT": "Cognitive Behavioural Therapy",
"CBS": "Can't Be Stuffed",
"YWCA": "Young Women's Christian Association",
"TDG": "Too Damn Good| Three Days Grace (band)",
"TDF": "To Die For",
"TRIPPING": "Under the influence of drugs",
"TDI": "Turbo Direct Injection| Turbo Diesel Injection",
"TDH": "Tall, Dark, and Handsome",
"TACKY": "In bad taste",
"TDM": "Team DeathMatch",
"TDL": "To Do List",
"CCC": "Coricidin Cough & Cold",
"TDS": "Technical Dificulties",
"CCG": "Collectible Card Game",
"TDY": "Temporarily Deployed for a Year",
"CYL8R": "See You Later",
"CCL": "Couldn't Care Less",
"WHATCHA": "What are/do you ...",
"DIDO": "DIDDO",
"SUCKS": "Is bad| is rubbish",
"FOMCL": "Falling Off My Chair Laughing",
"CCR": "Creedence Clearwater Revival (band)",
"TEC": "Tec 9 handgun",
"TEE": "T-shirt",
"CCW": "Counterclockwise| Carrying a Concealed Weapon",
"TEH": "The| Used to emphasise the next word",
"2BH": "To Be Honest",
"TEK": "Tek-9 machine pistol| Technique",
"NFAA": "Not Funny At All",
"TEN": "Person with a perfect body",
"PROLLY": "Probably",
"TF2": "Team Fortress 2",
"CDC": "Cult of the Dead Cow| Crudely Drawn Cock",
"DIGG": "Social bookmarking website",
"FOMO": "Fear Of Missing Out",
"RANK": "Disgusting| gross",
"CDN": "Canadian",
"CDP": "Calm Down Please",
"FOMS": "Fear Of Missing Something",
"PLIX": "Please",
"TFA": "Thanks For Asking",
"MTBF": "Mean Time Between Failures",
"CDR": "Compact Disc Recordable",
"TFC": "Team Fortress Classic",
"DIGS": "Home| lodgings",
"TFI": "Thanks For the Information",
"TFM": "Total Frat Move",
"TFL": "Tip For Life| Transport For London",
"TFP": "Thanks For Playing",
"TFS": "Thanks For Sharing",
"GAMS": "Female legs",
"D&D": "Dungeons and Dragons",
"TFT": "The Frozen Throne| Thin Film Transistor",
"GROK": "Fully understand",
"GROM": "Grommet| young surfer",
"D&M": "Deep and Meaningful",
"CEO": "Chief Executive Officer",
"HURT": "Ugly| unattractive",
"AYRTON": "Tenner| ten pounds",
"CES": "Consumer Electronics Show",
"CET": "Central European Time",
"IGTG": "I Got To Go",
"RROD": "Red Ring Of Death",
"TGE": "the",
"WH40K": "Warhammer Forty-Thousand",
"GANK": "Steal| Gang kill",
"GANJ": "Ganja| marijuana",
"TGI": "Thank God It's ...",
"BANGING": "Very beautifu| sexy",
"SCALLY": "Low life| trailer trash",
"STOPE": "Awesome| cool",
"TGP": "Thumbnail Gallery Post",
"AQAP": "As Quickly As Possible",
"CFC": "Chlorofluorocarbon",
"GRRL": "Girl",
"CFL": "Canadian Football League",
"BCBW": "Bow Chica Bow Wow",
"M2AF": "Message To All Friends",
"CFN": "Ciao For Now",
"ELLO": "Hello",
"GRRR": "Anger| Frustration| Intimacy",
"SDSL": "Symmetric DSL",
"THA": "The",
"CFO": "Chief Financial Officer",
"THC": "Tetrahydrocannabinol| active ingredient in marijuana",
"D'S": "Dayton rims",
"HDTH": "How Did That Happen?",
"CFV": "Call For Votes",
"IGWT": "In God We Trust",
"THO": "Though",
"THN": "Then",
"CGA": "Color Graphics Adapter",
"IH8U": "I Hate You",
"HDTV": "High Definition Television",
"CGI": "Computer-Generated Imagery",
"SUSS": "Discover| understand| Suspicious",
"THX": "Thanks",
"2EZ": "Too Easy",
"ISWYM": "I See What You Mean",
"RAOK": "Random Act Of Kindness",
"TIA": "Thanks In Advance",
"#1": "Number one",
"TID": "Take It Down",
"GAWJUS": "Gorgeous",
"TIC": "Tongue in Cheek",
"TIF": "That Is Funny",
"TIE": "Take It Easy",
"TIL": "Until| Today I Learned",
"TIM": "Time Is Money",
"BTDT": "Been There Done That",
"CHA": "Yes",
"GOLD DIGGER": "Person who loves someone just for their money",
"ICYMI": "In Case You Missed It",
"SCOUSE": "Relating to Liverpool",
"TIS": "It Is",
"TIX": "Tickets",
"CHI": "Chicago Life force| energy",
"IGYB": "I've Got Your Back",
"WTGP?": "Want To Go Private?",
"NWIM": "Not What I Mean",
"CHP": "California Highway Patrol",
"TJA": "Hi",
"ELNS": "Eller noe s",
"NWIH": "No Way In Hell",
"DIKU": "Do I Know You?",
"CHU": "You",
"TGTBT": "Too Good To Be True",
"DIKY": "Do I Know You?",
"ABFAB": "Absolutely Fabulous",
"PICNIC": "Problem In Chair Not In Computer",
"ROTFFNAR": "Rolling On The Floor For No Apparent Reason",
"GRUB": "Food",
"CIA": "Central Intelligence Agency",
"CIC": "'Cause I Can",
"HIT YOU UP": "Call you later",
"CIB": "Complete In Box",
"CID": "Acid| LSD",
"CIG": "Cigarette",
"%)": "Drunk| giddy",
"CIO": "Check It Out",
"TKD": "Taekwondo",
"CIR": "Committed Information Rate (minimum bandwidth)",
"FOPS": "Fake cops (security guards etc)",
"PLMA": "Please Leave Me Alone",
"TKM": "Te Quiero Mucho| I love you",
"PEWPEW": "Laser sound",
"TKO": "Technical Knock Out",
"TKT": "Don't worry",
"TKS": "Thanks",
"PLMK": "Please Let Me Know",
"PLOX": "Please",
"DIMM": "Dual In-line Memory Module",
"PLOS": "Parents Looking Over Shoulder",
"BTHO": "Beat The Hell Outta",
"FOSS": "Free and Open Source Software",
"RATS": "Damn",
"TLA": "Three Letter Acronym",
"TLC": "Tender Loving Care",
"TLD": "Top Level Domain",
"RATM": "Rage Against the Machine (band)",
"TLI": "Too Little Information",
"TLK": "Talk",
"FOTB": "Fresh Off The Boat",
"FOTD": "Friend Of The Day",
"FOTC": "Flight of the Conchords",
"DING": "Just hit new level in RPG",
"CKI": "Chair-Keyboard Interface",
"ELOL": "Evil Laugh Out Loud",
"CKL": "Chuckle",
"TMB": "Text Me Back",
"TMA": "Take My Advice",
"TMD": "Damn in Chinese| Too Much Detail",
"TMF": "Too Much Forehead",
"TME": "Too Much Effort",
"TMH": "Touch My Hand",
"TOMBOY": "Girl who behaves like a boy",
"TMI": "Too Much Information",
"JAMBO": "Hello",
"TML": "Thank Me Later| Text Me Later",
"TMK": "To My Knowledge",
"TMN": "Touch Me Not",
"TMM": "Tell Me More",
"TMP": "Tactical Machine Pistol",
"TMO": "Tomorrow",
"SPECTRE": "Special Executive for Counter-intelligence, Terrorism, Revenge, and Extortion",
"TMR": "Tomorrow",
"TMT": "Too Much Tuna",
"TMS": "Too Much Swag",
"DIME": "$10 of a drug| Very attractive person",
"U.U": "Expressing sympathy",
"TMW": "Tomorrow",
"2K9": "2009",
"HECTIC": "Cool| good",
"PDBAZ": "Please Don't Be A Zombie",
"TMZ": "Thirty Mile Zone",
"CLI": "Command Line Interface",
"TMY": "Tell Me Why",
"FOUO": "For Official Use Only",
"ZING": "Owned",
"BUGLE": "Cocaine",
"CLM": "Career Limiting Move",
"GAWD": "God",
"TNA": "Total Nonstop Action",
"TNB": "There's Nothing Better",
"CLT": "Currently Listening To",
"TNG": "The Next Generation",
"TNF": "That's Not Funny",
"FRESH": "Very good| cool",
"(": ":Happy",
"TNL": "To Next Level",
"TNO": "Trust No One",
"ANZAC": "Australian and New Zealand Army Corps",
"CMC": "Casi Me Cago",
"CMB": "Call Me Back",
"DD FREE": "Disease And Drug Free",
"TOKER": "Someone who smokes marijuana",
"CME": "Close My Eyes",
"NWNC": "No Woman, No Cry",
"CMD": "Command",
"TNT": "Til Next Time",
"2L8": "Too late",
"CMI": "Count Me In",
"CMH": "Cross My Heart",
"TNX": "Thanks",
"DINK": "Dual Income No Kids",
"CML": "Call Me Later",
"FOTM": "Flavor Of The Month",
"CMN": "Call Me Now",
"FOTR": "Fellowship of the Ring",
"HDYD": "How Do You Do?",
"CMS": "Content Management System",
"QOTD": "Quote Of The Day",
"TOD": "Time Of Death| Time Of Day",
"TOC": "Table Of Contents",
"CHEDDA": "Cash| money",
"CMU": "Cracks Me Up",
"CMT": "Comment",
"DINT": "Did not| didn't",
"M2FP": "My Two Favorite People",
"TOH": "The Other Half",
"HDYK": "How Do/Did You Know?",
"TOL": "Thinking Out Loud| Thinking Of Laughing",
"BROLLY": "Umbrella",
"NWLY": "Never Wanna Lose You",
"TON": "One hundred",
"TOM": "Time Of Month",
"CNA": "Certified Nursing Assistant",
"CHIPPY": "Fish and chip shop",
"CND": "Campaign for Nuclear Disarmament",
"RAVE": "A Large Dance Party",
"TOT": "Small child",
"CNC": "Command and Conquer",
"D/C": "Disconnected",
"TOS": "Terms Of Service",
"*$": "Starbucks",
"TOW": "The Other Wiki",
"TOY": "Thinking Of You",
"CNN": "Cable News Network",
"TYFAYS": "Thank You For All Your Support",
"CWOT": "Complete Waste Of Time",
"CO2": "Carbon Dioxide",
"G'DAY": "Good Day, hello",
"TPB": "The Pirate Bay",
"TPE": "Total Power Exchange",
"ROFTL": "ROTFL",
"SWAGG": "Style| coolness| confidence",
"NWOT": "New Without Tags (eBay)",
"2MI": "Too Much Information",
"TPK": "Total Party Kill",
"CNY": "Chinese New Year",
"AFAIR": "As Far As I Remember",
"2MM": "Tomorrow",
"TPO": "Time, Place, Occasion",
"COA": "Certificate Of Authenticity",
"TPS": "Totally pointless stuff",
"COD": "Cash On Delivery| Call Of Duty",
"TPT": "Trailer Park Trash",
"COG": "Coalition of Gears (game)",
"AFAIA": "As Far As I'm Aware",
"COL": "Crying Out Loud",
"AFAIC": "As Far As I'm Concerned",
"COO": "Cool",
"RAWL": "Roll Around While Laughing",
"D12": "Dirty Dozen",
"U2B": "YouTube",
"COP": "Officer of the law| Close of Play",
"RAWK": "Rock",
"+1": "Partner |Me too",
"COS": "Because",
"COR": "God",
"AFAIK": "As Far As I Know",
"COW": "Offensive term for a woman",
"RAWR": "Right Answer, Wrong Reason",
"SWAGGA": "Style or personality",
"COZ": "Because",
"VNDS": "Very Near Dead Stock",
"AIMBOT": "Trageting script in games",
"TQM": "Te Quiero Mucho| I Like You A Lot",
"BRODA": "Bro Who Is Like Yoda",
"CPC": "Cost Per Click",
"MENTHOL": "Mint flavored cigarette",
"U2U": "Up To You",
"CPI": "Consumer Price Index",
"CPL": "Cyberathlete Professional League (gaming)",
"IYAM": "If You Ask Me",
"CPM": "Cost Per Thousand",
"DISO": "Desperately In Search Of",
"CPP": "C Plus Plus| C++ (programming language)",
"THAR": "There",
"TRA": "Bye",
"CPR": "Cardiopulmonary Resuscitation",
"DISS": "Disrespect",
"CPT": "Colored People's Time",
"TRG": "Tiny Rascal Gang",
"CPU": "Central Processing Unit",
"RAZZ": "Make fun of| Get drunk",
"TRM": "That Reminds Me",
"HVAC": "Heating, Ventilating, Air Conditioning",
"CQC": "Close Quarters Combat",
"CQB": "Close Quarters Battle",
"DITD": "Down In The Dumps| sad",
"IYDK": "If You Didn't Know",
"TRU": "True",
"D2F": "DTF",
"IYDM": "If You Don't Mind",
"D2M": "Dead to Me",
"TSB": "True Story Bro",
"IHAC": "I Have A Customer...",
"TSA": "Transportation Security Administration",
"NIMBY": "Not In My Back Yard",
"SNITCH": "Person who tells on someone",
"TSF": "That's So Funny",
"DIRT": "Low grade marijuana| Gossip",
"BROAD": "Woman",
"TSK": "Sound of annoyance",
"TYBG": "Thank You Based God",
"TSN": "The Sports Network",
"TSM": "Thanks So Much",
"TSP": "Teaspoon",
"CRB": "Criminal Records Bureau",
"YANKEE": "American",
"CRC": "Cyclic Redundency Check| Chat Room Chuckle",
"ROLLED": "Robbed| beaten",
"RANGA": "Redhead derogatory",
"PLUR": "Peace, Love, Unity, and Respect",
"CRM": "Customer Relationship Management",
"TTC": "Trying To Conceive",
"P.I.C.": "Partner In Crime",
"CRT": "Cathode Ray Tube",
"IHDK": "I Honestly Don't Know",
"TTD": "Textually Transmitted Disease",
"CRV": "Compact Recreational Vehicle",
"TTG": "Time To Go",
"TTH": "Trying Too Hard",
"TTM": "Talk To Me",
"TTL": "Thank The Lord| Time To Live",
"TTO": "Time To Own",
"CSA": "Cheap Shot Artist",
"DIVA": "Talented, but arrogant, female performer",
"IB4TL": "In Before The Lock",
"HEAT": "Gun(s)",
"TTP": "To The Point",
"CSB": "Cool Story Bro (sarcastic)",
"TTR": "Time To Relax",
"TTT": "Third Tier Toilet| bad school",
"BCNU": "Be Seeing You",
"5EVER": "Because Forever Isn't Long Enough| Longer Than 4ever",
"CSI": "Crime Scene Investigation (TV show)",
"TTY": "Talk To You",
"CSM": "Client Side Modification",
"CSL": "Can't stop laughing",
"LIHOP": "Let It Happen On Purpose",
"GROOVY": "Cool| awesome",
"//": "Love| I love you",
"CAPTCHA": "Completely Automated Public Turing Test To Tell Computers and Humans Apart",
"CSS": "Counter Strike: Source| Cascading Style Sheet",
"CSR": "Customer Service Representative",
"CSU": "California State University",
"CST": "Central Standard Time",
"CSV": "Comma Separated Values",
"TUL": "Tell You Later| Text You Later",
"BU2M": "Been Up To Much?",
"CTD": "Crash To Desktop",
"CTC": "Call The Cell",
"CTF": "Capture The Flag| Claim To Fame",
"BIDDY": "Girl| woman",
"TUX": "The penguin Linux mascot| Tuxedo",
"BCOS": "Because",
"LAND MINE": "Ugly| skinny girl",
"CTN": "Can't Talk Now",
"CTM": "Chuckling To Myself",
"GLOCK": "Semi-automatic pistol",
"0/": "Waving",
"CTO": "Check This Out",
"CTR": "Click Through Rate",
"MTBMW": "May The Best Man Win",
"YWU?": "Yo What's Up?",
"CTS": "Chuckles To Self",
"TVD": "The Vampire Diaries",
"/S": "Sarcasm",
"HECK": "Mild expletive",
"CTU": "Counter Terrorist Unit",
"LATERZ": "See you later, goodbye",
"NONYA": "None of Your (business)",
"/W": "Whisper",
"ROLLIN": "High on drugs",
"TVM": "Thanks Very Much",
"LATERS": "See you later, goodbye",
"SWERVE": "Move On",
"WYWTA": "What You Wanna Talk About",
"CUG": "Closed User Group",
"CUH": "Cousin| homie| bro",
"CUL": "See You Later.",
"CWUC": "Call When You Can",
"CUS": "Friend",
"TWD": "Texting While Driving",
"TWF": "That Was Funny",
"CUT": "See You Tomorrow| See You There",
"TYFE": "Thank You For Everything",
"DIVX": "Video format",
"CUZ": "Because",
"2U2": "To You Too",
"TWT": "Time Will Tell",
"TWS": "Typical Woman Syndrome",
"1D": "One Direction (band)",
"TWU": "That's What's Up",
"IYGM": "If You Get Me",
"NFSW": "Not For Showing Wife",
"CLAPPED": "Shot",
"CVV": "Card Verification Value",
"RBAU": "Right Back At You",
"EYEBALL": "Look at| Estimate measurement by looking",
"BTTT": "Back To The Top",
"RBAY": "Right Back At You",
"CWB": "Crazy White Boys",
"2B": "To Be",
"ISWYDT": "I See What You Did There",
"CWD": "Comment When Done| Cringe, Wither, Die| Chronic Wasting Disease| Chicks With Dicks",
"TXT": "SMS message",
"2G": "Too Good",
"DISTRO": "Distribution",
"HOOKED": "Addicted",
"MTUK": "More Than You Know",
"2M": "Tomorrow",
"BTTF": "Back To The Future",
"TYB": "Try Your Best",
"SEEN": "I see, understood",
"CWT": "Hundred Weight, 100lbs",
"2U": "To You",
"LUVVIE": "Stage actor",
"TYG": "There You Go",
"TYJ": "Thank You Jesus",
"TYL": "Text You Later",
"WULLNP": "Wishing You Long Life And Prosperity",
"TYM": "Thank You Much",
"3>": "Hate",
"TYT": "Take Your Time",
"TYS": "Thank You Sir",
"GMAIL": "Google web email service",
"WTHAY": "Who The Hell Are You?",
"3G": "Third Generation mobile phone network",
"PEEPS": "People",
"GOBLIN": "Gangster",
"PEEPZ": "People",
"3Q": "Thank You",
"IOKIYAR": "It's Okay If You're A Republican",
"TOOTLES": "See you later| goodbye",
"D": ":Suprised| shocked",
"MILEAGE": "Value for money",
"CYB": "Count Your Blessings",
"CYE": "Check Your Email",
"CYF": "Cross Your Fingers",
"4G": "4th Generation",
"CYM": "Check Your Mail",
"CYL": "Catch You Later",
"CWYL": "Chat With You Later.",
"CYO": "See You Online",
"4M": "Forum",
"GAWJUSS": "Gorgeous",
"L8R G8R": "Later Gator",
"LQTM": "Laughing Quietly To Myself",
"(YN)": "Fingers crossed (MSN)",
"CYT": "See You Tomorrow",
"4U": "For You",
"M33T": "Meet",
"KNOT": "Roll of paper money",
"LGBTQ": "Lesbian, Gay, Bisexual, Transexual, Questioning",
"W00T": "Hurrah",
"5N": "Fine",
"GBFN": "Goodbye for Now",
"LEMME": "Let Me",
"IHML": "I Hate My Life",
"MTYK": "More Than You Know",
"SKEEN": "I See| Seen",
"TYLJ": "Thank You Lord Jesus",
"LQTY": "Laughing Quietly To Yourself",
"8)": "Cool",
"8*": "Kiss",
"KTHNXBI": "OK, Thanks, Bye",
"SKEED": "High",
"SEHR": "Very",
"EMAW": "Every Man a Wildcat",
"LULAS": "Love You Like A Sister",
"HUNDOS": "Hundreds",
"T^T": "Crying",
"8D": "Manic",
"SEKC": "Sexy",
"LULAB": "Love You Like A Brother",
"THNX": "Thanks",
"PWEASE": "Babyish form of Please",
"GLOMP": "Hug very hard",
"T_T": "Crying",
"HORNY": "Turned on",
"WALAHI": "I swear to God",
"IHNC": "I Have No Clue",
"UAC": "User Account Control",
"IHNI": "I Have No Idea",
"UAE": "United Arab Emirates",
"WYSIWYG": "What You See Is What You Get",
"UAK": "You Already Know",
"UOENO": "You Don't Even Know",
"PHOAR": "lust| physical attraction",
"SYOTOS": "See You On The Other Side",
"UAN": "Universal Access Number",
"8|": "Balls To The Wall",
"UB2": "You Be Too",
"UAV": "Urban Assault Vehicle",
";(": "Crying",
";)": "Wink",
"THE BOMB": "Really cool",
"ROXORZ": "Rocks",
"UBD": "User Brain Damage",
";;": "Sad| Confused",
"IHOP": "International House of Pancakes",
"TYPO": "Typing mistake",
"DAB": "Digital Audio Broadcasting",
"<#": "Misspelling of <3",
"MODEM": "Modulator-Demodulator",
";D": "Wink",
"DAE": "Does Anybody Else",
"DAG": "Damn| Unfashionable person",
"APPROX": "Approximately",
"DAM": "Don't Ask Me",
"DAP": "Touching of fists greeting",
";O": "Joking",
";P": "Winking and sticking tongue out",
"DAQ": "Don't Ask Questions",
"<2": "Not quite love (<3)",
"DAT": "That",
"<3": "Love (heart shape)",
"<4": "More than love (<3)",
"DAW": "Digital Audio Workstation",
"THOU": "You",
"THOT": "HOE",
"<>": "Not equal| No comment",
"DBA": "Don't Bother Asking",
"IYTE": "All right",
"DBC": "Dime By Comparison",
"NACKER": "Irish gypsy| traveller",
"<B": "Hate",
"DBD": "Don't Be Dumb| Death Before Dishonor",
"WTHIT": "What The Hell Is That?",
"DBF": "Dat Be Fair",
"DBI": "Douche Bag Index| Don't Beg It",
"DBJ": "Don't Be Jealous",
"DBM": "Don't Bother Me",
"DBL": "Don't Be Late",
"THRU": "Through",
"JAILBREAK": "Crack an iPhone",
"IYTT": "If You Think That",
"WOMBAT": "Waste Of Money, Brains, And Time",
"UDA": "Ulster Defence Association",
"DBS": "Don't Be Stupid| Don't Be Sad| Don't Be Scared",
"DBR": "Damaged Beyond Repair",
"DBT": "Doing Big Things",
"FTLOG": "For The Love Of God",
"GBML": "Good Bye My Love",
"UDI": "Unidentified Drinking Injury",
"DBZ": "Dragon Ball Z",
"UDK": "You Don't Know",
"DNFTT": "Do Not Feed The Troll",
"PHWOAR": "fit| sexy| hot",
"UDS": "Ugly Duckling Syndrome",
"DCF": "Department of Children and Families",
"HORSE": "Heroin",
"TNSTAAFL": "There's No Such Thing As A Free Lunch",
"DCI": "Drum Corps International",
"IYSS": "If You Say So",
"DCT": "Dream Come True",
">3": "Evil but happy|Hate",
"PEERS": "People in the same group, equals in age, background etc",
"TYSM": "Thank You So Much",
"CMIYC": "Catch Me If You Can",
"><": "Frustration",
"UES": "Upper East Side",
"DDB": "Deaf, Dumb, Blind",
"DDG": "Drop Dead Gorgeous",
"HAXXOR": "Hacker",
"DDF": "Drug and Disease Free",
"DDI": "Don't Do It",
"HERB": "Stupid person",
"PEBCAK": "Problem Exists Between Chair And Keyboard",
"THTH": "To Hot to Handle",
"WHEEL": "Chat up| pick up",
"SKENG": "Knife| blade| Gun| firearm",
"DDR": "Dance Dance Revolution (Game)",
"UFC": "Ultimate Fighting Championship",
"UFF": "annoyance",
"DDT": "Don't Do That",
"RSPB": "Royal Society for the Protection of Birds",
"UFG": "Up For Grabs",
"JTUSK": "Just Thought You Should Know",
"PMJI": "Pardon My Jumping In",
"NTTAWWT": "Not That There's Anything Wrong With That",
"TYVM": "Thank You Very Much",
"PLASTIC": "Credit Card",
"UFO": "Unidentified Flying Object",
"??": "What?",
"DEB": "Don't Even Bother",
"DEA": "Drug Enforcement Administration",
"BONE UP": "Learn about| revise",
"DED": "Dead from joy| hilarity etc",
"DEC": "Decent",
"PENDEJO": "Idiot| fool",
"5 BY 5": "Loud and clear, fine",
"BUCK": "Fight| Money",
"DEF": "Cool| great| excellent",
"GSOH": "Good Sense Of Humor",
"BDAY": "Birthday",
"DEH": "DOH",
"THT": "That",
"HASIAN": "Hot Asian",
"MUAH": "The sound of giving a kiss",
"DEN": "Then| Private room",
"DEM": "Them",
"DER": "Duh!",
"UGC": "User Generated Content",
"DET": "Don't Even Trip",
"GBNF": "Gone But Not Forgotten",
"BUBZ": "Pet name for BF or GF",
"UGG": "Ugly",
"3DG": "Three Days Grace (band)",
"MUAK": "MUAH",
"PEBCAC": "Problem Exists Between Chair And Computer",
"UGH": "disappointment or disgust",
"7DAW": "Seven Days A Week",
"DEY": "They",
"BUBS": "Babe",
"SESH": "Session",
"DFE": "Dead Fish Experience",
"DFF": "Don't Forget to Flush",
"DFI": "Don't Force It",
"DFM": "Dance Floor Makeout",
"DFO": "Done Fell Out",
"A1": "Top quality",
"DFS": "Dating For Sport",
"A3": "Anywhere, Any time, Any place",
"UHF": "Ultra High Frequency",
"DYJHIW": "Don't You Just Hate It When...",
"DFW": "Down For Whatever",
"CHEERIO": "Goodbye",
"NO WORRIES": "OK, no problem",
"UHM": "UH| UM",
"UHQ": "Ultra High Quality",
"AA": "Alcoholics Anonymous| African-American| Automobile Association",
"UHT": "Ultra High Temperature",
"THUG": "Violent criminal| Person struggling to make something of their life",
"AC": "Air Conditioning| Alternating Current",
"AD": "Anno Domini",
"UHU": "Misspelling of HUH",
"AE": "American Eagle",
"B&": "Banned| Band",
"AF": "As Fuck",
"AG": "Aggressive",
"ATGATT": "All The Gear All The Time",
"B)": "Smiley with sunglasses",
"DGI": "Don't Get Involved| Don't Get It",
"AI": "Artificial Intelligence|As If",
"AJ": "Apple Juice",
"DGK": "Dirty Ghetto Kids (skateboards)",
"TTYITM": "Talk To You In The Morning",
"AK": "AK-47 Assault Rifle| Alaska",
"AL": "Alcohol",
"AO": "Adults Only",
"LSHIC": "Laughing So Hard I'm Crying",
"POPPER": "Amyl Nitrate",
"AP": "Associated Press",
"AQ": "Adventure Quest (game)",
"DGT": "Don't Go There",
"AR": "Assault Rifle",
"UID": "User Identification",
"B4": "Before",
"IYWW": "If You Were Wondering",
"AU": "Alternate Universe| Gold",
"ICBINB": "I Can't Believe It's Not Butter",
"MUCK": "Ugly",
"AV": "Anti-Virus| Audio/Video| Adult Video",
"B7": "Banned (B7:B&:Band)",
"SERP": "Search Engine Results Page",
"THUR": "There",
"TWIHARD": "Die-hard Fan Of The Twilight Saga",
"AX": "Ask",
"WRUTA": "What Are You Talking About?",
"UIN": "Unique Identification Number",
"BB": "Big Brother| Be Back",
"C#": "Microsoft programming language",
"BC": "Before Christ",
"MUFC": "Manchester United Football Club",
"BD": "Big Deal",
"BF": "Boyfriend| Best Friend",
"LOLZA": "LOL",
"BG": "Bad Game| Baby Gangster",
"BH": "Be Happy| Bloody Hell| Big Head",
"GSTQ": "God Save The Queen",
"BK": "Burger King",
"DHL": "Shipping Company",
"BL": "Bad Luck| Bottom Line",
"BM": "Bowel Movement| Black Man| BMW",
"BUFF": "Good Looking",
"BO": "Body Odour",
"C1": "Affirmative| Roger that",
"DHS": "Department of Homeland Security| Department of Health Services",
"BQ": "Be Quiet",
"BR": "Best Regards",
"C4": "Plastic explosive",
"BT": "BitTorrent| British Telecom",
"BU": "Backup",
"DHV": "Demonstration of Higher Value",
"BV": "Bad Vibe",
"BUEY": "similar to DUDE",
"FIZZLE": "Fool",
"WAAM": "What's going on| what's up",
"DIC": "Do I Care?",
"WAAN": "Want",
"CC": "Credit Card| Carbon Copy| Crowd Control| Country Code",
"CD": "Compact Disc| Cross Dresser",
"SETI": "Search For Extraterrestrial Intelligence",
"CE": "Casual Encounters",
"DIG": "Understand| Enjoy| like",
"CG": "Computer Graphics",
"SETE": "Smiling Ear To Ear",
"+REP": "Boost reputation",
"CI": "Confidential Informant",
"DIL": "Daughter-In-Law",
"CK": "CRIP Killa",
"DIN": "Do It Now",
"SIMMER": "Relax| chill",
"CL": "Chain Letter| Craig's List",
"CM": "Call Me",
"DIP": "Leave",
"CO": "Commanding Officer| Corrections Officer| Company| Counter Offer",
"DIT": "Details In Thread",
"CR": "Credit| Copyright| Carriage Return",
"DIS": "Did I Say",
"CS": "Counter-Strike",
"DIV": "Stupid person",
"CU": "See You",
"CV": "Curriculum Vitae| resume",
"MANKY": "Nasty| dirty",
"CW": "Clockwise| Continuous Wave",
"D8": "Date",
"2MORO": "Tomorrow",
"DIY": "Do It Yourself",
"THWU": "To Hell With You",
"CZ": "Condition Zero (game)",
"D;": "Crying",
"DA": "The District Attorney| Dad",
"DB": "Douche Bag",
"DC": "Don't Care",
"UKT": "You Know That",
"BOGAN": "Poorly educated| vulgar person",
"DE": "The",
"DF": "Dumbass Frankenstein| WTF",
"DG": "Don't Go",
"DH": "Darling Husband",
"DJ": "Disc Jockey| Dow Jones| Dinner Jacket",
"DJM": "Don't Judge Me",
"DK": "Don't Know",
"DL": "Down Low| Download",
"DM": "Don't Mind| Doesn't Matter| Direct Message",
"DN": "Doing Nothing",
"E1": "Everyone",
"DQ": "Disqualified",
"DR": "Double Rainbow",
"RBTL": "Read Between The Lines",
"DS": "Darling Son",
"DT": "Down Town",
"DW": "Don't Worry",
"DX": "D-generation X",
"DY": "DWAI",
"DZ": "Drop Zone",
"ULM": "You Love me",
"E>": "<3",
"WACK": "Low quailty| lame",
"ADMIN": "Administrator on forum, network etc",
"EA": "Electronic Arts",
"HICKEY": "Love bite",
"ULT": "Last month",
"EB": "Eyeball",
"RSVP": "Repondez s'il vous plait (Please reply)",
"EG": "For example| Evil Grin",
"EH": "What? Huh?",
"EI": "Eat It",
"POPPIN": "Going on| happening",
"DKM": "Don't Kill Me",
"EL": "Chicago's elevated rail system",
"DKP": "Dragon Kill Points",
"EM": "E-Mail| Them",
"EP": "Extended Play",
"F1": "Formula One",
"EQ": "EverQuest (game)",
"ER": "Emergency Room| Indicating hesitation",
"DAMAGE": "Cost of something",
"DILLIC": "Does It Look Like I Care",
"ET": "Extra Terrestrial",
"F5": "Refresh",
"EU": "European Union",
"EV": "Expected Value",
"EW": "Exclamation of disgust",
"F8": "Fate",
"DKY": "Don't Know Yet| Don't Kid Yourself",
"EX": "Former",
"F9": "Fine",
"EY": "Hey",
"HINKEY": "Strange| unusual",
"EZ": "Easy",
"GBTM": "Get Back To Me",
"COOKIE": "Small file relating to a website",
"DLC": "Downloadable Content",
"F@": "Fat",
"DLB": "Don't Look Back",
"FC": "Fingers Crossed| Full Combo",
"FE": "Fair Enough",
"GBTW": "Get Back To Work",
"V.V": "Sad",
"FF": "Final Fantasy",
"FG": "Fat Girl| Family Guy",
"NECRO": "Revive a very old forum thread",
"FI": "Forget It",
"FJ": "Foot Job",
"DLL": "Dynamic Link Library",
"B-DAY": "Birthday",
"FL": "Fake Laugh",
"DLN": "Don't Look Now",
"FM": "Freqeuncy modulation",
"STATS": "Statistics",
"DLS": "Dirty Little Secret",
"FP": "Fair Play",
"G1": "Good One| Gone",
"WDITOT": "Why Didn't I Think Of That",
"ANFSCD": "And Now For Something Completely Different",
"ARGH": "frustration| anger",
"FR": "For Real",
"DLV": "Demonstration of Lower Value",
"FW": "Forward",
"G8": "Great",
"UNI": "University",
"FX": "Effects",
"G9": "Good Night",
"UNK": "Crackhead",
"UNO": "You know",
"DMB": "Dave Matthews Band",
"V/R": "Very Respectfully",
"GA": "Go Ahead",
"PITME": "Peace In the Middle East",
"DMC": "Deep Meaningful Conversation",
"GB": "Great Britain| Gigabyte",
"GBWY": "God Be With You",
"GC": "Good Contition",
"CHIRPSE": "Flirt",
"GD": "Gangsta Disciple",
"GE": "Good Evening",
"DMG": "Damage",
"GF": "Girlfriend",
"GG": "Good Game",
"DMI": "Don't Mention It",
"GH": "Guitar Hero",
"BUKU": "A lot of",
"GI": "Government Issue| American Solider| Good Idea",
"GJ": "Good Job",
"DMN": "Damn",
"GK": "Good Kid",
"DMM": "Don't Mind Me",
"GL": "Good Luck| Good Looking",
"GM": "General Motors| Good Morning| Genetically Modified| Good Move",
"IMYSM": "I Miss You So Much",
"GN": "Good Night",
"QQ4U": "Quick Question For You",
"SEXT": "Send suggestive text messages",
"GP": "General Principle",
"DMT": "Di-Methyl Tryptamine (drug)",
"GQ": "Gentlemen Quarterly| Suave| well dressed man",
"DMS": "Dirty Money Syndicate",
"GR": "Gotta Run",
"H3": "Halo 3 (Game)",
"DMV": "Department of Motor Vehicles",
"GT": "Grand Touring| Gran Turismo",
"H5": "High Five",
"DMX": "Dark Man X (rapper)",
"GU": "Grow Up",
"DMW": "Dead Man Walking",
"GV": "Good Vibes",
"DMZ": "De-militarized zone",
"GW": "Good Work",
"H8": "Hate",
"WADR": "With All Due Respect",
"DMY": "Don't Mess Yourself",
"H9": "Really hate",
"HEYO": "Hello",
"GZ": "Congratulations",
"DNA": "Deoxyribonucleic Acid| Did Not Attend| Does Not Apply",
"DNC": "Democratic National Committee",
"DNB": "Drum And Bass",
"HA": "Sarcastic laugh",
"DNE": "Do Not Erase| Does Not Exist",
"HB": "Hurry Back",
"DND": "Do Not Disturb",
"HC": "Holy Cow",
"HEYY": "Hey You",
"DNF": "Did Not Finish",
"HF": "Have Fun",
"HG": "Holy Grail",
"DNK": "Do Not Know",
"HH": "Holding Hands",
"HI": "Hello",
"DNM": "Does Not Matter| Deep and Meaningful",
"DNO": "Don't Know",
"HL": "Half Life (game)",
"HM": "Thinking",
"MULE": "Person who carries something for someone else",
"DNP": "Do Not Post",
"HO": "Hold On| Head Office",
"DNS": "Domain Name Service| Did Not Start",
"HP": "Harry Potter",
"DNR": "Do Not Resuscitate",
"HQ": "High Quality",
"HR": "Human Resources",
"DNT": "Don't",
"HS": "Headshot",
"DNW": "Do Not Want",
"HT": "Hat Tip| Thanks| Home Time| Handheld Transceiver",
"HU": "Hollywood Undead (Band)",
"UPG": "Upgrade",
"HW": "Homework",
"I8": "Aight| aite| alright",
"HY": "Hell Yes| Hi",
"HZ": "Hertz",
"MULU": "Miss You, Love You",
"DOB": "Date Of Birth",
"DOA": "Dead On Arrival| Deteriorate On Approach",
"CIGGIE": "Cigarette",
"DGTWM": "Don't Go There With Me",
"DOD": "Day of Defeat",
"IA": "I Agree",
"UPT": "Uptown",
"DOC": "Doctor| Drug of Choice",
"IB": "I'm Back",
"UPS": "United Parcel Service| Uninterruptible Power Supply",
"DOF": "Depth of Field",
"IC": "I See",
"DOE": "Depends On Experience",
"ID": "Identity",
"DOH": "frustration| realization",
"IE": "Internet Explorer| Id Est",
"DOG": "Telephone| Ugly person",
"HANG UP": "Inhibition| emotional difficulty",
"IG": "Ignorant",
"THIZZING": "Tripping on Ecstasy",
"WAGS": "Wives And Girlfriends",
"DOI": "Duh",
"DOL": "Dying Of Laughter",
"IJ": "I'm Joking",
"LAFS": "Love At First Sight",
"DON": "Top man| Mafia boss",
"FINNA": "Going To",
"IK": "I Know",
"IL": "In Love",
"IM": "I am",
"BUTTERFACE": "A Woman With An Attractive Body But An Ugly Face",
"IP": "Internet Protocol| Intellectual Property",
"DOT": "Damage Over Time",
"IQ": "Intelligence Quotient",
"AIKRN": "All I know right now",
"BUMP": "Bring Up My Post",
"DOS": "Disk Operating System| Denial Of Service| Dad Over Shoulder",
"IR": "Infrared| Injured Reserve",
"DOX": "Documents",
"DOW": "Dawn Of War (game)| Dow Jones Industrial Average",
"IV": "Four| Intravenous",
"GWAAN": "Go on",
"PG-13": "Parental Guidance if under 13 (movie rating)",
"IZ": "Is| it's",
"SOCOM": "Special Operations Command",
"LAGG": "Defeat| PWN",
"JA": "Yes",
"JB": "Jail Bait| Jonas Brothers",
"JC": "Jesus Christ| Just Curious| Just Chilling",
"MULA": "Money",
"JD": "Jack Daniel's",
"DPI": "Don't Push It",
"WAG1": "What's Going On?",
"JG": "Juicy Gossip",
"JI": "Very| sort of",
"YGLT": "You're Gonna Love This",
"JJ": "Just Joking",
"JK": "Just Kidding",
"COALBURNER": "White woman that dates black men",
"JM": "Just Messing",
"JN": "Just Now",
"MHBFY": "My Heart Bleeds For You",
"JO": "Yo",
"MUNG": "Mash Until No Good",
"URA": "You Are A ...",
"DPS": "Damage Per Second",
"JP": "Just Playing",
"K1": "The best| awesome",
"BUNS": "Buttocks",
"DPU": "Data Processing Unit",
"DPT": "Dipropyltryptamine",
"JS": "Just Saying",
"TO BOOT": "As well, also",
"JV": "Junior Varsity| amateur",
"JW": "Just Wondering",
"URI": "Uniform Resource Indicator| URL",
"URL": "Uniform Resource Locator| internet address",
"MDMA": "3,4-Methylenedioxymethamphetamine",
"URM": "You Are Mad",
"URP": "Vomit| throw up",
"NC-17": "No Children under 17 (movie rating)",
"KB": "Kind Bud| Kilobytes",
"KD": "Kraft Dinner",
"KF": "Kinda Funny",
"URW": "You Are Welcome",
"KG": "Kilogram",
"KH": "Kingdom Hearts (RPG)",
"KI": "Kilogram",
"KJ": "Killjoy",
"KK": "OK, cool",
"KL": "Cool",
"KM": "Keep Mum",
"USA": "United States of America",
"KO": "Knock Out| OK",
"USC": "University of Southern California",
"KP": "Korean Pride",
"USB": "Universal Serial Bus",
"KR": "Kind Regards",
"USD": "United States Dollar",
"WAHM": "Work At Home Mom",
"KS": "Kill Steal",
"L7": "Square",
"KW": "Kilowatt",
"L8": "Late",
"TIAB": "To Infinity And Beyond",
"MINGING": "Dirty| disgusting| ugly",
"USP": "Unique Selling Point",
"USS": "United States Ship",
"LA": "Cantonese Exclamation",
"LC": "Lower Case",
"M$": "Microsoft",
"LD": "Link Dead| Disconnection From Internet",
"USW": "Und So Weiter| ETC",
"LE": "Limited Edition| Law Enforcement",
"V4V": "Vote For Vote",
"LF": "Looking For",
"LG": "Little Girl",
"LH": "Living Hell",
"LI": "Laughing Inside",
"DRM": "Digital Rights Management",
"LK": "LOLK",
"DRO": "HyDROponically grown marijuana",
"LL": "Laugh Loudly| LOL",
"LM": "Loud Mouth",
"LO": "Hello| Little One",
"LP": "Long Play",
"UTD": "Up To Date",
"LQ": "Liquor| Lover's Quarrel",
"M2": "Me Too",
"UTC": "Coordinated Universal Time| Under The Counter",
"M3": "BMW sports car",
"UTF": "Unicode Transformation Format",
"DRT": "Dead Right There",
"LS": "Lovesick| Life Story",
"UTE": "Utility vehicle| pickup truck",
"LALB": "Laughing A Little Bit",
"UTH": "Up The Hoods",
"VOIP": "Voice Over IP",
"LU": "Love You",
"LV": "Louis Vuitton| Las Vegas| Level",
"LW": "Lucky Win",
"M8": "Mate",
"UTI": "Urinary Tract Infection",
"LY": "Love You",
"LZ": "Landing Zone",
"UTP": "Up-Town Projects",
"UTO": "Unable To Obtain",
"UTR": "Under The Radar",
"DIZZY": "Crazy| mad",
"DSD": "Do Some Damage",
"MA": "Mom alert| Master of Arts",
"UTT": "Under The Table",
"MB": "Megabyte",
"UTS": "Under The Skin",
"MC": "Master of Ceremonies",
"MD": "Doctor of Medicine| Managing Director",
"TEKKERS": "Technique",
"SFAM": "Sister From Another Mister",
"MG": "Machine Gun| Milligram",
"JUGGALO": "Fan of ICP",
"MH": "Map Hack",
"DSL": "Digital Subscriber Line",
"MJ": "Michael Jackson| Michael Jordan| Marijuana",
"MK": "Mm OK",
"ML": "Much Love",
"DSP": "Digital Signal Processing",
"MM": "Married Man",
"MO": "Modus Operandi| method of operation",
"YGPM": "You've Got Private Message",
"MP": "Military Police| Member of Parliament",
"N1": "Nice One",
"DST": "Daylight Saving Time",
"DSS": "Department of Social Services",
"MR": "Mentally Retarded",
"MUDKIP": "Water Pokemon",
"N3": "No Name Noob",
"MS": "Microsoft| Maple Story",
"MT": "Mistell",
"MU": "Multi User",
"MV": "Music Video",
"SWAK": "Sealed With A Kiss",
"N8": "Night",
"MX": "Motocross",
"DTA": "Don't Trust Anyone",
"SWAT": "Special Weapons And Tactics",
"DTC": "Down To Cuddle",
"DTB": "Don't Text Back",
"NA": "Not Available| Not Applicable| Not Announced",
"PHR33": "Free",
"DTE": "Down To Earth",
"NB": "Not Bad| Nota Bene| Please Note",
"DTD": "Drunk Till Dawn| Drunk Till Death",
"NC": "No Comment| No Choice",
"ND": "And",
"NE": "Any",
"NF": "Not Funny",
"DTH": "Down To Hang",
"NG": "No Good",
"NH": "Nice hand",
"NI": "No Idea",
"DTM": "Doing Too Much| Dead To Me",
"NJ": "Not Joking| Nice Job",
"DTL": "Does The Look",
"NK": "No Kidding| Not Known",
"B4YKI": "Before You Know It",
"NL": "Netherlands| No Limit",
"NM": "Never Mind| Nothing Much| No Message",
"NN": "Night Night| No Need| Not Nice",
"O/": "High five",
"DTP": "Disturbing The Peace",
"NO": "No Offence",
"WAML": "With All My Love",
"DTS": "Down The Shore| Down To Snuggle",
"NP": "No Problem",
"DTR": "Define The Relationship",
"NR": "No Reply| No Reserve",
"DTT": "Don't Touch That",
"MURK": "Kill| murder",
"NT": "No Thanks",
"NU": "No",
"NV": "Never",
"O7": "Salute",
"NW": "No Way",
"PILLOCK": "Idiot| fool",
"LABTOP": "Laptop",
"NY": "New York",
"BURR": "Cold",
"GTBB": "Good To Be Back",
"WAMH": "With All My Heart",
"DUB": "Twenty dollars",
"LANK": "Very",
"OA": "Over Acting| Over Achiever",
"TIFF": "Argument| disagreement",
"BURK": "Idiot| fool",
"OB": "Oh, Brother",
"OC": "Of Course| Original Character| Orange County",
"OD": "Overdose| Over Doing it",
"DUH": "Of course",
"OE": "Outlook Express| Operator Error",
"OG": "Original Gangster| Old Git",
"TARDIS": "Time And Relative Dimentions In Space",
"BURE": "Girl| woman",
"DUI": "Driving Under the Influence",
"MUSH": "Friend",
"OH": "Other Half",
"P)": "Pirate",
"BURD": "Bird| girlfriend",
"OI": "Hey",
"TIFN": "That's It For Now",
"OJ": "Only Joking",
"DUN": "Son",
"OK": "Only Kidding",
"DUM": "Dumb| stupid",
"OL": "Online| Office Lady",
"OM": "Old Man",
"DUO": "Do Unto Others",
"DUR": "Duh",
"GTA5": "Grand Theft Auto 5",
"OO": "Object Orientated| Suprised",
"OP": "Original Poster",
"DUS": "Does",
"OS": "Operating System",
"OT": "Off Topic",
"OU": "Open University",
"ARPU": "Average Revenue Per User",
"OW": "Ouch| Other Woman| Oh, Well",
"OX": "Sharp blade",
"OY": "Oh Yeah",
"KOKO": "Most important thing",
"OZ": "Australia| Ounce",
"WALE": "Rapper",
"TIDY": "Cool| good",
"LAME": "Stupid| Unoriginal",
"UWS": "Upper West Side",
"PA": "Personal Assistant",
"TIEF": "Thief| thieve",
"PB": "Personal Best",
"UWU": "Happy Anime Face",
"DVD": "Digital Versatile Disc",
"PC": "Politically Correct| Personal Computer",
"LAMO": "LMAO",
"PD": "Public Domain",
"RED COAT": "British soldier| Holiday camp worker",
"LAMN": "Look At Me Now",
"PE": "Physical Education",
"DVI": "Digital Visual Interface",
"PG": "Parental Guidance",
"PH": "Pantyhose",
"PI": "Greek letter| number",
"PJ": "Poor Joke| Cheap alcoholic beverage",
"TIEM": "time",
"LAMP": "Hit",
"PK": "Player Kill",
"ARPA": "Advanced Research Projects Agency",
"PL": "OK",
"Q1": "First Quarter| Quake 1",
"DVR": "Digital Video Recorder",
"PQ": "Party Quest (game)",
"PR": "Public Relations",
"DVT": "Deep Vein Thrombosis",
"PS": "PhotoShop| Play Station| Post Script",
"SKYPE": "Internet telephone",
"PT": "Phantasy Tour| Physical Training",
"PV": "Promotional Video",
"Q8": "Kuwait",
"PX": "Post Exchange| Part Exchange",
"LAPD": "Los Angeles Police Department",
"PZ": "Peace",
"HWGA": "Here We Go Again",
"DWB": "Driving While Black",
"DWA": "Driving While Asian",
"QA": "Quality Assurance| Question and Answer",
"QB": "Quarter Back",
"DWF": "Divorced White Female",
"QC": "Quality Control",
"DWI": "Deal with it",
"DWL": "Dying With Laughter",
"HFFA": "Hot From Far Away",
"QI": "Quite Interesting",
"QK": "Ninja",
"DWM": "Divorced White Male",
"QL": "Cool| Quietly Laughing",
"DWP": "Drunk While Posting",
"QP": "Quarter Pound (of weed)",
"DWT": "Driving While Texting",
"QQ": "Crying| Quit",
"QR": "Queer",
"QS": "Quick Scope",
"DWU": "Don't Wait Up",
"QT": "Cutie",
"R5": "Region 5 (DVD region)",
"SWED": "Smoke Weed Every Day",
"YGTR": "You Got That Right",
"MUSM": "Miss You So Much",
"YGTI": "You Get The Idea",
"ROM COM": "Romantic Comedy",
"DXC": "Deathcore (music style)",
"RA": "Resident Advisor| Resident Assistant",
"RC": "Random Chat| Radio Controlled",
"RD": "Real Deal",
"RE": "Regarding",
"RH": "Road Head",
"HUCKLEBERRY": "The man you're looking for",
"RL": "Real Life",
"RM": "Remake",
"RN": "Right Now| Registered Nurse",
"S/": "Substitute first word with second",
"(_8^(I)": "Homer Simpson",
"RO": "Ragnarok Online (MMORPG)",
"RP": "Role Play",
"HTSYS": "Hope To See You Soon",
"RQ": "Random Questions",
"S2": "Love",
"AARP": "American Association of Retired Persons",
"MUDSHARK": "White woman dating black men",
"RR": "Restroom",
"RS": "RuneScape",
"HFGL": "Have Fun, Good Luck",
"JLMK": "Just Let Me Know",
"RT": "Retweet| Real Time",
"RU": "Are You?",
"RV": "Recreational Vehicle",
"RW": "Read-Write",
"RX": "Prescription",
"DYD": "Don't You Dare",
"SA": "Sibling Alert",
"SB": "Somebody",
"LARP": "Live Action Role Play",
"SC": "StarCraft (game)| Silent Chuckle",
"SD": "Sweet Dreams| Secure Digital| So Drunk",
"SE": "Special Edition",
"SF": "Science Fiction| San Francisco",
"DYJ": "Do Your Job",
"FOOTY": "Football| soccer",
"LART": "Luser Attitude Readjustment Tool",
"SH": "Same Here",
"DYL": "Do You Like...?",
"SI": "Stop It| Stupid Idiot",
"DYK": "Did You Know",
"SJ": "Society of Jesus| San Jose",
"DYM": "Do You Mind| Do You Mean",
"GARMS": "Clothes",
"SL": "So Lame",
"SN": "Screen Name| Side Note",
"FRIED": "High on drugs",
"WASTEMAN": "Worthless person| waste of space",
"DYT": "Don't You Think?",
"SQ": "Sound Quality",
"DYS": "Do You See?",
"SS": "Screen Shot",
"ST": "Something",
"KOOL": "Cool",
"SU": "Shut Up",
"SW": "So What?| Star Wars| Street Worker",
"SY": "Sincerely Yours",
"TIIC": "The Idiots In Charge",
"TTMAB": "TO THE MOON AND BACK",
"S^": "What's Up",
"TA": "Thank you",
"TB": "Text Back",
"TC": "Take Care",
"TD": "Touchdown",
"TG": "Thank God| Transgender| Too Good",
"TI": "Texas Instruments| Rapper from Atlanta",
"TK": "Team Killer",
"TL": "Talk Later| True Love| Team Leader",
"TM": "Text Message| Text Me| Trademark",
"MUVM": "Miss You Very Much",
"TP": "Toilet Paper| Town Portal",
"TQ": "Thank You",
"U2": "You Too",
"TR": "Triple Rainbow",
"TT": "Telegrahic Transfer| Till tomorrow",
"LATA": "LATERS",
"WRUD": "What Are You Doing?",
"TW": "Trigger Warning",
"TX": "Thanks",
"LATE": "Missed a period| possibly pregnant|Dead",
"TY": "Thank You",
"LATM": "Laughing At The Moment",
"WASP": "White Anglo-Saxon Protestant",
"RE-GIFT": "Pass an unwanted gift on to someone else",
"UG": "Ugly",
"UH": "hesitation",
"THIZZ": "Ecstasy",
"UI": "User Interface",
"GTHO": "Get The Hell Out",
"UK": "Are You OK?| United Kingdom",
"UL": "Upload",
"UM": "hesitation",
"TWEET": "A post on Twitter",
"UN": "United Nations",
"UO": "Ultima Online",
"UR": "Your| You Are",
"US": "United States",
"UT": "Unreal Tournament",
"UV": "Unpleasant Visual",
"UW": "You're Welcome",
"V8": "Engine type| Vegetable drink",
"RTFI": "Read The Flipping Instructions",
"UX": "User eXperience",
"UY": "Up Yours",
"RTFD": "Read The F***ing Description",
"BIDDIE": "Good looking girl",
"SWIR": "So Wrong, It's Right",
"BTDUBS": "BTW",
"VA": "Various Artists",
"WWCND": "What Would Chuck Norris Do?",
"GTGG": "Got To Get Going",
"VC": "Voice Chat",
"RTFM": "Read The Flipping Manual",
"SWIM": "Someone Who Isn't Me",
"VF": "Vampire Freaks",
"VG": "Video Game| Very Good",
"SWIZ": "Con| rip-off",
"SWIY": "Someone Who Isn't You",
"ARVO": "Afternoon",
"FQDN": "Fully Qualified Domain Name",
"VJ": "Video Jockey",
"RTFT": "Read The F***ing Thread| Read The F***ing Topic",
"VM": "Voice Mail",
"VN": "Very Nice",
"W/": "With",
"VP": "Vice President",
"VR": "Virtual Reality",
"VS": "Versus| against",
"VT": "Video Tape",
"VV": "Very Very",
"VW": "Volkswagen",
"W8": "Wait",
"BREKKIE": "Breakfast",
"TING": "Thing",
"W@": "What",
"WA": "Wife Agro",
"WB": "Welcome Back| Write Back",
"BUZZ": "Excitement| Feeling when high",
"DENCH": "Awesome| cool",
"WC": "Welcome| Water Closet| toilet",
"WD": "Well Done",
"WE": "Whatever",
"PINK TOE": "White girl",
"TINA": "Crystal Meth",
"WH": "Wall hack",
"X)": "Happy| mischevious",
"WK": "White Knight",
"WM": "Woman Marine",
"WN": "Why Not?| What's New?",
"WP": "Wrong Person| Well Played",
"WR": "Warm Regards",
"X3": "Cute face",
"WT": "What",
"BROWN": "Heroin",
"WU": "What's Up",
"ENDZ": "Area where you live",
"WW": "White Widow| White Woman",
"WY": "Why| Would You",
"W^": "What's up",
"FOOPA": "Fat in belly area",
"FREEBSD": "Free Berkeley Software Distribution (Unix like OS)",
"XB": "XBox",
"XC": "Cross Counrty",
"XD": "LOL",
"SHOULDA": "Should have",
"WATP": "We Are The People",
"KOTC": "Kiss On The Cheek",
"WAUD": "What Are You Doing?",
"RTHX": "Thank you for the Retweet",
"XL": "Extra Large",
"BRUH BRUH": "Brother| friend",
"XM": "Extreme",
"KOTD": "KICKS Of The Day",
"XO": "Hugs & Kisses",
"XP": "Windows operating system",
"VAC": "Valve Anti-Cheat",
"KOTH": "King of the Hill (game)",
"XS": "Excess| Extra Small",
"XT": "Cross Training",
"IINM": "If I'm Not Mistaken",
"KOTL": "Kiss On The Lips",
"SCRAP": "Fight",
"XX": "Kisses",
"HAXX0R": "Hacker",
"GTLD": "Generic TLD",
"X]": "Happy",
"VAT": "Value Added Tax",
"YA": "You| Yes| Young Adult",
"YC": "You're Cool",
"YE": "Yes| yeah| You",
"YF": "Your Friend",
"WRYD": "What Are You Doing?",
"YG": "Young Gangster",
"YH": "Yeah",
"YK": "You're Kidding",
"PROBS": "Probably",
"YL": "Young Lady",
"YM": "Yahoo Messenger| Your Mom",
"WIKTT": "When I Kissed the Teacher",
"YN": "Crossed fingers",
"YO": "Your| Hi| Years Old",
"YP": "Your Problem| Yes, Please",
"YQ": "I Like You",
"VBD": "Very Big Deal",
"YR": "Yea, Right.",
"VBG": "Very Big Grin",
"SWOG": "Uncoolness| opposite of SWAGG",
"YT": "Are you there?",
"AAWY": "And Also With You",
"SCRAN": "Food",
"YW": "You're Welcome",
"SCRAM": "Go away",
"LAWD": "Lord",
"VBL": "Visible Bra Line",
"CYBI": "Can You Believe It?",
"WAVY": "Well dressed| Very nice",
"VBS": "Very Big Smile",
"VBR": "Variable Bit Rate",
"ZA": "Pizza",
"PROBZ": "Probably",
"LAWL": "LOL",
"NAMEAN": "Do you know what I mean?",
"SITREP": "Situation Report",
"AAYF": "As Always, Your Friend",
"I'MA": "I am going to",
"EAS": "End of Active Service",
"KUDOS": "Respect and recognition",
"VCD": "Video Compact Disc",
"GULLY": "Rough| raw| unpolished",
"WAWA": "Convenience store",
"FRILL": "For Real",
"ALCHY": "Alcoholic",
"SFLR": "Sorry For Late Reply",
"CHAW": "Chewing tobacco",
"VCR": "Video Cassette Recorder",
"CHAV": "Poorly educated| vulgar youth",
"EBD": "End of Business Day",
"SWPM": "Single White Professional Male",
"SWPL": "Stuff White People Like",
"PWN3D": "Owned",
"DLTBGYD": "Don't Let The Bastards Grind You Down",
"CHAP": "Man",
"PINNER": "Thin marijuana cigarette",
"CHAO": "Goodbye",
"TIRF": "That/This Is Really Funny",
"EBM": "Electronic Body Music",
"EBT": "Electronic Benefits Transfer",
"GCMG": "God Calls Me God",
"QUACK": "Doctor",
"GWARN": "Go On",
"CYDM": "Check Your Direct Messages",
"CYDI": "Can You Dig It?",
"DKDC": "Don't Know, Don't Care",
"SWOL": "Well built| muscular",
"VDU": "Visual Display Unit",
"ECG": "Electrocardiogram",
"SWOT": "Person who is always studying",
"YAHWEH": "The name of God in hebrew",
"ECO": "Ecological| Enconomical",
"IIRC": "If I Remember Correctly",
"\O": "Waving",
"EBONY": "Dark skin tone",
"ECU": "Electronic Control Unit",
"ECT": "etc",
"ECW": "Extreme Championship Wrestling",
"VEG": "Sit around and do nothing",
"JLUK": "Just Letting You Know",
"SWPF": "Single White Professional Female",
"WAYD": "What Are You Doing",
"TITF": "Take It Too Far",
"VER": "Version",
"EDD": "Estimated Delivery Date",
"VET": "Veteran",
"EDC": "Every Day Carry",
"OLD LADY": "Wife| GF",
"JELLO": "Jealous",
"YHBT": "You Have Been Trolled",
"SG2M": "Sounds Good To Me",
"EDJ": "Energy Drink Junkie",
"IIUC": "If I Understand Correctly",
"YHBW": "You Have Been Warned",
"NORLY": "No, Really?",
"EDM": "Electronic Dance Music",
"TITT": "Throw In The Towel| give up",
"EDT": "Eastern Daylight Time| Estimated Delivery Time",
"SUMFIN": "Something",
"^5": "High five",
"DOUCHE": "Idiot",
"VFM": "Value For Money",
"RUH ROH": "Uh Oh, Scooby Doo style",
"EEC": "European Economic Community",
"EEE": "excitement",
"GTOG": "Got To Go",
"TISA": "That Is So Awesome",
"EEG": "Extremely Evil Grin",
"SAUCED": "Drunk",
"VFX": "Visual Effects",
"EEK": "surprise| distress",
"BEAR": "Large man with body hair",
"ON ONE": "Under the influence of drugs",
"VGA": "Video Graphics Array/Adaptor",
"BEAK": "Nose| Insult| To Steal",
"VGC": "Very Good Condition",
"DKFS": "Don't Know For Sure",
"VGG": "Very Good Game",
"VGL": "Very Good Looking",
"VGM": "Video Game Music",
"VH1": "Video Hits One",
"^^": "Happy",
"EFA": "Edited For Accuracy",
"SFSG": "So Far So Good",
"BRONY": "Fan of My Little Pony",
"4E4": "Forever",
"V-SAFE": "Had a vasectomy",
"BLEEZY": "BLUNT",
"EFG": "Epic Fail Guy",
"GTRM": "Going To Read Mail",
"EFL": "English as a Foreign Language",
"BEBO": "A social networking site",
"CHEM": "Chemistry",
"EFT": "Electronic Funds Transfer",
"MACHING": "Going at maximum Speed",
"VHF": "Very High Frequency",
"EMOJI": "Tiny Pictures Used In Texts To Communicate A Feeling Or A Concept",
"BEBE": "Babe",
"LSAT": "Law School Admission Test",
"CHEA": "Yeah",
"VHS": "Video Home System",
"ASBO": "Anti Social Behaviour Order",
"MEAN": "Cool",
"DLTBBB": "Don't Let The Bed Bugs Bite",
"EGL": "Elegant Gothic Lolita",
"ZOOTED": "High on drugs",
"ANIME": "Japanese style animation",
"VID": "Video",
"VIG": "Interest paid on a loan",
"JLYK": "Just Letting You Know",
"RCON": "Remote Console",
"ASAS": "As Soon As Sensible",
"VIN": "Vehicle Identification Number",
"ASAT": "As Simple As That",
"DARPA": "Defense Advanced Research Projects Agency",
"VIP": "Very Important Person",
"ASAP": "As Soon As Possible",
"MEDS": "Medications",
"VIZ": "Adult comic",
"<333": "Multiple hearts",
"GTTS": "Got The T-Shirt",
"BADASS": "Cool| confident person",
"GCSE": "General Certificate of Secondary Education (UK)",
"THIZZLE": "Esctasy tablet",
"4GM": "Forgive me",
"GNSTDLTBBB": "Good Night Sleep Tight Don't Let the Bed Bugs Bite",
"EIE": "Enough Is Enough",
"IIWY": "If I Were You",
"CHIB": "Stab",
"NHFT": "Not Here For That",
"ABBR": "Abbreviation",
"GTSY": "Good To See You",
"BEEF": "Problem| fight| argument",
"FREEBIE": "Item or service one gets for free",
"EFTPOS": "Electronic Funds Transfer at Point Of Sale",
"IMNSHO": "In My Not So Humble Opinion",
"NUMPTY": "Idiot",
"SWYD": "Sleep When You're Dead",
"CHIZ": "Swiz| swindle",
"PRO BONO": "For good",
"IKWYDLS": "I Know What You Did Last Summer",
"CHIO": "Sexy| good looking, girl",
"OOSOOM": "Out Of Sight, Out Of Mind",
"RICK ROLL": "Trick someone with a link to Rick Astley video",
"WSWTA": "What Shall We Talk About?",
"SWYP": "So What's Your Point?",
"WAYDRN": "What Are You Doing Right Now?",
"CEEBS": "CBB",
"MEEP": "An exclamation for any situation",
"CEEBZ": "CBB",
"VMA": "Video Music Awards",
"W.E": "Whatever",
"NMJCU": "Not Much, Just Chillin', U?",
"VMI": "Virginia Military Institute",
"HWYD": "How Was Your Day?",
"ABDC": "America's Best Dance Crew (TV show)",
"ELF": "Every Lady's Fantasy",
"RAPID": "Cool| excellent",
"AT&T": "American Telephone & Telegraph",
"ELE": "Everybody Love Everybody",
"ELH": "Every Little Helps",
"ROFLOL": "Rolling On Floor, Laughing Out Loud",
"ELP": "Emerson, Lake, and Palmer (band)",
"ELO": "Electric Light Orchestra (band)",
"VNC": "Virtual Network Computing",
"W/E": "Whatever",
"MS-DOS": "Microsoft Disk Operating System",
"VNH": "Very Nice Hand",
"W/O": "Without",
"BISCUIT": "Attractive person| Cookie",
"EMA": "E-mail Address",
"VNS": "Very Nice Shot",
"BUPPIE": "Black Urban Professional",
"UMAD": "Are You Mad?",
"E-DATING": "Online dating",
"LBDN": "Look Busy Doing Nothing",
"MEGO": "My Eyes Glazed Over",
"EMO": "Emotional",
"EMP": "Electromagnetic Pulse",
"VOA": "Voice Of America",
"EMS": "Emergency Medical Service",
"VOD": "Video-On-Demand",
"POKER FACE": "Expressionless face",
"EMT": "Emergency Medical Technician",
"SEEDS": "Marijuana seeds| Children",
"XVID": "Video format",
"DKNY": "Donna Karan New York",
"VOL": "Volume",
"VOM": "Vomit",
"RCVD": "Received",
"WASTED": "Drunk| stoned",
"ENC": "Enclosure",
"ENE": "Northerner",
"ENO": "Awesome| cool",
"UMBJ": "You Must Be Joking",
"ENT": "Isn't",
"CBLTH": "Can't Breathe Laughing Too Hard",
"ASIC": "Application Specific Integrated Circuit",
"SEVIE": "Seventh Grader",
"VPL": "Visible Panty Line",
"ALRATO": "Later",
"VPN": "Virtual Private Network",
"EOC": "End Of Conversion",
"EOB": "End Of Business",
"EOE": "Equal Opportunity Employer",
"EOD": "End Of Discussion| End of Day",
"NYPD": "New York Police Department",
"EOG": "End Of Grade",
"EOF": "End Of File",
"WSIC": "Why Should I Care?",
"BELF": "Blood Elf",
"EOM": "End Of Message",
"EOL": "End Of Lecture",
"EOS": "End Of Show",
"EOR": "End Of Rant",
"W2F": "Way Too Funny",
"EOT": "End Of Thread |end of discussion",
"EOW": "End Of Watch",
"W2G": "Way To Go",
"CYPH": "To Share A Smoke",
"NYOB": "Not Your Own Business",
"ST00F": "Stuff",
"EPL": "English Premier League",
"MELT": "Idiot",
"W3C": "World Wide Web Consortium",
"SPERG": "Dramatically Passionate Or Obsessive About A Subject",
"PISSED OFF": "Mad| angry",
"BLAZED": "Stoned",
"CHOW": "Bye| Food",
"MEME": "An idea that spreads like a virus by word of mouth, email, blogs etc",
"EQE": "Easy Quick Efficient",
"BLOWN": "High",
"CHOO": "You",
"F2F": "Face To Face",
"YHNI": "You Have No Idea",
"IJAF": "It's Just A Fact",
"F2P": "Free To Play| Files To Peers",
"ASLP": "Age, Sex, Location, Picture?",
"F2T": "Free To Talk",
"NHOI": "Never Heard Of It",
"W4M": "Woman looking for a man",
"ERB": "Herb| Marijuana",
"ERK": "Irk| annoy",
"ERM": "Awkward conversation filler",
"ERP": "Erotic Role-Play",
"ERO": "Erotic",
"ILUSM": "I Love You So Much",
"B4MV": "Bullet For My Valentine (Band)",
"LOLAK": "Lots Of Love And Kisses",
"IJDK": "I Just Don't Know",
"VTL": "Visible Thong Line",
"ESA": "European Space Agency",
"ESC": "Escape key",
"ESB": "Empire Stikes Back",
"VTR": "Video Tape Recorder",
"BENT": "Dishonest| corrupt",
"ESE": "Friend| homeboy",
"ESD": "Electro-Static Discharge",
"ESH": "Donkey",
"KOSHER": "Good, OK",
"GYALDEM": "Group of girls",
"ESL": "English as Second Language",
"FETCH": "Cool| trendy| awesome",
"ESQ": "Esquire",
"ESP": "Extra Sensory Perception",
"CUTIE": "Cute| attractive person",
"EST": "Eastern Standard Time| Established",
"ETA": "Estimated Time of Arrival| Edited to Add",
"ETD": "Estimated Time of Departure",
"ETC": "Et Cetera| and so on",
"W'EVER": "Whatever",
"ETF": "Escape The Fate (band)",
"NYSE": "New York Stock Exchange",
"JUICE": "Respect| power",
"JONESING": "Craving| wanting really badly",
"MW2": "Modern Warfare 2 (game)",
"ETW": "Enjoy The Weekend",
"NHTB": "Never Heard That Before",
"VVN": "Very Very Nice",
"EUA": "End User Agreement",
"TTYIAB": "Talk To You In A Bit",
"EUC": "Excellent Used Condition",
"HIATUS": "A Pause| A Break",
"LBNL": "Last But Not Least",
"VVV": "Refering to comment below",
"TAPPED": "Crazy",
"BERK": "Idiot| fool",
"CHUG": "Drink| Offensive term for a Native American",
"IMGUR": "An Internet Image Sharing Website",
"JMHO": "Just My Humble Opinion",
"VWP": "Very Well Played| Very Well Put",
"EVA": "Ever",
"GRATZ": "Congratulations",
"OKEY DOKEY": "OK",
"4TW": "For The Win",
"FEENING": "Craving| wanting really badly",
"EVL": "Evil Laugh",
"GUAP": "Money",
"DOOBIE": "Marijuana cigarette| joint",
"BESO": "Kiss",
"GIWIST": "Gee, I Wish I'd Said That.",
"EVT": "Ei Voi Tiet",
"BF4L": "Best Friends For Life",
"EVS": "Whatever",
"RUCA": "Girlfriend",
"FLOSSING": "Showing off",
"JIFFY": "Short period of time",
"EWE": "Epilogue, What Epilogue? (Harry Potter)",
"FRAG": "Kill",
"FRAK": "Swear word from Battlestar Galactica",
"FRAP": "Frappuccino",
"LOLED": "Past tense of LOL",
"FRAT": "Fraternity",
"CUTCH": "Snuggle up",
"NEOCON": "Neo-conservative",
"EWW": "disgust",
"CYOOT": "Cute",
"MERC": "Mercenary| Injure or kill",
"POP A SQUAT": "Sit down",
"BETA": "Pre-release version| Better",
"POCKET DIAL": "Accidentally make a call with the phone in your pocket",
"MERP": "Meaningless word for when you don't know what else to say",
"EXE": "Windows program file",
"GDAY": "Hi",
"MERK": "Kill",
"EXP": "Experience points in games",
"MERT": "Mate",
"EXT": "Extension",
"4WD": "Four-Wheel Drive",
"FAAK": "Falling Asleep At Keyboard",
"SPOILER": "Something you would've prefered to find out yourself",
"YHWH": "Yahweh| God",
"KYOOT": "Cute",
"4X4": "Four wheel drive vehicle",
"UMML": "You Make Me Laugh",
"LBRS": "Lower Blackrock Spire",
"SO SO": "Neither good nor bad| average",
"GUFN": "Grounded Until Further Notice",
"EYO": "Hey, you",
"AMTRAK": "American passenger railroad corporation",
"BLOOD": "Gang member| Close friend",
"GCWOK": "Gay Couple Without Kids",
"ABT2": "About To",
"BIFFL": "Best Internet Friend For Life",
"LOLLY": "Money",
"KPOP": "Korean popular music",
"DIN DIN": "Dinner",
"METH": "Methamphetamine",
"CHYA": "Yes| yeah",
"FAAB": "Female-Assigned At Birth",
"SPEXY": "Sexy with glasses",
"FITTIN TO": "About to| getting ready to",
"YARDIE": "gang member",
"WASTE MAN": "Someone who is a waste of time and space",
"TOMOZ": "Tomorrow",
"GDGD": "Good Good",
"WSUL": "Will See You Later",
"FADE": "Kill| disgrace",
"MUFFIN TOP": "Fat spilling out over the top of pants",
"LSTM": "Laughing Silently To Myself",
"CORNY": "Uncool| overused",
"O RLY": "Oh, Really?",
"OUGHTA": "Ought to",
"HJNTIY": "He's Just Not That Into You",
"FABU": "Fabulous",
"GTHBA": "Good Times Had By All",
"CDROM": "Compact Disk Read Only Memory",
"BBALL": "Basketball",
"CHEESY": "Corny| lame",
"LBVS": "Laughing But Very Serious",
"GOD SPEED": "Good luck",
"LOLOL": "Lots Of Laughing Out Loud",
"DLAM": "Don't Laugh At Me",
"BLOKE": "Guy| man",
"PROJECT": "HOOD",
"BARE BACK": "Unprotected Sex",
"BOFFIN": "Technical expert",
"SNAKING": "Flirting",
"CHEERS": "Thanks| Goodbye",
"WAA": "Crying",
"FRFR": "FR, FR",
"ME GUSTA": "I like",
"STRAP": "Gun",
"CLICK": "Kilometre",
"CHEESE": "Drug| starter heroin| Money",
"WAH": "Working At Home",
"WAK": "Weird| awful",
"WAJ": "What A Joke",
"WAM": "Wait A Minute| What A Mess",
"RUKM": "Are You Kidding Me?",
"WAL": "What A Loser",
"WAO": "Wow",
"WAN": "What A Newb",
"WAP": "Wireless Access Point",
"E_E": "Tired| exhausted",
"WAU": "What About You?",
"WAT": "What",
"INKED": "Tattooed",
"WAW": "What A Wanker| What A Waste",
"WAY": "Where Are You?",
"WAX": "Without Any Experience",
"STACK": "$1000",
"KUTGW": "Keep Up The Good Work",
"WBB": "Will Be Back",
"WBN": "Would Be Nice",
"WBM": "Wanna-Be Moderator",
"FAB": "Fabulous",
"WBR": "With Best Regards",
"FAA": "Federal Aviation Administration",
"WBT": "Will Be There",
"WBS": "Write Back Soon",
"FAE": "From",
"WBU": "What About You?",
"FAG": "Cigarette",
"WBW": "White Boy Wasted",
"FAI": "Forget About It",
"WBY": "What about You?",
"ASDFGHJKL": "Excitement| Frustration| Boredom",
"FAM": "Family",
"FAO": "For Attention Of",
"FAQ": "Frequently Asked Question",
"FAT": "File Allocation Table",
"WCD": "What you Doing?",
"FAV": "Favorite",
"WCG": "World Cyber Games",
"RUKA": "Gangster girl| MOLL",
"STANG": "Ford Mustang",
"FBC": "FaceBook Chat",
"WCS": "Worst Case Scenario",
"FBG": "Fat Bottomed Girl",
"WCW": "World Championship Wrestling",
"CIAO": "Italian for Hi or Bye",
"FBF": "Facebook Friend",
"STROP": "Sulk| tantrum",
"FBI": "Federal Bureau of Investigation",
"FBM": "Fine By Me",
"FBO": "Facebook Official",
"MANDY": "MDMA| ecstasy",
"TILII": "Tell It Like It Is",
"WDC": "What's Da Craic?",
"OLED": "Organic Light-Emitting Diode",
"WDE": "Worst Day Ever",
"FBW": "Fly by wire",
"WDH": "What Da Hell",
"STAMP": "OK",
"MANDO": "Mandatory",
"BT DUBS": "BTW",
"WDW": "Who Dares Wins",
"WDY": "Why Don't You?",
"WEB": "World wide web| internet",
"BOGOF": "Buy One Get One Free",
"WEE": "Small",
"FCS": "For Christ's Sake",
"WEG": "Wicked Evil Grin| Wide Evil Grin",
"WEF": "With Effect From",
"DBEYR": "Don't Believe Everything You Read",
"FAIK": "For All I Know",
"FAIL": "Suck",
"WEL": "Whatever, Loser",
"JYEAH": "Cool version of Yeah",
"FDA": "Food and Drug Administration",
"WEP": "Wireless Encryption Protocol",
"SHOOK": "Scared| frightened",
"SHOOP": "Photoshop",
"NICU": "Neonatal Intensive Care Unit",
"WEY": "Dude| buddy",
"PANTHER": "Woman who likes older men",
"RINGTONE": "Sound a cell phone makes when receiving a call",
"TLDNR": "Too Long; Did Not Read",
"FDR": "Franklin Delano Roosevelt",
"FMTKFYTFO": "For Me To Know, For You To Find Out",
"BUZZ KILL": "Ruin a special moment",
"WFH": "Working From Home",
"WFI": "Wait For It",
"WFM": "Works For Me",
"FEE": "Female",
"FEH": "dismissal| disgust",
"BFAM": "Brother From Another Mother",
"PEANUT": "Pet name for BF or GF",
"FER": "For",
"WGC": "Wild Goose Chase",
"FF5": "Family Force 5 (band)",
"MWAH": "Kiss",
"NICE": "National Institute of Clinical Excellence",
"RUOK": "Are You OK?",
"BFAB": "Born From A Boombox| Back For A Bit",
"UNDIES": "Underwear",
"WGO": "What's Going On?",
"COPYCAT": "Someone who copies someone else",
"FFA": "Free For All",
"FAMS": "Family",
"FROG": "Offensive word for a French person",
"FFI": "For Further Information",
"GURL": "Girl",
"FFL": "Falling From LOLing",
"FFN": "Farewell For Now",
"WHA": "What",
"FFR": "Flash Flash Revolution| For Future Reference",
"FFX": "Final Fantasy 10 (game)",
"STEEZY": "Style With Ease",
"WHT": "What",
"WHS": "What He Said",
"FGJ": "For Great Justice",
"PITCH": "Contribute money",
"WIA": "Wounded In Action",
"STAHP": "Stop",
"WIC": "Women, Infants and Children| World in Conflict",
"FGS": "For God's Sake",
"WID": "With",
"WIF": "With",
"WII": "Nintendo Game Console",
"ATAP": "Any Time, Any Place",
"THICK": "Well proportioned| not fat or skinny",
"FAMO": "Friend| dude| family member etc",
"WIP": "Work In Progress",
"WIR": "When It's Ready",
"FHE": "First Hand Experience",
"GUTI": "Get Used To It",
"WCB": "Will Call Back",
"WIT": "With",
"POMO": "Permanent Open Market Operations| Post Modern| A Group Of Native American People In California",
"WIV": "With",
"WTBS": "With That Being Said",
"WIX": "WICKED",
"FHM": "For Him Magazine",
"FHO": "Friends Hanging Out",
"GUTS": "Courage",
"STEEZE": "Style with ease",
"FHS": "For Heaven's Sake",
"WJD": "What Jesus Did",
"RDRR": "Har-de-har-har",
"JAPAN": "Just Always Pray At Night",
"GUUD": "Good",
"HGTV": "Home and Garden Television",
"FIC": "Fanfiction",
"FIF": "Fifth Amendment",
"CIHY": "Can I Help You?",
"FIL": "Father-in-law",
"POLY": "Polyamory| loving more than one person at a time",
"FIN": "Finished",
"YCMIU": "You Couldn't Make It Up",
"C'MON": "Come On",
"SGTM": "Sounds Good To Me",
"IITYWTMWYBMAD": "If I Tell You What This Means Will You Buy Me a Drink?",
"FIO": "Figure It Out",
"FIT": "Good lokking| hot| sexy",
"BOGIE": "Cigarette",
"WKD": "Wicked",
"FIX": "A dose of drugs",
"TOTES PRESH": "Totally Precious",
"POMG": "Oh My God",
"WKS": "Well Kept Secret",
"HSPDA": "High Speed Packet Data Access",
"WTDT": "What To Do Today",
"MWHA": "Kiss",
"POP3": "Post Office Protocol v3 (for email)",
"ACCT": "Account",
"DFTBA": "Don't Forget To Be Awesome",
"WLD": "Would",
"CHOOB": "High level player who acts like a noob",
"CHONG": "Good looking| attractive, cute",
"BFFN": "Best Friend For Now",
"BFFL": "Best Friend For Life",
"COTCH": "Relax| chill",
"WLM": "Windows Live Messenger",
"POPO": "Police Officer(s)",
"BFFE": "Best Friends Forever",
"FKA": "Formerly Known As",
"PONR": "Point Of No Return",
"AD HOC": "For the specific purpose| Improvised| impromptu",
"MFEO": "Made For Each Other",
"WMA": "Windows Media Audio",
"WMD": "Weapons of Mass Destruction",
"WMG": "Warner Music Group",
"HOMEDOG": "HOMEBOY",
"WMF": "Wagging My Finger",
"WMJ": "Wat maak jy?| what are you doing?",
"WML": "Wish Me Luck",
"SHOPPED": "Photoshopped| manipulated",
"FLA": "Florida",
"HTHFYS": "Hope To Hear From You Soon",
"WMP": "Windows Media Player| With Much Pleasure",
"OLLG": "One Less Lonely Girl",
"WMV": "Windows Media Video",
"A2DP": "Advanced Audio Distribution Profile",
"WTFT": "What The French Toast| polite WTF",
"X.X": "Exasperated| irritated| annoyed",
"FLK": "Funny Looking Kid",
"FUTAB": "Feet Up, Take A Break",
"JARHEAD": "US Marine",
"TRYNA": "Trying To",
"FLS": "Flu-Like Symptoms",
"FLW": "Famous Last Words",
"NILF": "Nerd I'd Like To F***",
"FLY": "Zipper| Awesome| cool",
"WNL": "Within Normal Limits",
"KCACO": "Keep Calm And Carry On",
"TT_TT": "Crying",
"FMB": "F***-Me Boots",
"FMA": "Full Metal Alchemist",
"CHOLO": "Mexican Gangster",
"FMH": "For Him Magazine",
"FMJ": "Full Metal Jacket",
"WANNABE": "Someone who wants to be what they are not",
"FMI": "For More Information",
"TYMBI": "Thought You Might Be Interested",
"POPS": "Father",
"WAVEY": "Drunk| high",
"FMO": "For Members Only",
"WOC": "Women of Color",
"WOB": "Waste Of Bandwith",
"WOE": "What On Earth",
"FMV": "Full Motion Video",
"WOG": "Offensive term for a black person| A foreigner",
"WOF": "While On Fire",
"NIKE": "American sportswear company",
"WOM": "Women Over Mates",
"FNA": "For Necessary Action",
"WOP": "Offensive term for an Italian",
"WOS": "Waste Of Space| Wife Over Shoulder",
"FNB": "Football And Beer",
"WOR": "Our",
"FNE": "Free And Easy",
"WOT": "What",
"LTIC": "Laugh Til I Cry",
"WOZ": "Was",
"POST": "Hang out",
"FNO": "From Now On",
"FUTCH": "FEMME + Butch",
"LYLAS": "Love You Like A Sister",
"WPA": "Wi-Fi Protected Access",
"FAVE": "Favorite",
"POTD": "Post Of The Day",
"POTC": "Pirates Of The Carribean",
"WONKY": "Weird| not right",
"WTHN": "Why The Heck Not?",
"EOTW": "End Of The World",
"FNX": "Thanks",
"WPM": "Words Per Minute",
"WTII": "What time Is It?",
"FOB": "Fresh Off the Boat",
"LYLAB": "Love You Like A Brother",
"FOA": "First Of All",
"FOC": "Free Of Charge",
"POTS": "Parents Over The Shoulder| parents watching, I can't really talk",
"FOF": "Full of Fail",
"LYLAF": "Love You Like A Friend",
"UNCE": "Sound of club music",
"WWTBAM": "Who Wants To Be A Millionaire?",
"FOE": "Family Over Everything",
"CIMH": "Chuckle In My Head",
"RENTS": "Parents",
"POTO": "Phantom Of The Opera",
"WPW": "wow",
"FOI": "Freedom Of Information",
"OLOY": "Only Losers Obey Yolo",
"ACID": "LSD",
"FON": "Freak Of Nature",
"FOM": "Far Out Man",
"D(M)V": "DC, Maryland, Virginia| Department of Motor Vehicles",
"FOO": "Fool",
"FOT": "Fat, but HOT",
"FOV": "Field Of View",
"FOX": "Sexy| attractive woman",
"WTGP": "Way To Go, Partner",
"WTHC": "Who The Hell Cares?",
"POSE": "Suppose",
"CINO": "Catholic/Christian In Name Only",
"EOTD": "End Of The Day",
"POSS": "Possibly",
"CINI": "Cinema",
"NIML": "Never In My Life| Not In My Lifetime",
"FLAKE OUT": "Cancel at the last minute",
"FIG JAM": "F*** I'm Good Just Ask Me",
"SHIRT LIFTER": "Homosexual Male",
"DU JOUR": "Of the day",
"FPS": "First-Person Shooter",
"P2C2E": "Process Too Complicated To Explain",
"SWMBO": "She Who Must Be Obeyed",
"G2B": "Go To Bed",
"WRT": "With Regard To",
"G2F": "Got To Fly",
"G2E": "Got To Eat",
"WRU": "Where Are You?",
"G2H": "Go To Hell",
"G2G": "Gotta go",
"HOWDY": "How Do You Do?",
"WCIS": "What Can I Say",
"G2K": "Good To Know",
"POTY": "Post Of The Year",
"WSA": "Wow, Strong Arms",
"G2R": "Got To Run",
"BUSTED": "Very Ugly| Caught",
"POTW": "Patient of the Week",
"G2S": "Go To Sleep",
"BWOI": "Friend| buddy",
"WSM": "Woman Seeking Man",
"BFMV": "Bullet For My Valentine (band)",
"WSP": "What's up?",
"WSS": "Why So Serious?",
"WST": "We're Still Friends",
"XHTML": "Extensible Hyper-Text Markup Language",
"G3G": "One better than G2G",
"YINS": "You guys| you all",
"AMIGO": "Friend",
"WTLW": "Welcome To Last Week",
"FRL": "For Real",
"FRO": "Afro hairstyle",
"WT?": "What The ...? or Who the ...?",
"BTGOG": "By The Grace Of God",
"WTB": "Want To Buy",
"WTA": "Winner Takes All",
"WTD": "What The Deuce?",
"WTC": "World Trade Center| What The Crap",
"WTH": "What The Hell",
"ATMO": "According To My Opinion",
"WTG": "Way To Go",
"LTMS": "Laughing To Myself",
"MWPP": "Moony, Wormtail, Padfoot, and Prongs (Harry Potter)",
"WTL": "Will Talk Later",
"LTMQ": "Laughing To Myself Quietly",
"WTK": "Want To Know",
"WTM": "What's The Matter?",
"WTMI": "Way Too Much Information",
"WTP": "What's The Point?",
"WTO": "World Trade Organization",
"FSB": "Front Side Bus",
"WTML": "Welcome To My Life",
"WTT": "Want To Trade",
"FSF": "Free Software Foundation",
"WTV": "Whatever",
"FSE": "Funniest Sh** Ever",
"WTW": "Walk Through Walls| Walk this Way",
"IKEA": "Swedish furniture store",
"WTY": "Why, Thank You",
"LTNC": "Long Time No Chat",
"DLTM": "Don't Lie To Me",
"FSM": "Female Seeking Male",
"FEDEX": "Federal Express",
"WUA": "What You At?",
"ACME": "A Company that Makes Everything",
"FSR": "For Some Reason",
"WUB": "Love",
"WUD": "What You Doing?",
"WUG": "What You Got?| Where You Going?",
">": "O:Angry",
"FSU": "Friends Stand United",
"G4U": "Good For You",
"WUF": "Where are you from?",
"WUH": "What?",
"(^^^)": "Shark",
"WUM": "Woman| Wind-Up Merchant",
"WUL": "Watched User List",
"OLTL": "One Life To Live",
"SCOUSER": "Person from Liverpool",
"WUN": "Bye",
"FTA": "From The Article",
"POWN": "PWN",
"WUP": "What",
"FTC": "Favorite|Thumbs up| Comment",
"VEXED": "Angry| annoyed",
"WUS": "Wimp",
"ACLU": "American Civil Liberties Union",
"FTD": "Fresh To Death",
"WUT": "What",
"WUW": "What You Want?",
"FTF": "First To Find",
"WUV": "Love",
"FTH": "For The Horde",
"FTK": "For The Kids| For The Kill",
"FTJ": "Fuel The Jet",
"WUZ": "Was",
"BFFLTDDUP": "Best Friends For Life Till Death Do Us Part",
"FTM": "Female To Male",
"FTL": "For The Loss| Faster Than Light",
"LTMA": "Less Talk, More Action",
"ATOY": "Always Thinking of you",
"FTP": "File Transfer Protocol",
"FTR": "For The Record",
"NITE": "Good Night",
"FTV": "For The Victory",
"CITA": "Caught In The Act",
"NITM": "Not In The Mood",
"WW1": "World War One",
"COPY THAT": "I understand",
"DFTSYP": "Don't Forget To Say Your Prayers",
"WW2": "World War Two",
"FUD": "Fear, Uncertainty, and Doubt",
"YITB": "Yours In The Bond",
"YUCKY": "Disgusting| gross",
"NITW": "Not In That Way",
"WWA": "Wiggers with attitude",
"WWE": "World Wrestling Entertainment",
"WWF": "World Wildlife Fund| World Wrestling Federation",
"LTNT": "Long Time No Talk",
"LTNS": "Long Time No See",
"WTMW": "Welcome To My World",
"NISM": "Need I Say More?",
"WWU": "Where Were You?",
"WWW": "World Wide Web",
"WWY": "Where Were You?",
"THREAD": "Topic On A Message Board| Clothes",
"HEINA": "Girlfriend",
"VIBIN": "Hanging| chilling",
"SHUCKS": "modesty| disappointment",
"JAWOHL": "Yes| affirmative",
"FWD": "Foward",
"BUZZ OFF": "Go away",
"BWTH": "But What The Hell",
"LILABOC": "Life Is Like A Box Of Chocolates",
"FSAS": "Famous Stars and Straps",
"FWP": "Friend With Privileges",
"WYA": "Where You At?",
"MSDOS": "Microsoft Disk Operating System",
"WYB": "Watch Your Back",
"FWT": "Flying With Turban",
"LOONY": "Crazy",
"WYD": "What You Doing?",
"WYG": "What You Get",
"FWU": "F****** With You",
"VERLAN": "French reverse slang",
"WYF": "What's Your Favorite ...?",
"WYK": "Would You Kindly",
"YITE": "Are You All Right?",
"WYM": "Watch Your Mouth| What You Mean?",
"WYL": "Whatever You Like",
"WYN": "What's Your Name?",
"CUL8ER": "See You Later",
"HYFB": "Hope You Feel Better",
"L2MS": "Laughing To Myself",
"UNIX": "Computer operating system",
"WYP": "What's Your Problem?| What's Your Point?",
"WYS": "What You Said",
"WYR": "Would You Rather",
"SMALLS": "Underwear",
"WYW": "Whatever You Want",
"L2PK": "Learn To Player Kill",
"ATST": "At The Same Time",
"FYA": "For Your Amusement",
"FYC": "For Your Convenience| For Your Consideration",
"5X5": "Loud and Clear",
"FYE": "For Your Entertainment",
"NEWAY": "Anyway",
"RASTA": "Rastafarian",
"FSCK": "File System Consistency Check",
"FYI": "For Your Information",
"FYK": "For Your Knowledge",
"FYN": "Fine",
"HYIP": "High Yield Investment Program",
"FYM": "Free Your Mind",
"FYP": "Fixed Your Post",
"FYR": "For Your Review| For Your Records",
"BFTP": "Blast From The Past",
"FYT": "For Your Time",
"OMGNA": "Oh My Gosh, Not Again",
"ATTY": "Atomizer from electronic cigarette",
"ATTT": "Ain't That The Truth",
"CUDDY": "Buddy| friend",
"ATTN": "Attention",
"SYSOP": "System Operator",
"PCMCIA": "Personal Computer Memory Card International Association",
"FSBO": "For Sale By Owner",
"BFUT": "Best Friends Until Tomorrow",
"ATVB": "All The Very Best",
"HHIS": "Hanging Head In Shame",
"L33T": "Elite",
"IKLY": "I Kinda Like You",
"BFWB": "Best Friend With Benefits",
"ACTA": "Anti-Counterfeiting Trade Agreement",
"HCDAJFU": "He Could Do A Job For Us",
"LCTS": "Let's Change The Subject",
"SHIP": "Romantic relationship",
"TGIAF": "Thank God It's Almost Friday",
"NIZZ": "No",
"LYMYWY": "Love You, Miss You, Want You",
"WD40": "Multi-purpose lubricant",
"SYIH": "See You In Hell",
"ATWA": "Air, Trees, Water, Animals| All The Way Alive",
"WTTC": "Welcome To The club",
"CACTUS": "Broken| dead",
"L337": "LEET in LEET speak",
"GPOYW": "Gratuitous Picture Of Yourself Wednesday",
"GEEZ": "Jesus",
"DUNZO": "Done| over| finished",
"WTTJ": "Welcome To The Jungle",
"NOCAL": "Northern California",
"REEM": "Gorgeous| sexy| nice",
"GNITE": "Good Night",
"WTTM": "Without Thinking Too Much",
"OMGSH": "Oh My Gosh",
"DMAF": "Don't Make A Fuss",
"DMAL": "Drop Me A Line",
"LUNCHBOX": "Bulge in a man's pants",
"PLONK": "Cheap wine",
"('_')": "Emoticon Representing Boredom",
"SHIV": "Home made knife",
"ILYAM": "I Love You As a Mate",
"UNPC": "Not Politically Correct",
"LYLMB": "Love You Like My Brother",
"REGS": "Regular| Low quality marijuana",
"SYLA": "See You Later Alligator",
"NIAMY": "Never In A Million Years",
"DMCA": "Digital Millennium Copyright Act",
"SYNT": "See You Next Time",
"KICKS": "Shoes| Thrills",
"OBGYN": "Gynecologist",
"SHADY": "Shifty| sly| sneaky",
"ATYS": "Anything You Say",
"WTYO": "Whatever Turns You On",
"INNIT": "Isn't It",
"XBF": "Ex-Boyfriend",
"XBL": "XBox Live",
"HYNA": "Good looking girl",
"GAC": "Guilty As Charged| Get A Clue",
"GAG": "About to throw up| Joke",
"GAH": "frustration",
"GAJ": "Get A Job",
"GAL": "Girl| Get A Life",
"FSHO": "For Sure",
"GAS": "Flatulence| Guitar Acquisition Syndrome",
"GAT": "Gun",
"SYNC": "Synchronize",
"SHADE": "Casual| disrespectful",
"NEFIN": "Anything",
"LAWLS": "LOL",
"GBA": "God Bless America",
"OMDS": "OMD",
"$_$": "Has money",
"GBH": "Grievous Bodily Harm",
"OMDZ": "Oh My Dayz",
"GBP": "Great British Pounds (Sterling)",
"GBU": "God Bless You",
"LAWLZ": "LOL",
"GBY": "God Bless You",
"SHAFT": "Screw| rip off",
"MY B": "My Bad",
"GCF": "Good Clean Fun",
"HYPH": "Hyperactive| crazy",
"HYPO": "Hypodermic needle/syringe",
"UNTZ": "Sound of a pulsing beat",
"OMDB": "Over My Dead Body",
"HHOJ": "Ha-Ha, Only Joking",
"REKT": "Wrecked",
"HHOK": "Ha-Ha, Only Kidding",
"WCYD": "What Can You Do?",
"SYRS": "See You Real Soon",
"IKTR": "I Know That's Right",
"AIN'T": "Am not| are not| is not",
"JEJEMON": "People who type stupidly",
"GDI": "God Damn It",
"MINTED": "Rich| wealthy",
"GDP": "Gross Domestic Product",
"IDWTAI": "I Don't Wanna Talk About It",
"OMGA": "Oh My Giddy Aunt",
"OMGD": "Oh My God Dude",
"EPIC": "Extremely awesome",
"GED": "General Educational Development",
"GEG": "Interrupt a conversation",
"H&K": "Hugs and Kisses",
"EPIN": "Epic Win",
"GEN": "Information",
"XGA": "eXtended Graphics Array/Adaptor",
"GINGRICH": "Hypocrite",
"OTLTA": "One Thing Led To Another",
"GANGNAM": "Relating to wealth and class",
"IKTF": "I Know The Feeling",
"JEWFRO": "Curly Jewish hair style",
"GFA": "Grenade Free America",
"GFC": "Global Financial Crisis",
"HYUK": "Lose| despite having a good lead",
"GFF": "Grenade Free Foundation",
"LINGO": "Language",
"GFI": "Go For It",
"TRIFE": "Trifling| unimportant",
"GFN": "Gone For Now",
"GFP": "Good For Pics",
"FSOT": "For Sale Or Trade",
".MIL": "military",
"MGBY": "May God Bless You",
"GFU": "Good For You",
"GFX": "Graphics",
"GFY": "Good For You",
"PPLE": "People",
"GGA": "Good Game All",
"ZOMFG": "OMFG",
"SOWIE": "Sorry",
"DUB DUB DUB": "WWW| World Wide Web",
"GGG": "Good, Giving, and Game",
"GGL": "Giggle",
"GGN": "Gotta Go Now",
"LMIRL": "Let's Meet In Real Life",
"GGS": "Good Games",
"OMGZ": "OMG",
"GGW": "Girls Gone Wild",
"P!ATD": "Panic! At The Disco (band)",
"4CHAN": "Internet message/image board",
"GHB": "Gamma Hydroxybutyrate",
"SHUG": "Sugar",
"GHD": "Good Hair Day",
"GHM": "God Help Me",
"HHVF": "Ha-Ha, Very Funny",
"SHUP": "Shut Up",
"ADAD": "Another Day Another Dollar",
"GHZ": "Gigahertz",
"WUBU": "What You Been Up to?",
"IKYP": "I'll Keep You Posted",
"SHUX": "Shucks| modesty",
"TOOTSIES": "Toes| feet",
"GIG": "Live performance| Gigabyte| Job",
"GIF": "Graphics Interchange Format - Picture Format",
"MUD DUCK": "Ugly woman",
"LUBB": "Love",
"PPLZ": "People",
"FLYFF": "Fly For Fun",
"BANGTIDY": "Sexy| fit",
"LUBE": "Lubricant",
"GIS": "Google Image Search",
"TURNT": "Hyped Up| Drunk| High| Horny",
"OMJC": "Oh My Jesus Christ",
"GIT": "Idiot",
"TIOLI": "Take It Or Leave It",
"GIY": "Google It Yourself",
"ILYLT": "I Love You Long Time",
"DUNNO": "Don't Know",
"BOLSHY": "Bolshevik| communist| Argumentative",
"HHWW": "Holding Hands While Walking",
"GOMEN": "Sorry",
"SHWE": "Should We",
"DMML": "Don't Make Me Laugh",
"HIT ME UP": "Call me later",
"XLS": "MS Excel file extension",
"GOMER": "Get Out of My Emergency Room| unwelcome patient",
"ADDY": "Address",
"BTDTGTTS": "Been There, Done That, Got The T-Shirt",
"RESH": "Depressing",
"EPOS": "Electronic Point Of Sale",
"KRAM": "Smoke weed",
"HASHTAG": "Label on Twitter to aid searching",
"GKY": "Go Kill Yourself",
"XML": "eXtensible Markup Language",
"SHANK": "Homemade knife| Stab",
"GLA": "Good Luck All",
"GLD": "Good Life Decision",
"GLF": "Group Looking For",
"H4X0R": "Hacker",
"GEWD": "Good",
"GLU": "Guys| Girls Like Us",
"SCRIM": "Practice game",
"Y/O": "Year(s) Old",
"Y/N": "Yes/No",
"GMA": "Good Morning, America",
"SMIDSY": "Sorry Mate, I Didn't See You",
"ADED": "All Day Every Day",
"GMC": "General Motors Corporation",
"FBTW": "Fine, Be That Way",
"PPPW": "Per Person, Per Week",
"Y/Y": "Yes/Yes",
"CODEC": "Coder-Decoder (usually for audio/video data)",
"GMH": "Gives Me Hope",
"SNAFU": "Situation Normal, All Fouled Up",
"GMO": "Genetically Modified Organism",
"LUFF": "Friendly Love",
"G2G2B": "Got To Go To Bed",
"GMT": "Greenwich Mean Time",
"GMV": "Got My Vote",
"GMY": "Good Man Yourself",
"IMING": "Instant Messaging",
"WNDITWB": "We Never Did It This Way Before",
"GND": "Girl Next Door",
"GNE": "Good Night and Everything",
"XOX": "Hugs & Kisses",
"ADHD": "Attention Deficit Hyperactivity Disorder",
"GNK": "Glitch No Kill (gaming)",
"GNP": "Gross National Product",
"HICKY": "Love bite",
"GNO": "Girls/Guys Night Out",
"H/O": "Hold On",
"GNR": "Guns 'N Roses",
"H/T": "Hat Tip",
"GNU": "GNU's Not Unix| Linux OS",
"H/U": "Hook Up| Make Out",
"SOUND": "Cool| awesome| good",
"GOB": "Mouth",
"JOINT": "Rolled marijuana cigarette",
"GOG": "Person from north Wales",
"SPITAL": "Hospital",
"GOI": "Get Over It",
"VACA": "Vacation, holiday",
"GOK": "God Only Knows",
"PROMO": "Promotional material",
"WDFD": "Whoop Dee F***ing Doo",
"GOM": "Idiot| fool",
"GOL": "Giggle Out Loud",
"MMAMP": "Meet Me At My Place",
"FSVO": "For Some Value Of ...",
"GON": "Go on| Going to",
"GOP": "Grand Old Party",
"GOS": "Girlfriend Over Shoulder| Game Of Soldiers",
"GOU": "Good On You",
"GOW": "God of War (Game)| Gears of War (Game)",
"GOY": "Non-Jewish person",
"LSHMBH": "Laughing So Hard My Belly Hurts",
"Y2K": "Year 2000",
"H1B": "US employment visa",
"GPA": "Grade-Point Average",
"GPC": "Generic Pack of Cigarettes| Get/Post/Cookie",
"GPL": "General Public License",
"GPS": "Global Positioning System",
"GPU": "Graphics Processing Unit",
"WDIB": "Why Do I Bother?",
"HAWTSOME": "Hot and awesome",
"WDIC": "What Do I Care?",
"OMSJ": "Oh My Sweet Jesus!",
"FANNYING": "Wasting time| messing about",
"DMTA": "Dirty Minds Think Alike",
"FEELS": "Feelings",
"WDIM": "What Does It Mean?",
"H2G": "Had to Go| Honest To God",
"H2H": "Heart To Heart| Head To Head",
"H2O": "Water",
"PHAIL": "Fail",
"H2T": "Head To Toe",
"GR8": "Great",
"SCREB": "Dirty| scruffy person",
"GR9": "Better than GR8",
"CHILAXING": "Chilling and relaxing",
"GRE": "Graduate Record Examination",
"KAFM": "Keep Away From Me",
"VEETA": "Money",
"GRL": "Girl",
"NAIJA": "Nigerian",
"GRO": "Guest Relations Officer",
"FUBAR": "Fouled Up Beyond All Recognition",
"GRR": "Anger| frustration",
"LUMI": "Love You, Mean It",
"XTC": "Ecstasy",
"GOKIL": "Crazy (Indonesian)",
"GRX": "Gracias| Merci",
"IAWTC": "I Agree With That/This Comment",
"FOSHO": "For Sure",
"LUMU": "Love You Miss You",
"HTTPS": "Hyper-Text Transfer Protocol Secure",
"IAWTP": "I Agree With This Post",
"GSD": "Getting Stuff Done",
"SALAM": "Peace",
"TOODLE": "Goodbye| toodle-pip",
"GSH": "GSOH",
"GSM": "Global System for Mobile communications",
"CLUBBING": "Visiting nightclubs",
"MGMT": "The Management (band)",
"BARKING": "Mad| crazy",
"GSR": "Gun Shot Residue",
"GSW": "Gun Shot Wound",
"H4X": "Hacks| cheats",
"LULS": "LOL",
"GTB": "Go To Bed",
"LULZ": "LOL",
"DIMER": "10",
"GTA": "Grand Theft Auto",
"GTD": "Getting Things Done",
"GTH": "Go To Hell",
"GTG": "Got To Go",
"WULA": "What're you Looking At?",
"GTI": "High performance hatchback",
"GTL": "Gym, Tan, Laundry",
"GTK": "Good To Know",
"LQTMS": "Laughing Quietly To Myself",
"GTM": "Giggling To Myself",
"GTO": "Gran Turismo Omologato",
"GWAS": "Game Was A Success",
"GTR": "Getting Ready",
"GTS": "Good Times",
"OMWH": "On My Way Home",
"WDMB": "Will Do My Best",
"GTW": "Got To Work",
"OUNCE": "28 grams",
"GTZ": "Good Times",
"KOTOR": "Knights Of The Old Republic (Game)",
"GUD": "Good",
"SICK": "Awesome| cool",
"GUI": "Graphical User Interface",
"GUH": "Annoyed| angry",
"XVX": "Straightedge Vegetarian| Straightedge Vegan",
"TLDR": "Too Long; Didn't Read",
"SIAP": "Sorry If Already Posted",
"YAHOO": "Yob, lout",
"GUV": "Guvnor| boss",
"RELLIES": "Relatives",
"TLDW": "Too Long; Didn't Watch",
"SIAS": "Say It Ain't So",
"GVG": "Guild Versus Guild",
"VAIR": "Very",
"GWAP": "Money",
"TWFAF": "That's What Friends Are For",
"SCART": "Audio/Video TV connector",
"SIDS": "Sudden Infant Death Syndrome",
"GET LAID": "To Have Sexual Intercourse",
"TLGO": "The List Goes On",
"WILCO": "Will Comply",
"GWC": "Guy With Camera",
"GWB": "George W. Bush",
"GWG": "Girl With Glasses",
"GWI": "Get With It",
"XXX": "Kisses",
"GWK": "Game Winning Kill",
"BTHOM": "Beats The Hell Outta Me!",
"FTAO": "For the Attention Of",
"BROHAN": "BRO",
"SICS": "Sitting In Chair Snickering",
"GWP": "Gift With Purchase",
"MY FOOT": "disbelief",
"CAPISH": "Do you understand?",
"GWS": "Get Well Soon",
"GWR": "Guinness World Records",
"H8R": "Hater",
"GFAK": "Go Fly A Kite",
"SITTING DUCK": "Vulnerable| helpless target",
"XYZ": "Examine Your Zipper",
"SIDK": "Sorry, I Didn't Know",
"IPSEC": "Internet Protocol Security",
"LUSM": "Love You So Much",
"SIFS": "Secret Internet Fatties",
"DAVID": "Cool guy",
"GYC": "Get Your Coat",
"SIGH": "Exhale depressedly",
"SNAKE": "Snitch| back stabber",
"WUSS": "Wimp| coward",
"GYO": "Get Your Own",
"GYP": "Swindle| rip off",
"LURK": "Read a forum but never post",
"ADSL": "Asymmetric Digital Subscriber Line",
"GYT": "Get Yourself Tested",
"KRON": "CHRONIC",
"DWEET": "Do it",
"CHOPPER": "AK-47 assault rifle",
"SNAKE OIL": "Fake medicine",
"THUMBNAIL": "Small version of a picture",
"LFTSU": "Look Forward To Seeing You",
"LURV": "Love",
"LUSH": "Drunk person| Great| brilliant",
"URYYFM": "You Are Too Wise For Me",
"HIIK": "Hell If I Know",
"IDUNNO": "I Don't Know",
"DONNO": "I don't know",
"VAPE": "Inhale vapor from E-cigarettes",
"DWEEB": "Dork| nerd",
"ILLY": "Fresh, cool",
"ONION": "Ounce of a drug",
"WUUT": "WUU2",
"PROPS": "Respect| recognition",
"FACE PALM": "Slap forehead with the palm of your hand",
"BELLIG": "Belligerant",
"WUU2": "What are You Up To?",
"JOOC": "Just Out Of Curiosity",
"ADTR": "A Day To Remember (band)",
"FTFY": "Fixed That For You",
"WDUT": "What Do You Think?",
"WUWH": "Wish You Were Here",
"TLND": "True Love Never Dies",
"ILNY": "I Love New York",
"SALTS": "Smiled A Little Then Stopped",
"WDUM": "What Do You Mean?",
"GL HF": "Good Luck, Have Fun",
"GOONER": "Supporter of Arsenal Football Club",
"WUWT": "What's Up With That",
"WUWU": "What's Up With You?",
"SIKE": "Indicating the previous statement is false",
"ILML": "I Love My Life",
"ILMO": "In Loving Memory Of",
"WDTM": "What Does That Mean?",
"X_X": "Dead",
"WDUD": "What Do You Do?",
"FTFT": "For The First Time",
"YAK": "You Already Know",
"YAL": "Girl",
"PIECE": "Gun| weapon| Work of graffiti| Pipe for smoking drugs",
"HOMEGIRL": "Closest female friend",
"YAP": "Steal| rob| Young Asian Professional",
"YAS": "Emphatic Yes| You are Sexy",
"YAR": "Yes",
"YAT": "Girl| woman| Where You At?",
"VATO": "Dude",
"YAW": "You Are Welcome",
"YAY": "Cocaine| Exclamation of approval",
"HUSPAZ": "Hurray",
"SKANKIN": "Form of dancing",
"DECENT": "Good| awesome",
"OWTTE": "Or Words To That Effect",
"YKWIM": "You Know What I Mean",
"YKWIS": "You Know What I'm Saying?",
"RFID": "Radio Frequency Identification",
"FCFS": "First Come First Served",
"HAC": "Hot Asian Chick",
"YBS": "You'll Be Sorry",
"6_6": "Looking down| Suspicious",
"SALUT": "Hi",
"HAG": "Unattractive old woman",
"YBW": "You've Been Warned",
"HAI": "Hey, hi| Yes",
"TLOL": "Truly Laughing Out Loud",
"HAK": "Hugs And Kisses",
"HAN": "Alone",
"HAR": "Sarcastic laugh",
"MURKED": "Badly defeated| killed",
"HAX": "Hacking| Hackers| hacks",
"WDYT": "What Do You Think?",
"HAY": "How Are You",
"WDYW": "What Do You Want?",
"WDYM": "What Do You Mean?",
"HBD": "Happy Birthday",
"WDYS": "What Did You Say?",
"HBK": "Heart Break Kid",
"HBO": "Home Box Office",
"HBN": "How About Now",
"PWNED": "Owned| dominated| Perfectly Owned",
"HBP": "Half-Blood Prince (Harry Potter)",
"OATUS": "On A Totally Unrelated Subject",
".NET": "network providers",
"HBU": "How About You?",
"HBT": "How 'Bout That",
"BREDRIN": "Friend",
"TINAG": "This Is Not A Game",
"YDG": "You Dig?",
"HBY": "How About You?",
"YDI": "You Deserve It",
"YDL": "Yellow Dog Linux",
"YDK": "You Don't Know",
"HD": "High-Definition",
"TINAR": "This Is Not A Recommendation",
"DNDC": "Don't Know, Don't Care",
"YDS": "You Don't Say",
"YDU": "You Don't Understand",
"SCHEMIE": "Someone from a council estate",
"ONCO": "Or Nearest Cash Offer",
"WDYD": "What Did You Do?",
"WDYE": "What Did You Expect?",
"URAQT": "You Are A Cutie",
"WDYG": "Where Did You Go?",
"YEA": "Yeah| yes",
"HCO": "Hollister",
"YEH": "Yes",
"SCRUB": "Loser who thinks he's something",
"ILU2": "I Love You Too",
"PROXY": "Agent| middle man",
"SKINT": "Broke| having no money",
"YEP": "Yes",
"OMFUG": "Other Music Found UnderGround",
"YER": "Your| Yes",
"HDD": "Hard Disk Drive",
"YEW": "You",
"HDF": "Halt Die Fresse| Shut up",
"YEY": "YAY",
"YWIMC": "Your Wish Is My Command",
"COCKY": "Thinking too highly of yourself",
"HDL": "Hab Dich Lieb| I love you",
"TLTR": "Too Long To Read",
"HDU": "How Dare You",
"ADORBS": "Adorable",
"ILUM": "I Love You More",
"BYAM": "Between You And Me",
"HDY": "How Dare You",
"HEA": "Happily Ever After",
"RWNJ": "Right Wing Nut Job",
"HEH": "Cynical laugh",
"YFW": "Your Face When ...",
"GWOP": "Money",
"HEP": "Hip| cool",
"INORITE": "I Know, Right?",
"I<3U": "I Love You",
"YGG": "You Go Girl",
"HEX": "Hexadecimal| Curse",
"YGI": "You Got It",
"HEY": "Hello, hi",
"ONG": "OMG",
"YGM": "You Get Me?| You've Got Mail",
"OMFSM": "Oh My Flying Spaghetti Monster",
"HF?": "Have Fun",
"YGP": "You Got PWNED!",
"YHA": "Youth Hostel Association",
"GINNEL": "Alley| passageway",
"TWEEKER": "A methamphetamine user",
"HFY": "Here For You| Happy for You",
"YHL": "You Have Lost",
"YHM": "You Have Mail",
"ILUT": "I Love You Too",
"HGB": "Hellogoodbye (band)",
"FORTE": "Area of expertise| strength",
"PANSY": "Sissy| unmanly person",
"YHU": "You",
"HGH": "Haters Gonna Hate",
"HGI": "How Goes It?",
"MY BAD": "apology",
"HGL": "Happy Go Lucky",
"MYCH": "Maybe You Can Help",
"YID": "Yes I Do",
"HGV": "Heavy Goods Vehicle",
"YSMTU": "You So Made That Up",
"PANTS": "Rubbish| Trousers| Underpants",
"YIK": "Yes, I Know",
"YIM": "Yahoo! Instant Messenger",
"WONGA": "Money",
"GWTF": "Go With The Flow",
"JOANING": "Putting someone down| making fun of",
"ILYA": "I Love You All",
"AEAP": "As Early As Possible",
"GWTP": "Get With The Program",
"ILYB": "I Love You Both",
"WILTW": "What I Learned This Week",
"ILYF": "I'll Love You Forever",
"ILYG": "I Love You Guys/Girls",
"ILYL": "I Love You Lots",
"HI5": "Social networking site",
"ILYM": "I Love You More",
"AEAE": "And Ever And Ever",
"ILYK": "I'll Let You Know",
"SHOTTA": "Thug| gangster| Drug dealer",
"ILYT": "I Love You Too",
"JUNGLE FEVER": "Attraction between people of different race",
"KEYGEN": "Software serial number generator",
"SITD": "Still In The Dark",
"BADMAN": "Gangster",
"RANDOM": "Unexpected| unpredictable",
"HIG": "How's It Going?",
"HIP": "Cool| stylish| contemporary",
"HIR": "HIS\HER",
"SITU": "Situation",
"HIV": "Human Immunodeficiency Virus",
"BNWOT": "Brand New Without Tags",
"YKK": "Yoshida Kogyo Kabushikikaisha| zipper manufacturer",
"FCOL": "For Crying Out Loud",
"QTPI": "Cutie Pie",
"CHOUT": "Chill Out| Watch Out",
"LEAF": "Marijuana",
"ILY2": "I Love You Too",
"FTSE": "Financial Times and the London Stock Exchang",
"GFTW": "Go For The Win",
"SKITZ": "Schitzophrenic",
"YKW": "You Know Who",
"BLATES": "Blatently",
"FTSK": "Forever The Sickest Kids (Band)",
"RFTS": "Reach For The Stars",
"YLI": "You Love It",
"YLM": "You Love Me?",
"TTIUWP": "This Thread Is Useless Without Pictures",
"FTTB": "For The Time Being",
"BRAVO": "Well done",
"WEBA": "Welcome Back",
"TLYK": "To Let You Know",
"AVGN": "Angry Video Game Nerd",
"YMW": "You're Most Welcome",
"LEES": "Very Attractive Man/Woman",
"WEED": "Marijuana",
"WEEE": "Exclamation of excitement",
"LEET": "Chat room language| Elite",
"GRULE": "Hard work| nuisance",
"FTUW": "For The Ultimate Win",
"BTWITIAILWU": "By The Way, I Think I Am In Love With You",
"HLV": "Hasta La Vista",
"8-)": "Cool",
"GFWL": "Games For Windows Live",
"BTWITIAILWY": "By The Way, I Think I Am In Love With You",
"YNK": "You Never Know",
"HMB": "Hit Me Back (reply)",
"YNT": "Why Not?",
"HMF": "HMPH",
"KTBPA": "KTBSPA",
"YNW": "You're Not Welcome",
"SKITTLES": "Drugs",
"HMI": "Human Machine Interface",
"HML": "Hate My Life",
"HIYA": "Hello",
"BANJAX": "Break| destroy",
"BYKI": "Before You Know It",
"HMP": "Help Me, Please",
"NUTCASE": "Crazy person",
"YOB": "Hooligan| thug",
"HMS": "Her Majesty's Ship (UK)",
"HMV": "His Master's Voice",
"HMU": "Hit Me Up",
"AINEC": "And It's Not Even Close",
"CUFFED": "Taken| has a BF or GF",
"L4D2": "Left 4 Dead 2 (game)",
"ONNA": "Oh No, Not Again",
"YOO": "You",
"(.)(.)": "Female Breasts",
"HNG": "Horny Net Geek",
"HNK": "Hugs and Kisses",
"EXTRA": "Over the top| excessive",
"I/O": "Input/Output",
"HNR": "Hit 'N' Run",
"LURVE": "Love",
"HNY": "Happy New Year",
"HOA": "Home Owners Association",
"HOF": "Hall Of Fame| Heavy on Flag",
"BYKT": "But You Know That",
"YPU": "you",
"CUWYL": "Catch Up With You Later",
"HOH": "Head Over Heels| Hard Of Hearing",
"HOG": "Harley Owner Group",
"Ã": "Registered Trademark Symbol",
"GLEAK": "Spit",
"HON": "Honey",
"8-|": "Nerdy",
"HOP": "Heroin",
"HOT": "Very good looking| sexy",
"BHL8": "Be Home Late",
"HOS": "Husband Over Shoulder",
"HOV": "High Occupancy Vehicle",
"ONOZ": "Oh No",
"RATCHET": "Undesirable| coarse woman",
"RWYS": "Reap What You Sow",
"HDYDT": "How Did You Do That?",
"YQW": "You're Quite Welcome",
"IMBA": "Imbalanced",
"FTBOMH": "From The Bottom Of My Heart",
"TMAI": "Tell Me About It",
"JPEG": "Joint Photographic Experts Group| Image file format",
"DINLO": "Stupid person",
"YRG": "You Are Good!",
"Ã´": "Trademark",
"IMBO": "In My Biased Opinion",
"YRO": "Your Rights Online",
"TMBG": "They Might Be Giants (band)",
"80S": "The Eighties| 1980s",
"MYOB": "Mind Your Own Business",
"WEIN": "What Else Is New?",
"YRS": "Years",
"HQF": "LOL",
"YRU": "Why Are You...?",
"MHMM": "Yes, sure, OK",
"WDYWTTA": "What Do You Want To Talk About?",
"BASER": "Drug user",
"RFYL": "Run For Your Life",
"MEETER": "Person who likes to look at fat men",
"YSC": "Your So Cute",
"BYOM": "BYO Music| BYO Meat",
"MYMP": "Make Your Mama Proud",
"PEER TO PEER": "Computer network without a central server",
"YSI": "You Stupid Idiot",
"IMAO": "In My Arrogant Opinion",
"YSK": "You Should Know",
"IMAP": "Internet Message Access Protocol",
"FMDIDGAD": "Frankly My Dear , I Don't Give A Damn",
"WEHT": "What Ever Happened To?",
"HIWTH": "Hate It When That Happens",
"YSL": "Yves Saint Laurent",
"BYOC": "BYO Computer",
"BYOB": "BYO Beer",
"DINKY": "Tiny| small| Dual Income No Kids Yuppie",
"HRC": "Human Rights Campaign",
"YSS": "You Suck Severely",
"HRH": "His/Her Royal Highness",
"IMDB": "Internet Movie Database",
"YT?": "You Think ?",
"YTB": "You're The Best",
"YTD": "Year To Date",
"HRU": "How Are You?",
"ONUD": "Oh No You Didn't",
"YTM": "You're The Man",
"GONNA": "Going to",
"YTP": "YouTube Poop",
"YTO": "You're The One",
"AWHFY": "Are We Having Fun Yet",
"HSH": "Home Sweet Home",
"HSL": "Hue, Saturation, Lightness",
"DNTK": "Don't Need To Know",
"POIDH": "Pics Or It Didn't Happen",
"HSM": "High School Musical",
"HSP": "Highly Sensitive Person",
"TMBI": "Tell Me about It",
"HST": "High Speed Train",
"GLEEK": "A fan of the TV show Glee",
"ONTD": "Oh No They Didn't",
"YUH": "Yes| You",
"YUM": "Said when something tastes good or is attractive",
"IMCO": "In My Considered Opinion",
"YUO": "you",
"YUP": "Yes",
"HTC": "Hit The Cell",
"YUS": "Yes",
"HTE": "Hard To Explain",
"STEXT": "Sexy text message",
"HTG": "Hard To Get",
"HTH": "Hope That Helps| How The Hell",
"HTM": "Hand To Mouth",
"WEML": "Whatever Major Loser",
"DIGITS": "Telephone number",
"HTX": "Houston, Texas",
"YALDI": "excitement",
"LENG": "Weapon| gun",
"BLIMEY": "surprise| OMG",
"HUA": "Heard Understood Acknowledged",
"IWIWT": "I Wish I Were There",
"HUD": "Heads-Up Display",
"HUH": "Confused sound",
"YVW": "You're Very Welcome",
"HUN": "Honey",
"TIPSY": "Slightly drunk",
"HUR": "Here| Hair",
"IKWUM": "I Know What You Mean",
"WELL": "Very, so",
"HUS": "Hustler",
"AVPM": "A Very Potter Musical",
"IMEO": "In My Educated Opinion",
"WELC": "You're Welcome",
"PRANG": "Scared",
"HVD": "Happy Valentine's Day",
"H2CUS": "Hope To See You Soon",
"WELP": "Well",
"DINGY": "Ignore",
"GGBB": "Good Game Bye Bye",
"IMHO": "In My Humble Opinion",
"DOG FOOD": "Heroin",
"BYTM": "Better You Than Me",
"ONYD": "Oh No You Didn't",
"HWB": "Hottie With Body",
"LF1M": "Looking For One More",
"WEPA": "All right| cool, yeah!",
"YATLOML": "You Are The Love Of My Life",
"PRAWN": "Girl who is good looking except for her face",
"HWK": "Homework",
"HWP": "Height Weight Proportional",
"MYSM": "Miss You So Much",
"HWU": "Hey, What's Up?",
"HOLLA BACK": "Get Back To Me",
"WENG": "Good looking",
"HXC": "Hardcore (music)",
"YKTR": "You Know the Rest",
"BHUM": "Sexy| fit",
"FUBU": "Fouled Up Beyond Use| Clothing brand",
"SCRILLA": "Money| cash",
"SAVAGE": "Brutal but awesome",
"WET BACK": "Illegal immigrant",
"HYB": "How You Been?",
"HYD": "How You Doing?",
"HYH": "Have You Heard| Hold Your Horses",
"HYG": "Here You Go",
"TRU DAT": "That is True| I agree",
"IYKWIM": "If You Know What I Mean",
"HYP": "Harvard, Yale, Princeton",
"IKWYL": "I Know Where You Live",
"IKWYM": "I Know What You Mean",
"HYS": "Have Your Say",
"HYU": "Hit You Up",
"BLAZE": "Smoke Weed",
"FUBB": "Fouled Up Beyond Belief",
"BUTT DIAL": "Accidentally call from a phone in your pocket",
"RHINO": "Older man looking for a younger woman",
"MULTI": "Player with multiple online accounts",
"ARIGATO": "Thank You",
"JPOP": "Japanese pop music",
"EMCEE": "MC, Master of Ceremonies",
"IMMD": "It Made My Day",
"IMMA": "I'm Going To",
"SWOLE": "Muscular| in good shape",
"KITTEN": "Woman who dates older men",
"RGDS": "Regards",
"SJGR": "Shit Just Got Real",
"WERD": "agreement",
"WERK": "approval| praise",
"PRATT": "Stupid person",
"WTHIWWY": "What The Hell Is Wrong With You?",
"SEXPOT": "Sexy Person",
"KBPS": "KiloBytes per second",
"CHOP IT UP": "Talk",
"IMMAO": "In My Most Arrogant Opinion",
"LOLCAT": "Cat picture with silly caption",
"*\O/*": "Cheerleader",
"LEVA": "Traitor",
"WEVA": "Whatever",
"BUTT HURT": "Offended| upset",
"TMNT": "Teenage Mutant Ninja Turtles",
"WAZ UP": "What's Up?",
"ILLEST": "Best",
"4/20": "April 20th| WEED smoking day",
"DUBSTEP": "Music genre",
"IMML": "It Made Me Laugh",
"TMLT": "To My Last Tweet",
"TMMS": "That Made Me Smile",
"IMPO": "In My Personal Opinion",
"WEWT": "Alternative spelling of WOOT",
"TRUFAX": "True Fact",
"OOBE": "Out Of Body Experience",
"EMOTICON": "Sequence of characters representing an emotion eg :)",
"PREE": "Watch| stare at",
"FUIC": "Farmers Union Iced Coffee",
"PREP": "Prepare",
"PMYMHMMFSWGAD": "Pardon Me, You Must Have Mistaken Me For Someone Who Gives A Damn",
"DMIID": "Don't Mind If I Do",
"EBCAK": "Error Between Chair And Keyboard",
"BLAST": "Damn| Good time",
"PABX": "Private Automatic Branch Exchange",
"IMOO": "In My Own Opinion",
"PREGGERS": "Pregnant",
"IAC": "In Any Case",
"IAB": "I Am Bored",
"IAE": "In Any Event",
"IAD": "It All Depends",
"LEWL": "LOL",
"DOBE": "Idiot",
"IAG": "It's All Good",
"2K10": "2010",
"OOAK": "One Of A Kind",
"IAK": "I Already Know",
"PUKKA": "Genuine| top quality",
"IAL": "I Ain't Laughing| I Actually Laughed",
"IAS": "In A Second",
"FIRME": "Cool| good| nice",
"IAU": "I Adore You",
"IAW": "In Accordance With| I Agree With",
"IAY": "I Adore You",
"CLAP": "Gonorrhea| STD",
"IBB": "I'll Be Back",
"(!)": "Sarcasm",
"IBL": "In Before Lock",
"IBK": "Idiot Behind Keyboard",
"YKIMS": "You Know It Makes Sense",
"IBM": "International Business Machines",
"WINRAR": "Winner",
"TMRW": "Tomorrow",
"DEROB": "Very bored",
"IBS": "Irritable Bowel Syndrome",
"IBU": "Ibuprofen",
"QOTSA": "Queens Of The Stone Age (Band)",
"IBW": "I'll Be Waiting",
"ICC": "Ice Crown Citadel",
"BLARG": "boredom",
"ICB": "I Can't Believe...",
"ICE": "Diamonds| Crystal meth| Kill",
"ICK": "UGH",
"PB&J": "Peanut Butter and Jelly",
"ICQ": "I Seek You",
"ICP": "Insane Clown Posse (band)",
"ZED": "Zombie",
"ICR": "I Can't Remember",
"ICU": "Intensive Care Unit| I See You",
"ICT": "Information and Communication Technology",
"ICW": "In Connection With| I Can't Wait",
"CHRONIC": "High quality marijuana",
"FUMA": "Fake Puma shoes",
"IDD": "Indeed",
"PRIT": "Ugly",
"GIMME": "Give me",
"IDC": "I Don't Care",
"TMTH": "Too Much To Handle",
"DOGG": "Friend| buddy",
"IDE": "Integrated Development Environment| Integrated Device Electronics",
"OOFT": "Sound made when impressed by something",
"IDJ": "I Don't Judge",
"IDI": "I Doubt It",
"TMTC": "Too Many To Count",
"IDL": "I Don't Like ...",
"IDK": "I Don't Know",
"IDN": "I Don't kNow",
"BESTIE": "Best Friend",
"IDM": "It Doesn't Matter| Intelligent Dance Music",
"IDR": "I Don't Remember",
"GGOH": "Gotta Get Outta Here",
"IDT": "I Don't Think",
"LEKKER": "Cool| sexy",
"IDU": "I Don't Understand",
"IDW": "I Don't Want",
"TMTT": "Too Much To Text",
"TMTS": "Too Much Too Soon",
"LSMFT": "Lucky Strike Means Fine Tobacco",
"RSRSRS": "LOL",
"IED": "Improvised Explosive Device",
"2K4U": "To Cool For You",
"SKI INSTRUCTOR": "Cocaine dealer",
"DOFL": "Dying On Floor Laughing",
"IEP": "Individualized Education Program",
"BIAB": "Back In A Bit",
"IMVU": "Instant Messaging Virtual Universe",
"ILYLAS": "I Love You Like A Sister",
"IFF": "If and only if| Identification: Friend or Foe| Internet Friends Forever",
"CLEO": "Chief Law Enforcement Officer",
"OPSEC": "Operations Security",
"IFM": "In Fond Memory",
"WYGOWM": "Will You Go Out With Me?",
"VLOGGER": "Video blogger",
"DOHC": "Dual Overhead Cam",
"IGG": "Ignore| I Gotta Go",
"IGI": "I Get It",
"RUGGER": "Rugby",
"IGM": "I Got Mine",
"BLANK": "Ignore",
"IGN": "In Game Name",
"IGS": "I Guess So",
"TWIMC": "To Whom It May Concern",
"IGU": "I Give Up",
"IGY": "I Get You",
"TWLOHA": "To Write Love On Her Arms",
"ZIP": "Ounce of marijauna",
"PS/2": "Computer port for keyboard or mouse",
"ZIT": "Spot| pimple",
"DOKE": "Diet Coke",
"MIDI": "Musical Instrument Digital Interface",
"AFAP": "As Far As Possible",
"WWBD": "What Would Batman Do?",
"FIXIN TO": "About to| getting ready to",
"IMYA": "I Miss You Already",
"'EM": "Them",
"IHM": "I Hate Mondays",
"YOUNS": "You guys| you all",
"IHP": "I Hate People",
"LSMIH": "Laughing So Much It Hurts",
"IHO": "In Honor Of",
"IHT": "I Hate This",
"IHS": "I Hope So",
"IHU": "I Hate You",
"IMYM": "I Miss You More",
"IHY": "I Hate You",
"IMYT": "I Miss You Too",
"JARRIN": "Annoying",
"DOJA": "Marijuana| weed",
"IIB": "Ignorance Is Bliss",
"IIH": "If I'm Honest",
"PAKI": "Offensive term for a Pakistani",
"IIL": "I'm In Love",
"IIS": "Internet Information Services",
"IIR": "Idiot In Room",
"BIEH": "Best I Ever Had",
"WWYDT": "Why Would You Do That?",
"DOODE": "DUDE",
"IJG": "I Just Giggled",
"IJL": "I Just Laughed",
"IJK": "I'm Just kidding",
"DOMO": "Thank you",
"DOMS": "Delayed Onset Muscular Soreness",
"LOW KEY": "Quiet| Don't tell anyone",
"MIGA": "Make It Go Away",
"IJS": "I'm Just Saying",
"SK8R": "Skater",
"BIFF": "Best Friend| BFF",
"OOML": "Out Of My League",
"OOMF": "One Of My Friends/Followers",
"IKE": "DUDE| I Know, Eh",
"IKH": "I Know, Huh?",
"IKM": "I Know Man",
"IKN": "I Know Not",
"DOLO": "Alone| solo",
"MANDEM": "Group of men| boys",
"PROB": "Probably",
"IKR": "I Know, Right",
"PROD": "Protestant",
"IKT": "I Knew That",
"PROC": "Programmed Random Occurrence",
"IKY": "I Know You",
"MIFF": "Annoy",
"QURL": "Girl",
"DOMA": "Defense Of Marriage Act",
"PROG": "Progressive rock music| Program",
"DOOD": "Dude",
"ILD": "Ich Liebe Dich (I Love You)",
"ILH": "I Love Him/Her",
"ZONG": "BONG",
"ILI": "I Love It",
"ILL": "Cool, tight, etc",
"|O|": "LOL",
"DOOM": "Classic FPS",
"ILM": "I lige m",
"ILT": "I Like That",
"ILS": "Inflated Lat Syndrome",
"ILU": "I Love You",
"ILY": "I Love You",
"GGWP": "Good Game, Well Played",
"YOUSE": "You",
"IMA": "I am going to",
"IMB": "I'm Bored",
"IME": "In My Experience",
"ZOMB": "Zombie",
"IMD": "In My Defence",
"DONE": "Finished| Drunk| intoxicated",
"IMF": "International Monetary Fund",
"IMH": "In My Head",
"OOMM": "Out Of My Mind",
"ZOMG": "OMG",
"IMK": "In My Knowledge",
"LOW IT": "Allow It| don't worry",
"IMO": "In My Opinion",
"LFDY": "Live Fast, Die Young",
"IMP": "It's My Pleasure",
"IMS": "Irritable Male Syndrome",
"IMR": "I Mean, Really",
"IMU": "I Miss You",
"IMY": "I Miss You",
"SWAGGER": "A person's style",
"INB": "I'm Not Bothered",
"YALLA": "Hurry Up",
"AFFA": "Angels Forever, Forever Angels",
"INC": "Incoming",
".ORG": "non-profit organizations",
"PARE": "Friend| pal",
"BIFFLE": "BFFL",
"PARA": "Paralytic| very drunk",
"BOILER": "Ugly woman",
"OOPS": "Word said after doing something wrong",
"INK": "I Never Knew| Tattoo",
"J/K": "JK",
"INO": "I Know",
"INR": "IKR",
"INT": "Isn't it",
"INS": "I'm Not Sure",
"WWIC": "Why Would I Care?",
"INY": "I Need You",
"DORB": "Adorable",
"TWITA": "That's What I'm Talking About",
"IOC": "International Olympic Committee",
"COUGH": "Confess| come clean",
"DOPE": "Heroin| Cool| awesome",
"IOI": "Indication of Interest| I'm Over It",
"IOH": "I'm Outta Here",
"IOM": "Isle Of Man",
"KCCO": "Keep Calm and Chive On",
"ION": "In Other News",
"IOU": "I Owe You",
"IBMTS": "I've Been Meaning To Say",
"IOW": "In Other Words",
"ZOOT": "Spliff| joint",
"VTEC": "Variable Valve Timing and Lift Electronic Control System",
"IPA": "India Pale Ale| Internet Protocol Address",
"IPC": "Ignore Post Count",
"AWKS": "Awkward",
"DOSH": "Money",
"YLOD": "Yellow Light Of Death",
"PATD": "Panic At The Disco (band)",
"INBD": "It's No Big Deal",
"IPO": "Initial Public Offering",
"BILY": "Because I Love You",
"LWKM": "Laugh Wan Kill Me (LOL)",
"WWJP": "What Would Jesus Post?",
"JUICED": "Happy| excited",
"PATT": "Party All The Time",
"DOTA": "Defence Of The Ancients",
"KTHX": "OK, Thanks",
"MIMO": "Multiple Input Multiple Output",
"TEAL DEER": "TLDR",
"PER SE": "By or in itself| intrinsically",
"WWIT": "What Was I Thinking?",
"DORK": "Silly| socially inept person",
"SANGA": "Sandwich",
"UNICEF": "United Nations Children's Fund",
"INAH": "I Need A Hug",
"WWJD": "What Would Jesus Do?",
"INB4": "In Before",
"MILL": "Million",
"IRA": "Irish Republican Army",
"AWKY": "Awkward",
"IRC": "Internet Relay Chat",
"NLOL": "Not LOL",
"IRL": "In Real Life",
"MINX": "Cheeky| mischievous girl",
"MINT": "Nice| cool",
"OOTW": "Out Of This World",
"ZTA": "Zeta Tau Alpha",
"IRS": "Internal Revenue Service",
"TTOTM": "That Time Of The Month",
"BUN IN THE OVEN": "Pregnant",
"BINS": "Glasses| binoculars",
"ISA": "Inshaa Allah| OK",
"ILYSM": "I Love You So Much",
"INDY": "Independent",
"ISB": "I'm So Bored",
"J4F": "Just For Fun",
"BING": "But It's Not Google",
"ISH": "Shit| -vagueness",
"TNBM": "True Norwegian Black Metal",
"J4L": "Just for Laughs",
"ISO": "In Search Of",
"AWNP": "All Work, No Play",
"ISP": "Internet Service Provider",
"ISS": "International Space Station| I'm So Sorry",
"OOTC": "Obligatory On-Topic Comment",
"OOTB": "Out Of The Box",
"FUZZ": "Police",
"BIOS": "Basic Input Output System",
"GLUCK": "Good Luck",
"ZUP": "What's up?",
"OOTD": "Offer Of The Day| Outfit Of The Day",
"ITA": "I Totally Agree",
"WAEFRD": "When All Else Fails, Read Directions",
"Slag": "Meaning",
"ZUT": "Damn it",
"BION": "Believe It Or Not",
"ITC": "In That Case| In This Channel",
"ITF": "In The Future",
"ITE": "Alright?",
"ITG": "In The Groove",
"OOTO": "Out Of The Office",
"ITK": "In The Know",
"ITN": "I Think Not",
"ITM": "In the Mouth",
"ITP": "Inside The Perimeter",
"DETAI": "Don't Even Think About It",
"ITO": "In Terms Of",
"ITT": "In This Thread",
"ITV": "Independent TV (UK)",
"INFO": "Information",
"AWOL": "Absent Without Leave",
"TRILL": "True and Real",
"ITZ": "It's| In The Zone",
"PAYE": "Pay As You Earn",
"BLOOPER": "Mistake| embarrassing moment",
"PAYG": "Pay As You Go",
"SNAIL MAIL": "Regular postal service",
"WYLASOMWTC": "Would You Like A Saucer Of Milk With That Comment?",
"BIRD": "Cocaine| Young woman",
"WDOML": "Worst Day Of My Life",
"ILYTB": "I Love You To Bits",
"LFMF": "Learn From My Fail",
"TINSTAFL": "There Is No Such Thing As Free Lunch",
"BRUVA": "Brother",
"IVF": "In Vitro Fertilization",
"NSPCC": "National Society for Prevention of Cruelty to Children",
"MIPS": "Million Instructions Per Second",
"GYAL": "Girl",
"HANDLE": "Nickname| alias",
"VCARD": "Virtual business card",
"NTBCW": "Not To Be Confused With",
"MISC": "Miscellaneous",
"CHAMP": "Uncool",
"MISB": "Mint condition In Sealed Box",
"SHAWN": "Attractive man",
"OOYM": "Out Of Your Mind",
"IWC": "Internet Wrestling Community",
"MISH": "Missionary position",
"OOYL": "Out Of Your League",
"CLUT": "Clumsy person",
"MISO": "My Internet Shut Off",
"IWM": "It Wasn't Me",
"MONROE": "Piercing between nose and upper lip",
"BROWN BREAD": "Dead",
"GJOAC": "Go Jump Off A Cliff",
"ILYVM": "I Love You Very Much",
"IWU": "I Want You",
"SPINS": "Effect of drinking too much",
"UG2BK": "You've Got To Be Kidding",
"BITD": "Back In The Day",
"IWY": "I Want You",
"MIRL": "Meet In Real Life",
"BISS": "Because I Said So",
"SWND": "Sound",
"RACKS": "Money",
"AFRL": "Away From Real Life",
"STEEZ": "Style with ease",
"FEAE": "For Ever And Ever",
"RHCP": "Red Hot Chili Peppers (Band)",
"ANYWHO": "Anyhow",
"AWSM": "Awesome",
"IYA": "Hiya| hello",
"FEAT": "Featuring",
"CLWY": "Can't Live Without You",
"BITW": "Best In The World",
"IYD": "In Your Dreams",
"IYF": "In Your Face",
"(8)": "Musical note (MSN)",
"IYH": "In Your Head",
"ZZZ": "Sleepy| bored| tired",
"IYL": "If You're Lucky| It's Your Life",
"GYFS": "Get Your Facts Straight",
"IYO": "In Your Opinion",
"IYQ": "I Like You",
"IYT": "All right",
"IYW": "If You Want/Will",
"INIT": "Isn't it?| Initialization",
"ZILCH": "Nothing| zero",
"AFRO": "Hairstyle from Africa",
"LIFTED": "High",
"LFTI": "Looking Forward To It",
"YLYL": "You Laugh You Lose",
"MIHOP": "Made It Happen on Purpose",
"TNLN": "Took Nothing Left Nothing",
"WOWZER": "Exaggerated Wow",
"NLYG": "Never Let You Go",
"VTPR": "View To Permanent Relationship",
"REGGIE": "Regular weed",
"THUMBS DOWN": "Sign of disapproval",
"FEEN": "Man",
"LWWY": "Live While We're Young",
"DA BOMB": "Excellent| the best",
"STEAL": "Punch",
"ZIG ZAG": "Cigarette paper",
"401K": "Retirement investment plan",
"FEDS": "FBI agents| police",
"CONGRATS": "Congratulations",
"FAMALAM": "Family| close friend",
"FEDZ": "Police",
"SEKKLE": "Calm down",
"WWYD": "What Would You Do?",
"INPO": "In No Particular Order",
"K33L": "Kill",
"INOP": "Inoperative| In Need Of Practice",
"WOOSE": "WUSS",
"PAPER": "Money",
"WWWY": "What's Wrong With You?",
"JAF": "Just A Friend",
"JAH": "God| Jehovah",
"JAG": "Socially inept person",
"JAM": "Just A Minute",
"JAP": "Jewish American Princess| Person from Japan",
"TNOT": "Take Note Of That",
"RHIP": "Rank Has Its Privileges",
"JAT": "Just A Thought",
"TNOP": "Totally Not Our Problem",
"JAS": "Just A Second",
"DAGGY": "Uncool| not trendy| untidy",
"CMAR": "Cry Me A River",
"INRS": "It's Not Rocket Science",
"UA4I": "You Asked For It",
"JBH": "Just Being Honest",
"GYNO": "Gynecologist",
"JBU": "Just Between Us",
"OBHWF": "One Big Happy Weasely Family",
"JBT": "Jack-Booted Thug",
"YKHII": "You Know How It Is",
"JBY": "Just Be Yourself",
"JCB": "J C Bamford | excavator manufacturer",
"BOLTOP": "Better On Lips Than On Paper",
"(A)": "Angel (MSN)",
"WFYB": "Whatever Floats Your Boat",
"CROAK": "Die",
"INRI": "Iesus Nazarenus Rex Iudaeorum| Jesus of Nazareth King of the Jews",
"YKYLM": "You Know You Love Me",
"INTP": "Introverted iNtuitive Thinking Perceiving (personality type indicator)",
"RADAR": "Radio Detection And Ranging",
"IAWTCSM": "I Agree With This Comment So Much",
"VNSFW": "Very NSFW",
"NAIL IT": "Complete| get right",
"JDI": "Just Do It",
"GYPO": "Gypsy",
"JDM": "Japanese Domestic Market",
"TNTM": "That's News To Me",
"YMCA": "Young Mens Christian Association",
"HATERZ": "Enemies",
"JEL": "Jealous",
"TNSL": "Took Nothing, Signed Logbook",
"YMBJ": "You Must Be Joking",
"JET": "Leave quickly",
"COUGAR": "Older woman looking for a younger man",
"HOTTIE": "Attractive person",
"INTK": "I Need To Know",
"INVU": "I Envy You",
"CMEO": "Crying My Eyes Out",
"JFF": "Just For Fun",
"JFK": "John Fitzgerald Kennedy| New York airport| Just For Kicks",
"JFN": "Just For Now",
"TWYMS": "That's What Your Mom Said",
"JFT": "Just For Today",
"JFY": "Just For You",
"JGB": "Just Got Back",
"JGH": "Just Got Home",
"ESKI": "Type of urban music",
"JGI": "Just Google It",
"JGL": "Just Get Lost",
"DESPO": "Desperate",
"J CAT": "mentally unstable person",
"FTASB": "Faster Than A Speeding Bullet",
"JHC": "Jesus H Christ",
"BIG TIME": "More than usual",
"FLOTUS": "First Lady Of The United States",
"PC4PC": "Picture Comment For Picture Comment.",
"JHP": "Jacketed Hollow Point (bullet)",
"COSPLAY": "Costume Play",
"SKUX": "A guy who's good with women",
"ZIMBO": "Person from Zimbabwe",
"SSBHM": "Super Sized BHM",
"JIC": "Just In Case",
"CMHO": "Cry My Heart Out",
"HYDRO": "Hydroponically grown marijuana",
"JIP": "Rob| rip-off",
"JIT": "Young gangster| Just In Time",
"(H)": "Cool dude emoticon (MSN)",
"JJJ": "Australian radio station",
"ESPN": "Entertainment and Sports Programming Network",
"CHOONG": "Good looking| buff",
"GEEZER": "A guy| a bloke| a person",
"NMJC": "Not Much Just Chat",
"TTIWWP": "This Thread Is Worthless Without Pictures",
"JKD": "Jeet Kune Do| Martial Art",
"SQUARE": "Cigarette",
"TINWIS": "That Is Not What I Said",
"JKL": "Just Kidding Loser| Just Kidding, LOL",
"ESOL": "English for Speakers of Other Languages",
"WGAT": "what",
"JKN": "Joking",
"LGBT": "Lesbian, Gay, Bisexual, Transexual",
"WNOHGB": "Where No One Has Gone Before",
"9JA": "Nigeria",
"JKS": "Jokes",
"JKZ": "Jokes",
"LOLAGE": "The act of LOL",
"WRYTA": "What Are You Talking About?",
"CHUTIYA": "Mild Indian expletive",
"ID10T": "Computer error caused by user",
"JLO": "Jennifer Lopez",
"JLS": "Jack the Lad Swing (band)",
"JLT": "Just like that",
"LGFB": "Looks Good From Behind",
"SUGAR DADDY": "Rich older man",
"JLY": "Jesus Loves You",
"KUDO": "KUDOS",
"JICYDK": "Just In Case You Didn't Know",
"CAMEL": "Brand of cigarettes",
"WALLAH": "I swear to God in Arabic",
"JMA": "Just Messing Around",
"(K)": "Kiss (MSN)",
"JML": "Just My Luck",
"MYSPACE": "A social networking site",
"JMO": "Just My Opinion",
"JMT": "Just My Thoughts",
"JMS": "Just Making Sure",
"BUTTERZ": "Ugly",
"BENJAMIN": "$100 bill",
"SMACK": "Heroin",
"BUTTERS": "Ugly",
"ESRB": "Entertainment Software Rating Board",
"FESS": "Confess",
"HOMEBOY": "Closest friend",
"OPPS": "oops",
"YMMD": "You Made My Day",
"(L)": "Love heart (MSN)",
"AGGY": "Aggitated",
"YMMS": "You Make Me Smile",
"YMMV": "Your Mileage May Vary",
"PSTN": "Public Switched Telephone Network",
"YMML": "You Make Me Laugh",
"UBER": "Ultimate| best| above",
"ROGER THAT": "I Understand, OK",
"CMNT": "Comment",
"YGTBKM": "You've Got To Be Kidding Me",
"SCUBA": "Self-Contained Underwater Breathing Apparatus",
"OPPA": "Older brother, male friend",
"JPA": "Just Playing Around",
"SPIT GAME": "Act of flirting",
"ESTD": "Easier Said Than Done",
"JPG": "Image file format",
"(N)": "Thumbs down (MSN)",
"BUZZED": "Tipsy| slightly drunk",
"NALGO": "Not A Lot Going On",
"*HUG*": "A Cyberhug",
"BASIC": "Uncool| boring",
"TOAO": "The One And Only",
"LICKED": "Drunk or high",
"CMOS": "Complementary Metal-Oxide Semiconductor",
"IOAB": "I'm On A Boat",
"VANILLA": "Unexciting| conventional",
"MSFAM": "My Sister From Another Mister",
"NALGAS": "Buttocks",
"WAGWARN": "What's Going On?",
"KTHXBAI": "OK, Thanks, Bye",
"BOBFOC": "Body Off Baywatch| Face Off Crimewatch",
"COUGER": "Older woman looking for a younger man",
"DOXING": "Collect Personal Information",
"DUSTY": "Ugly",
"K3U": "I Love You",
"JAFA": "Just Another F***ing Aucklander",
"HLAS": "Hook, Line And Sinker",
"URGH": "annoyance| displeasure",
"SLAB": "Slow Low/Loud And Bangin'| customized car",
"PBUH": "Peace Be Upon Him",
"JSP": "Java Server Pages",
"WAFFLE": "Pointless talk",
"JSU": "Just Shut Up",
"DEAD ASS": "Serious",
"JTC": "Join The Club",
"JTB": "Just Too Bad",
"BWDIK": "But What Do I Know?",
"CLOCK": "Look at| Hit",
"GFETE": "Grinning From Ear To Ear",
"JTM": "Je t'aime| I love you",
"TE AMO": "I love you (Spanish)",
"ELBOW": "A pound of marijuana",
"(S)": "Seriously",
"SLAN": "Goodbye",
"HOODIE": "Hooded sweatshirt| Person wearing a hooded sweatshirt",
"LUWAMH": "Love You with All My Heart",
"WAFWOT": "What A F***ing Waste Of Time",
"JUZ": "Just",
"SLAP": "Sounds Like A Plan",
"PBWY": "Peace Be With You",
"LGMH": "Love Gives Me Hope",
"PBWU": "Peace Be With You",
"PERVING": "Flirting in a creepy way",
"BANDWIDTH": "Amount of data through an interface over time",
"DPYN": "Don't Pick Your Nose",
"ONNTA": "Oh No, Not This Again",
"TWEAKED": "High on drugs",
"4RL?": "For Real?",
"KNOCKED UP": "Pregnant",
"JAJA": "Spanish laugh",
"JWD": "Job Well Done",
"CUCKOO": "Mad| bonkers| crazy",
"JWG": "Just a Wild Guess",
"(U)": "Heartbroken",
"GETCHA": "Get You| understand| Get Your",
"COPYPASTA": "Post that has been copied and pasted",
"JWU": "Just Woke Up",
"JWW": "Just Was Wondering",
"TOFU": "Bean curd",
"TOFT": "Take One For the Team",
"CAKEHOLE": "Mouth",
"SLFN": "So Long For Now",
"DAYUM": "Damn",
"IKHYF": "I Know How You Feel",
"KDOT": "Okay, period",
"IOIC": "If Only I Could",
"CMYK": "Cyan, Magenta, Yellow, Key/blacK",
"HOLLER": "Shout",
"SLEW": "Cut down| kill",
"NMWH": "No Matter What Happens",
"POOL SHARK": "Good pool player",
"AGRO": "Aggravation",
"LY4E": "Love You Forever",
"RICE": "Race Inspired Cosmetic Enhancements",
"8BALL": "1/8 ounce of cocaine",
"AFAIAA": "As Far As I Am Aware",
"AFAIAC": "As Far As I Am Concerned",
"JAND": "England",
"G O A T": "Greatest Of All Time",
"JANK": "Worthless| useless",
"BRICKS": "Drugs| dope",
"LGTM": "Looks Good To Me",
"(Y)": "Thumbs up (MSN)",
"CARRIED": "Told off| DISSed",
"KUSH": "High grade marijuana",
"JRPG": "Japanese Role Playing Game",
"BIG UP": "respect",
"GIEF": "Give",
"TOKE": "Inhale marijuana smoke",
"LSHIDMT": "Laughing So Hard I Drop My Taco",
"SUMMAT": "Something",
"IONO": "I Don't Know",
"FAMBO": "Family| close friend",
"OKW/E": "OK What-Ever",
"TONG": "Asian gangster",
"TOTUS": "Teleprompter Of The United States",
"FFFA": "Fit From Far Away",
"TONK": "Strong| muscular",
"GIGO": "Garbage In, Garbage Out",
"BESITOS": "Little kisses",
"IOMW": "I'm On My Way",
"TOMO": "Tomorrow",
"WGWG": "White Guy With Guitar",
"TOPS": "Wonderful| fantastic",
"FWIW": "For What It's Worth",
"TOOC": "Totally Out Of Control",
"9_9": "Rolling the eyes",
"KAM": "Gorgeous| sexy",
"PTDR": "Pete De Rire| LOL",
"TOON": "Online character in RPG",
"TOOL": "Person of low intelligence| a fool",
"ZONKED": "Very tired| inebriated",
"KAY": "Okay",
"SLOB": "Lazy| untidy| dirty person",
"E2EG": "Ear To Ear Grin",
"SLOL": "Seriously Laughing Out Loud",
"JATQ": "Just Answer The Question",
"UARK": "You Already Know",
"CANTO": "Cantonese",
"UB3R": "Uber| very",
"JAVA": "Programming language| Coffee",
"FRONT": "Pretend| falsely represent",
"TOTE": "Carry",
"TOTD": "Tip Of The Day| Tweet Of The Day",
"TOTB": "Think Outside the Box",
"TOTM": "Time Of The Month",
"FROOB": "Free Noob. Non-paying player",
"KDR": "Kill/Death Ratio",
"TOTT": "Think on These Things",
"TOTS": "Totally",
"RINO": "Republican In Name Only",
"GIMP": "Uncool| Stupid person",
"*$$": "Starbucks",
"ETID": "EveryTime I Die (band)",
"LOADED": "Drunk| intoxicated| Rich",
"TMTOWTDI": "There's More Than One Way To Do It",
"RUMBLE": "Brawl| fight",
"KEK": "LOL in WOW",
"KEN": "Know",
"CHEDDAR": "Money",
"KET": "Ketamine",
"KEW": "KEWL",
"KEY": "Kilogram",
"CANON": "Original| official",
"SHMILY": "See How Much I Love You",
"ODAAT": "One Day At A Time",
"KFC": "Kentucky Fried Chicken",
"QFMT": "Quoted For More Truth",
"TOTZ": "Totally",
"KGB": "Russian secret police",
"JAWN": "Thing",
"TERRIBAD": "Terrible and Bad, awful",
"KGO": "OK, Go",
"MKAY": "Okay",
"TISNF": "That is so not fair",
"HOOCHIFIED": "Dressed Provocatively",
"IDDQD": "Cheat code for Doom",
"SHERM": "Cigarette dipped in PCP",
"WYBD": "What've You Been Doing?",
"SKRILLA": "Money",
"GRENADE": "Ugly girl in a group",
"IOYK": "If Only You Knew",
"KHZ": "Kilohertz",
"KIA": "Killed In Action| Know It All",
"KIG": "Keep It Gangsta",
"KIK": "LOL| Smartphone app for free chat",
"KIM": "Keep It Moving",
"KIO": "Knock it off",
"KIP": "Sleep| Bed",
"KIS": "Keep It Simple",
"KIR": "Keepin' It Real",
"AYCS": "As You Can See",
"KIU": "Keep It Up",
"KIT": "Keep In Touch",
"KIV": "Keep In View",
"XKCD": "Web comic",
"WYDN": "What You Doing Now?",
"AYEC": "At Your Earliest Convenience",
"COURSE": "Of course, certainly",
"RITE": "Right| Write",
"ITTET": "In These Tough Economic Times",
"HUNNIT": "Hundred dollars",
"SRSLY": "Seriously",
"QWERTYUIOP": "Bored",
"\M/": "Horns| metal salute",
"WYCM": "Will You Call Me?",
"KKK": "Ku Klux Klan",
"WDYMBT": "What Do You Mean By That?",
"BREDA": "Boy| man| dude",
"RAPPER": "Urban poet",
"KKZ": "Okay",
"HENCH": "Strong| muscular| well built",
"CLIQUE": "Group of friends",
"KLK": "Que lo que?| What's Up?",
"KLM": "Koninklijke Luchtvaart Maatschappij| Royal Dutch Airlines",
"BREAD": "Money",
"AHEM": "Throat clearing sound",
"SARGE": "Chat up",
"SMDHE": "Some Mothers Do Have Em",
";-)": "Wink",
"AYFT": "Are You Free Today?",
"FFTL": "From First To Last (band)",
"LOCAL": "Local public house| bar",
"AYFR": "Are You For Real?",
"\O/": "Happy| arms in the air",
"FWTD": "Fate Worse Than Death",
"GALDEM": "Group of girls",
"YDWTK": "You Don't Want to Know",
"KMK": "Kottonmouth Kings (band)",
"KML": "Killing Myself Laughing",
"KMN": "Kill Me Now",
"KMP": "Keep Me Posted",
"KMS": "Kill Myself",
"KMU": "Keep Me Updated",
"KMT": "Kiss My Teeth",
"YALLAH": "Let's Go",
"IITUWUTMAS": "If I Tell You Will You Tell Me A Secret?",
"BOOGER": "Dried nose mucus",
"KNP": "OK, No Problem",
"KNO": "Know",
"GIYF": "Google Is Your Friend",
"USBM": "United States Black Metal",
"KEED": "Kid, joke",
"CHADROOL": "Stupid person",
"KOD": "Kiss Of Death",
"KOI": "Koibito| lover",
"USAF": "United States Air Force",
"*.*": "Every file on a computer",
"JACKASS": "Stupid person| idiot",
"KOL": "Kingdom Of Loathing (Game)",
"KOO": "Cool",
"WHEW": "relief| amazement",
"2G2BT": "Too Good To Be True",
"KOS": "Kill On Sight",
"PTSD": "Post Traumatic Stress Disorder",
"KPC": "Keep Parents Clueless",
"KPG": "Keep the Party Going",
"YANNO": "You Know",
"AYKM": "Are You Kidding Me?",
"LYKE": "Like",
"KPI": "Key Performance Indicator",
"SARCY": "Sarcastic",
"IALTO": "I Always Love That One",
"TPAM": "The Person Above Me",
"WHIP": "Expensive car",
"NETIQUETTE": "Etiquette on the internet",
"D00D": "Dude",
"KEEL": "Kill",
"USCG": "United States Coast Guard",
"L2M": "Listening To Music",
"SLAPPED": "Drunk",
"L2P": "Learn To Play",
"RIYL": "Recommended If You Like",
"L2R": "Learn to Read",
"FFWF": "Fight Fire With Fire",
"VIXEN": "Attractive woman",
"VEEP": "VP| Vice President",
"FICCL": "Frankly, I Couldn't Care Less",
"WYLT": "What You Listening To?",
"KRO": "Keep Right On",
"LYMI": "Love You, Mean It",
"SAYONARA": "Goodbye",
"GLAMPING": "Glamorous/luxury camping",
"LYMY": "Love You, Miss You",
"SMAD": "Sad and mad",
"TOUCH": "Beat| kill| rob",
"L4D": "Left 4 Dead (game)",
"WWJBD": "What Would Jason Bourne Do?",
"UC&P": "Up Close and Personal",
"TWEAKING": "SPUN",
"PEACE OUT": "Goodbye",
"LYLC": "Love You Like Crazy",
"WHIZ": "Talented person",
"USER": "Someone who uses other people to gain an advantage",
"LYLT": "Love You Long Time",
"KTB": "Ok, Thanks, Bye",
"KTF": "Keep The Faith",
"4SHO": "For Sure",
"AYOR": "At Your Own Risk",
"WAYOA": "What Are You On About?",
"KTY": "Ok, Thank You",
"NNTP": "Network News Transfer Protocol",
"NNTR": "No Need To Reply",
"DIQUE": "Supposedly",
"NNTO": "No Need To Open",
"KUL": "Cool",
"ALKQN": "Almighty Latin King and Queen Nation",
"WYMM": "Will You Marry Me?",
"SMAO": "Study My Ass Off",
"AYPI": "And Your Point Is?",
"DRIVE BY": "Recce| check out",
"SHINNY": "Informal game of hockey",
"KVM": "Keyboard, Video, Mouse",
"LEDGE": "Legend| legendary",
"DUCWIC": "Do You See What I See?",
"GOOML": "Get Out Of My Life",
"SHUBZ": "House party| rave",
"WHOA": "surprise| Slow down",
"MINGER": "Ugly person",
"GOOMF": "Get Out Of My Face",
"DUTTY": "Dirty",
"PLANKING": "Lying face down in an unusual place",
"BUMMER": "Bad Luck",
"JHEEZE": "OMG| wow",
"PEN15": "Penis",
"SMEG": "All purpose swear word",
"STELLA": "Flirty girl",
"KWL": "Cool",
"BUMMED": "Disappointed| upset",
"L8R": "Later",
"BIMBO": "Stupid girl| usually pretty",
"DWISNWID": "Do What I Say Not What I Do.",
"SMDH": "Shaking My Damn Head",
"WAYLT": "What Are You Listening To?",
"PRBLY": "Probably",
"AYBRB2U": "All Your Base Are Belong To Us",
"LYSM": "Love You So Much",
"CUZZO": "Friend",
"ASPCA": "American Society for the Prevention of Cruelty to Animals",
"JSMN": "Just Shoot Me Now",
"SMFT": "So Much For That",
"MAUI WOWIE": "Marijuana| weed from Hawaii",
"USMC": "United States Marine Corps",
"DEECE": "Decent| cool| great",
"COPACETIC": "Cool| OK, excellent",
"VENT": "Ventrilo",
"LYTD": "Love You To Death",
"PORCH MONKEY": "lazy person",
"KYO": "Knock Yourself Out",
"WHPH": "Work Hard, Play Hard",
"INSIDER": "Inside joke",
"KYP": "Keep You Posted",
"KYS": "Kill Yourself",
"SHAWTY": "Young girl/woman",
"ITALY": "I Trust And Love You",
"LWICD": "Look What I Can Do",
"HELLA": "Hell Yeah",
"DYKWIM": "Do You Know What I Mean?",
"YNWA": "You'll Never Walk Alone",
"BLUE CHIP": "High-quality| low-risk",
"GAFIA": "Get Away From It All",
"SMIB": "Southern Maryland Inbred",
"MTLBWY": "May The Lord Be With You",
"USOA": "USA",
"J DUB": "Jehovah's Witness",
"OFWGKTA": "Odd Future Wolf Gang Kill Them All",
"WYTA": "What You Talking About?",
"MOBBING": "Bullying",
"SMHL": "Shaking My Head Laughing",
"USPS": "United States Postal Service",
"WAYWO": "What Are You Working On?",
"DRAG": "Inhale smoke from cigarette or joint| Dressed as a woman",
"DQYDJ": "Don't Quit Your Day Job",
"O/\O": "High five",
"VERS": "Versatile",
"WHUT": "What",
"WYWH": "Wish You Were Here",
"IPOD": "MP3 player from Apple",
"WYUT": "What You Up To",
"4SRS": "For Serious?",
">//<": "Embarrassed| blushing",
"AYYO": "Hi, hello",
"CREEPER": "Someone who views your profile but doesn't make contact",
"USSR": "Union of Soviet Socialist Republics",
"PLONKER": "Idiot",
"DRAW": "Cannabis",
"LAA": "Lad| mate",
"YEHET": "Yes In Sehun's Happy Mode",
"LAD": "Guy",
"LAG": "Slow response",
"LAL": "Laughing A Lot/Little",
"LAK": "Love And Kisses",
"LAN": "Local Area Network",
"LAM": "Leave A Message| Run",
"LAT": "Living Apart Together",
"GINGE": "Person with red hair",
"LAV": "Lavatory| toilet| Love",
"LAU": "Laughing At You",
"LAX": "Lacrosse",
"CLUTCH": "Perform under pressure",
"WOO HOO": "excitement",
"KEWL": "Cool",
"LBC": "Long Beach, California",
"DRED": "Cruel| bad",
"LBD": "Little Black Dress",
"2MOZ": "Tomorrow",
"LBH": "Let's Be Honest| Loser Back Home",
"KEWT": "Cute",
"SMOG": "Sweet Mother Of God",
"WAYTA": "What Are You Talking About",
"MOLLY": "MDMA| ecstasy",
"LBM": "Little Big Man",
"HMPF": "Sound made when irritated",
"HMPH": "apathy| disagreement",
"LBS": "Laughing But Serious| Weight",
"LBR": "Loser Beyond Repair",
"SMOP": "Simple Matter Of Programming",
"LBW": "Leg Before Wicket (cricket)",
"PUFF": "Marijuana",
"DADA": "Defence Against the Dark Arts (Harry Potter)",
"CHIPOTLE": "Mexican-style restaurant",
"UBRS": "Upper Black Rock Spire",
"LCD": "Liquid Crystal Display",
"WHYB": "Where Have You Been?",
"WHYD": "What Have You Done?",
"PDEA": "Public Display of Elderly Affection",
"LHYW": "Like Hell You Will",
"JSUK": "Just So You Know",
"COD4": "Call Of Duty 4 (Game)",
"CLOUD": "The Internet",
"LDA": "Long Distance Affair",
"TPTB": "The Powers That Be",
"2MRO": "Tomorrow",
"PEBKAC": "Problem Exists Between Keyboard And Chair",
"LDI": "Let's Do It",
"LDH": "Long Distance Hug",
"DADT": "Don't Ask, Don't Tell",
"CUSHTI": "Good| great",
"LDL": "Long Distance Lover",
"LDO": "Like Duh| Obviously",
"KEYS": "Kilogram",
"LDN": "London",
"LDS": "Latter Day Saint",
"ETHERNET": "LAN",
"LDR": "Long Distance Relationship",
"SIZZLE": "Drug OC| oxycontin",
"BLAR": "boredom| BLAH",
"DUCES": "Goodbye",
"OREO": "Black on the outside, white on the inside",
"LED": "Light Emiting Diode",
"M&D": "Mum and Dad",
"DAMHIKT": "Don't Ask Me How I Know That/This",
"LEC": "Local Exchange Carrier",
"BLAH": "Nothing to say",
"REDBONE": "Light skinned colored person",
"EDGY": "Cutting edge",
"BLAD": "Brother| friend",
"CODE": "Computer program| Write a computer program",
"LEL": "LOL",
"M&M": "Small chocolate candy",
"2MRW": "Tomorrow",
"LEO": "Law Enforcement Officer",
"M&S": "Marks & Spencer",
"LEV": "Low Emission Vehicle",
"EULA": "End User License Agreement",
"SOIDH": "Screenshot Or It Didn't Happen",
"YODO": "You Only Die Once",
"LFG": "Looking For Group",
"LFM": "Looking For More",
"COED": "Coeducational (men and women together)",
"LFP": "Looking For Party",
"LFW": "Looking For Work",
"JONES": "Craving| desire",
"LGD": "Let's Get Drunk",
"LGO": "Life Goes On",
"MLAS": "My Lips Are Sealed",
"LGT": "Link Goes To ...",
"JSYK": "Just So You Know",
"DAGO": "Offensive term for an Italian",
"AC/DC": "Rock Band",
"SHUSH": "Shut up| be quiet",
"PENG A LENG": "Really sexy| very fit",
"THRASH": "Thrash metal",
"LHH": "Laughing Hella Hard",
"LHK": "Love, Hugs and Kisses",
"LHM": "Lord Have Mercy| Lord Help Me",
"WUBU2": "What Have You Been Up To?",
"@}--->----": "A rose",
"LHR": "London Heathrow airport",
"SPLIF": "A cigarette of tobacco and cannibis",
"PULL": "Make out with",
"JTLUK": "Just To Let You Know",
"LIB": "Liberal",
"LID": "Ounce of marijuana",
"LIC": "Like I Care",
"SMTH": "Something",
"LIH": "Laugh In Head",
"LIG": "Let It Go",
"*G*": "Grin",
"BLEM": "High on marijuana",
"DUTCH": "Joint| Split the cost",
"LIL": "Little",
"SMTP": "Simple Mail Transfer Protocol",
"BLEH": "boredom",
"LIQ": "Liquor",
"LIT": "Intoxicated| stoned",
"LIS": "Laughing In Silence",
"LIU": "Look It Up",
"PUMA": "Woman who dates younger men",
"HMUL": "Hit Me Up Later",
"W8AM": "Wait A Minute",
"COOL BEANS": "Cool| awesome| great",
"ORLY": "Oh Really",
"CLOUD 9": "Feeling of euphoria| high",
"ZAIN": "Beautiful",
"PUNK": "Music genre",
"AIDS": "Acquired Immune Deficiency Syndrome",
"SELCA": "Self Cam picture",
"MUNTER": "Ugly woman",
"HMWK": "Homework",
"KWAN": "Love and Respect| Extraordinary",
"MUNTED": "Drunk| intoxicated",
"KWAM": "KWIM",
"LIEK": "Like",
"LLC": "Laughing Like Crazy",
"UNESCO": "United Nations Educational, Scientific and Cultural Organization",
"BUGGER OFF": "Go Away",
"PLANK": "Idiot| stupid person",
"LLH": "Laughing Like Hell",
"LLL": "Loony Liberal Left",
"LLO": "LOL",
"GEOCACHE": "Scavenger hunt destination point",
"LLN": "LOL LOL Neanderthal",
"^.^": "Happy",
"NEWPORT": "Cigarette brand",
"NOLA": "New Orleans, Louisana",
"MLIA": "My Life Is Average",
"AFAICS": "As Far As I Can See",
"COKE": "Cocaine| Coca Cola",
"MLIG": "My Life Is Great/Good",
"AFAICT": "As Far As I Can Tell",
"LLZ": "LOLZ",
"MLIB": "My Life Is Bro",
"LMA": "Leave Me Alone",
"SWISHER": "Cheap cigar used for making BLUNTs",
"LMC": "Let Me Check| Let Me See",
"OTOOH": "On The Other Other Hand",
"PDOA": "Public Display Of Affection",
"DANK": "Very potent marijuana| Incredibly good| awesome",
"LMG": "Light Machine Gun",
"DANG": "Damn",
"LML": "Love My Life",
"LMK": "Let Me Know",
"JTLYK": "Just To Let You Know",
"LMO": "Leave Me Alone",
"^.~": "Wink",
"LMR": "Last Minute Resistance",
"BLIM": "Small piece of HASH",
"LMT": "Let Me Think",
"LMS": "Like My Status| Last Man Standing",
"COLA": "Top bud of a marijuana plant",
"NOKD": "Not Our Kind, Dear",
"MYGGWY": "May Your God Go With You",
"YOMB": "You Owe Me Big",
"NOMS": "Food",
"LNF": "Life's Not Fair",
"M/F": "Male/Female",
"LNK": "Love And Kisses",
"<.<": "Shifty look",
"M/O": "Make Out",
"NAHMEAN": "You know what I mean",
"ENIAC": "Electronic Numerical Integrator and Computer",
"WIGO": "What Is Going On?",
"LNT": "Leave No Trace",
"MARY JANE": "Marijuana",
"TOTGA": "The One That Got Away",
"AIGF": "All In Good Fun",
"HOMMIE": "HOMIE",
"ROOTS": "Type of Reggae music",
"UCAP": "Up Close and Personal",
"LOB": "Throw| Line Of Business",
"NONG": "Idiot| twit",
"LOA": "Leave of Absence",
"</3": "Broken Hearted",
"LOC": "Library Of Congress",
"YOLA": "Cocaine",
"LIFO": "Last In First Out",
"PURP": "PURPLE",
"WAIFU": "Wife",
"LOI": "Laughing On the Inside",
"MLIM": "My Life Is Magic",
"LOL": "Laughing Out Loud",
"LOK": "LOL OK",
"NOLY": "No One Likes You",
"PREGGO": "Pregnant",
"LOP": "Stupid person",
"CONT": "Continued",
"LOO": "Toilet",
"LOQ": "Laugh Out Quietly",
"KWEL": "Cool",
"YOLO": "You Only Live Once",
"LOS": "Line Of Site",
"NOMB": "None Of My Business",
"LOW": "Unfair| immoral",
"LOZ": "Legend of Zelda",
"YOLF": "You Only Live Forever",
"L8ER": "Later",
"WIFF": "With",
"FO'SHIZZLE": "For Sure",
"WIFI": "Wireless Fidelity| wireless internet",
"PUSH": "Sell drugs",
"LPG": "Liquefied Petroleum Gas",
"YOOH": "You",
"LPL": "LOL",
"NOOBLET": "Young NOOB",
"AIIT": "Alright",
"LPT": "Life Pro Tip",
"NOPE": "No",
"AIIC": "As If I Care",
"HIPSTER": "Person who is HIP",
"M2D": "Make my day",
"TOTES": "Totally",
"({)(})": "Two People Hugging",
"*O*": "Confused| surprised| shocked",
"LQI": "Laughing Quietly Inside",
";P;": "Misspelling of LOL",
"DARN": "Damn",
"LQL": "Laughing Quite Loudly",
"DARL": "Darling",
"NONY": "Anonymous person",
"TOTEZ": "Totally",
"POKEY": "Prison",
"DARE": "Drug Abuse Resistance Education",
"NOOB": "New, inexperienced person",
"TOASTIE": "Toasted sandwich",
"MMORPG": "Massively Multiplayer Online Role Playing Game",
"LRG": "Lifted Research Group (clothing)",
"LRF": "Low Resolution Fox| attractive at a distance",
"UTFL": "Use The Force, Luke",
"LRL": "Laughing Really Loud",
"BLNT": "Better Luck Next Time",
"LRT": "Last ReTweet",
"XLNT": "Excellent",
"BLUNT": "Marijuana cigar",
"NORM": "Normal",
"WIKI": "Collaborative web site",
"CRIMBO": "Christmas",
"LSA": "D-Lysergic Acid Amide similar to LSD",
"LSD": "D-Lysergic Acid Diethylamide",
"LSE": "London School of Economics| Low Self-Esteem",
"LSH": "Laughing So Hard",
"LSI": "Limited Social Interaction",
"DATO": "Dining At The Orifice",
"LSP": "Love Sick Puppy",
"LSR": "Loser",
"BLOW": "Cocaine",
"FEENIN": "An Intense Craving Or Yearning",
"LSS": "Last Song Syndrome",
"CUIAB": "See You In A Bit",
"M4W": "Man looking for a woman",
"KWIM": "Know What I Mean?",
"KWIS": "Know What I'm Saying?",
"BLOL": "Big Laugh Out Loud",
"LTA": "Love To All",
"LTB": "Looking To Buy",
"BLOG": "Weblog| online diary",
"LTD": "Live/Living The Dream",
"<3U": "Love You",
"LTG": "Learn To Google",
"LTH": "Love Truth Honor",
"COTD": "Comment Of The Day",
"LTM": "Laughing To Myself",
"LTL": "Living The Life",
"ORITE": "Alright, hello",
"LTP": "Lost The Plot",
"FIDDY": "Fifty",
"LTS": "Laugh To Self",
"NOTA": "None Of The Above",
"LTR": "Long term relationship",
"NOTB": "None Of Their Business",
"LTY": "Laugh To Yourself",
"CHOICE": "Nice| great",
"AIMO": "Idiot",
"OAUS": "On An Unrelated Subject",
"LINE": "A line of a powdered drug",
"LUB": "Love",
"LUA": "Love You Always",
"AIMH": "Always In My Heart",
"WIMP": "Weak| cowardly person",
"LUH": "Love",
"*S*": "Salute",
"LUI": "Living Under the Infuence",
"LUL": "Lame Uncomfortable Laugh",
"MISSION": "Arduous or boring task",
"SNAG": "Sensitive New Age Guy| Sausage",
"NOSA": "No Sir, just kidding",
"LUV": "Love",
"SNAP": "dismay| surprise",
"LUX": "Luxury",
"AINT": "Am not| are not| is not",
"LIMB": "Laughing In My Brain",
"LIME": "Hang out| socialise",
"LIMH": "Laughing In My Head",
"LIMO": "Limousine| luxury car",
"LIMM": "Laughing In My Mind",
"LVL": "Level",
"MLRS": "MLAS",
"NOUT": "Nothing",
"SHURUP": "Shut Up",
"DAWG": "Close Friend",
"YOUS": "You| you guys",
"RKBA": "Right to Keep and Bear Arms",
"NOTW": "Not Of This World",
"LWM": "Laugh With Me",
"LWP": "Laughing With Passion",
"LWT": "Look Who's Talking",
"GUCCI": "Good| awesome| high quality",
"XLR8": "Accelerate",
"LXG": "League of Extraordinary Gentlemen",
"BM4L": "Best Mates For Life",
"HOLLA": "Used by a man to express interest in a woman",
"BLUF": "Bottom Line Up Front",
"LY2": "Love You Too",
"BLUE": "Sad",
"BLUD": "Mate",
"ACAPELLA": "Vocal music without instruments",
"LIRL": "Laughing In Real Life",
"LYB": "Love You, Bye",
"LYA": "Love You Always",
"LYF": "Love You Forever",
"STELLAR": "Awesome| excellent",
"BLTN": "Better Late Than Never",
"LYL": "Love You Lots",
"LYK": "Let you know",
"AISI": "As I See it",
"LYM": "Love You More",
"FLAGGED": "Banned",
"LYR": "Love You Really",
"COYG": "Come On You Gunners",
"LYT": "Love You Too",
"AISB": "As I Said Before",
"COYB": "Come On You Blues",
"SNES": "Super Nintendo Entertainment System",
"MLTR": "Multiple Long Term Relationships",
"NOWT": "Nothing",
"WYLION": "Whether You Like It Or Not",
"HACER": "Hacker",
"JTOL": "Just Thinking Out Loud",
"YOYO": "You're On Your Own",
"AITE": "Alright",
"LITE": "Light",
"PRESH": "Precious| Awesome| cool",
"JTOY": "Just Thinking Of You",
"DIPPED": "Fashionable",
"PMIGBOM": "Put Mind In Gear, Before Opening Mouth!",
"COYS": "Come On You Spurs",
"COYR": "Come On You Reds",
"FYEO": "For Your Eyes Only",
"ALLOW": "Let it be| forget it",
"INTEL": "Intelligence| CPU manufacturer",
"LITS": "Life Is Too Short",
"AIUI": "As I Understand it",
"UCLA": "University of California at Los Angeles",
"NOYB": "None Of Your Business",
"CRICKETS": "Said after a lame joke to break the silence",
"DEFFO": "Definitely",
"MOTORBOATING": "Nuzzling a woman's chest whilst making a noise",
"4COL": "For Crying Out Loud",
"AITR": "Adult In The Room",
"BOOZER": "Public House| bar",
"EOTWAWKI": "End Of The World As We Know It",
"IANAL": "I Am Not A Lawyer",
"PEAR": "Pear shaped| wrong",
"L8TR": "Later",
"PADDY": "Offensive term for an Irishman",
"LIVE": "Exciting",
"MLYP": "Much Like Your Post",
"BRITCHES": "Pants| trousers",
"TR00": "True| loyal",
"ASDFGHJKL;": "I'm bored",
"CAMPER": "Player who lies in wait for a target",
"CELLY": "Cellphone| Prison cell mate",
"TMSIDK": "Tell Me Something I Don't Know",
"CAN DO": "It will be done",
"SPLIFF": "SPLIF",
"<><": "Fish",
"IANAD": "I Am Not A Doctor",
"PEAK": "Humiliating",
"BEAST": "Person/thing that is good, awesome",
"WITW": "What In The World",
"WYLABOCTGWTR": "Would You Like A Bowl Of Cream To Go With That Remark?",
"LEG IT": "Run away",
"BDONG": "Bad and Wrong",
"BOOMM": "Bored Out Of My Mind",
"WIWT": "What I Wore Today| Wish I Was There",
"WHISKEY TANGO": "WT| White Trash",
"BOOMS": "BOOMERS",
"RACHET": "Crazy| nasty",
"AIYC": "As If You Care",
"AIYA": "Cantonese expression of exasperation",
"MAC": "Media Access Control| Macintosh",
"MAD": "Really| extremely",
"GLIRL": "Good Luck In Real Life",
"HUNDO": "Hundred dollars",
"MAG": "Magazine",
"MAH": "My",
"MAJ": "Major",
"MAL": "Mean, bad, evil",
"MAP": "Man-Alien-Predator",
"MAS": "Mildly Amused Smirk",
"TTBOMK": "To The Best Of My Knowledge",
"MOOLA": "Money",
"WIYP": "What Is Your Problem?",
"MAW": "Might As Well",
"MUGSHOT": "Head and shoulders picture",
"NPAA": "No Problem At All",
"HNOS": "Home Network Operating System",
"MBA": "Masters of Business Association| Married But Available",
"MBD": "My Bad Dudes",
"OBBY": "Oh, Baby",
"MBF": "My Best Friend",
"*_*": "In love| dazed",
"SNOG": "Kiss",
"MBN": "Must Be Nice",
"MBM": "Married Black Male",
"MBO": "Must Be Off| Management Buy-Out",
"MBR": "Master Boot Record",
"MBT": "Masai Barefoot Technology",
"BEANS": "Ecstacy",
"MBW": "Married Black Woman",
"TOMBOUT": "Talking About",
"IDFLI": "I Don't Feel Like It",
"TYSVM": "Thank You So Very Much",
"JTTY": "Just To Tell You",
"BDOML": "Best Day Of My Life",
"MCD": "McDonald's",
"MY BOO": "My Boyfriend| My girlfriend",
"MCM": "Man Crush Monday",
"MCL": "Much Clown Love",
"MCP": "Male Chauvanist Pig",
"MCR": "My Chemical Romance (band)",
"WIBNI": "Wouldn't It Be Nice If",
"FRONTING": "Misrepresenting oneself",
"MDA": "Drug related to Ecstasy (MDMA)",
"MDF": "Medium Density Fiberboard",
"J2LUK": "Just To Let You Know",
"CRUSTY": "Dirty| unwashed person",
"MDK": "Murder Death Kill",
"CRAIC": "Good times| gossip| music| drinking",
"MDR": "Mort De Rire| LOL",
"CHUNG": "Good looking| buff",
"ME2": "Me Too",
"5 HOLE": "Gap between goalie's legs in hockey",
"DUDETTE": "Female dude",
"MDW": "Memorial Day Weekend",
"ROACH": "The end of a JOINT",
"SNOW": "Cocaine",
"BMAK": "Blow Me A Kiss",
"MEC": "Dude",
"MED": "Medication",
"MEG": "Megabyte",
"MEH": "Indifference| equivalent to shoulder shrug",
"D BO": "Steal",
"MEP": "Member of the European Parliament",
"OSFA": "One Size Fits All",
"MER": "boredom| nonchalance",
"MEX": "Low grade marijuana from Mexico",
"PARTY POOPER": "A person who ruins the mood with their attitude",
"BMBO": "Blow My Brains Out",
"SNSD": "Sho Nyo Shi Dae (Girl's Generation) Korean girl group",
"MFC": "Mildly Fat Chick| Microsoft Foundation Classes",
"MFG": "Mit Freundlichen Gruessen",
"MFL": "Marked For Later",
"CHULA": "Cute| sexy",
"SMEXY": "Smart and sexy",
"MFW": "My Face When ...",
"OTAKU": "Obsessive fan",
"SMEXI": "Smart and Sexy",
"MMAO": "Make Me An Offer",
"MGD": "Miller Genuine Draft",
"TINALO": "This Is Not A Legal Opinion",
"MGK": "Machine Gun Kelly",
"MGM": "Male Genital Mutilation",
"MGO": "Metal Gear Online (game)",
"GET DOWN": "Party| dance",
"MGS": "Metal Gear Solid (game)",
"THANG": "Thing",
".BIZ": "Top level domain for business",
"<G>": "Grin",
"MHM": "MHMM",
"MHO": "My Honest Opinion",
"SPOT ON": "Absolutely correct| perfect",
"MHR": "Many Happy Returns",
"THANX": "Thanks",
"TROLL": "A deliberately provocative message board user",
"ICSTAY": "I Can't Stop Thinking About You",
"MI6": "Military Intelligence Service 6",
"REEFER": "Marijuana",
"J2LYK": "Just To Let You Know",
"MHZ": "Megahertz",
"MIA": "Missing In Action",
"MIC": "Microphone",
"MIB": "Men In Black| Mint In Box",
"MID": "Mid grade marijuana| Mobile Internet Device",
"BBIAB": "Be Back In A Bit",
"MIH": "Make It Happen",
"BBIAF": "Be Back In A Few",
"MIL": "Mother In Law",
"PLUMBUM": "Plumber's crack",
"MIN": "Minute",
"BBIAM": "Be Back In A Minute",
"MIP": "Minor In Possession",
"MAWOY": "May Angels Watch Over You",
"MIS": "Miserable",
"IDWTK": "I Don't Want to Know",
"SNIFF": "Cocaine",
"MIT": "Massachusetts Institute of Technology",
"BBIAS": "Be Back In A Sec",
"KNOCKOUT": "Stunning person or thing",
"SOLID": "Cool| awesome",
"BBIAW": "Be Back In A While",
"MINES": "Mine",
"MIZ": "Miserable",
"ITIAILWY": "I Think I Am In Love With You",
"SPOON": "Cuddle",
"OSLT": "Or Something Like That",
"PENG": "Good looking person",
"CRACK": "A form of cocaine",
"BMFL": "Best Mates For Life",
"NEWBIE": "New Person",
"KTBSPA": "Keep The Backstreet Pride Alive",
"BMFE": "Best Mates Forever",
"SPOOF": "Tube used to reduce smoking smells",
"2NYT": "Tonight",
"LJBF": "Let's Just Be Friends",
"NEAT-O": "Cool",
"DSLR": "Digital Single Lens Reflex| Camera",
"2NITE": "Tonight",
"MKZ": "Facebook",
"CIGGY": "Cigarette",
"MLB": "Major League Baseball",
"MLG": "Major League Gaming",
"SNYK": "So Now You Know",
"JUAD": "Jumping Up And Down",
"MLM": "Middle finger (mlm)",
"MLP": "My Little Pony",
"MM2": "Message Mode 2",
"MLS": "My Life Sucks",
"PEPS": "People",
"AWESOME": "Amazing| wonderful| cool",
"MMA": "Meet Me At...",
"MMC": "MultiMedia Card",
"MMD": "Make My Day",
"MMI": "Me Myself and I",
"MMH": "Meet Me Halfway",
"MMK": "Mm OK",
"MMJ": "Medical Marijuana",
"TFM2KAU2FO": "That's For Me to Know And You To Find Out",
"MMM": "pleasure| contentment",
"MML": "Made Me Laugh",
"MMO": "Short for MMORPG",
"PEON": "Low ranking person",
"MMS": "Multimedia Messaging Service",
"MMT": "Makes Me Think",
"NIHAO": "Hello",
"MMV": "Mileage May Vary",
"DBMS": "Database Management System",
"DEETS": "Details",
"N/A": "NA",
"MNC": "Mobile Network Code",
"N/C": "Not Cool",
"MNF": "Monday Night Football",
"PERF": "Perfect",
"^O^": "Hurrah",
"MNM": "Eminem (rapper)",
"MNT": "Maybe Next Time",
"N/T": "No Text",
"MINGA": "Minger| ugly person",
"PERO": "But",
"DBNT": "Don't Bother Next Time",
"PERV": "Pervert",
"WORD UP": "OK, I agree",
"MOC": "Marriage Of Convenience| My Own Creation",
"MOE": "Fetish| mania| taste",
"MOD": "Modification| 1960s youth culture",
"MOI": "Me",
"DTMWTD": "Don't Tell Me What To Do",
"MOH": "My Other Half",
"MOL": "More Or Less",
"MOO": "Matter of Opinion| My Own Opinion",
"MON": "Man",
"MOQ": "Minimum Order Quantity",
"MOP": "Hair on head",
"MOS": "Mom Over Shoulder",
"MP4": "Music/video file",
"MP3": "Music file",
"MOU": "Memorandum Of Understanding",
"MOT": "Ministry Of Transport test for car safety",
"MP5": "Heckler and Koch submachine gun",
"MOW": "Eat very fast",
"BOGSAT": "Bunch Of Guys Sitting Around Talking",
"SNIDE": "Fake or counterfeit",
"MY SIZE": "My type of girl",
"BMTTVE": "Best Mates 'Til The Very End",
"CAKING": "Flirting",
"MPA": "Music Publishers' Association",
"MPE": "My Point Exactly",
"MPH": "Miles Per Hour",
"MPG": "Miles Per Gallon",
"EVVK": "Couldn't Care Less",
"PETA": "People for the Ethical Treatment of Animals",
"HAISTK": "How Am I Supposed To Know?",
"MPO": "My Personal Opinion",
"BRGDS": "Best Regards",
"MPV": "Multi-Purpose Vehicle",
"TROTW": "The Rest Of The World",
"TRAW": "Thick, Rich And White",
"TBQFH": "To Be Quite F***ing Honest",
"N2B": "Not Too Bad",
"N2G": "Need To Go",
"ADEROL": "Drug that helps you focus",
"N2K": "Nice To Know| Need To Know",
"WALLY": "Stupid person",
"N2M": "Not Too Much",
"NPNT": "No Picture, No Talk",
"ROCKIN": "Awesome",
"N2S": "Needless To Say| Note To Self",
"IRAH": "Insert Relevant Acronym Here",
"TROUT": "Older man looking for a younger woman",
"MRA": "Man bra",
"BMOC": "Big Man On Campus",
"LOLWUT": "What?",
"/THREAD": "End of a thread on a forum",
"HOMEY": "Homeboy",
"MRE": "Meal, Ready to Eat",
"MRI": "Magnetic Resonance Imaging",
"HUGGLE": "Hug and snuggle",
"IRDC": "I Really Don't Care",
"MRP": "Manufacturer's Recommended Price",
"BONZA": "Brilliant| excellent",
"HOMES": "Homeboy",
"IABW": "In A Bad Way",
"IRDK": "I Really Don't Know",
"MRW": "My Reaction When...",
"HOAS": "Hold On A Second",
"SOAD": "System Of A Down (Band)",
"HOAY": "How Old Are You?",
"SOCKPUPPET": "Person using a false identity",
"WALLA": "Voila| here it is",
"MSG": "Message| Monosodium Glutamate",
"MSF": "Male Seeking Female",
"MSI": "Mindless Self Indulgence (band)",
"MSM": "Mainstream Media",
"MSN": "Microsoft Network",
"N4P": "Noob",
"MSR": "Mulder/Scully Romance",
"MST": "Mystery Science Theater",
"TCCIC": "Take Care 'Cause I Care",
"BOGART": "To Hold Onto Something For Oneself When It Is Supposed To Be Shared",
"SNIPE": "Bid on eBay just before the auction ends",
"MTB": "Mountain Bike",
"MTA": "More Than Anything",
"MTD": "Month To Date",
"MTC": "More To Come",
"MTF": "Male To Female",
"MTE": "My Thoughts Exactly",
"MTH": "Month",
"TFLMS": "Thanks For Letting Me Share",
"MTG": "Magic: The Gathering",
"PONED": "Powerfully owned| dominated",
"MTL": "Montreal, Canada",
"LAMBO": "Lamborghini",
"MTO": "Media Take Out",
"IDGAD": "I Don't Give A Damn",
"TADA": "Exclamation when something is finished",
"LADYBOY": "Transvestite",
"MTV": "Music Television",
"FOLDED": "Drunk| wasted",
"BABYSITTING": "Holding the weed too long",
"MTW": "Mum's the Word",
"MUA": "Make-Up Alley",
"THE NICK": "Prison",
"TOWIE": "The Only Way Is Essex",
"MUD": "Multi-User Dungeon",
"MUG": "Rob somebody| Face| Idiot",
"MUH": "Me| My",
"RAILS": "Lines of cocaine",
"MUM": "Mom",
"SOAG": "Son Of A Gun",
"UMTWTM": "You Mean The World To Me",
"WUWCB": "What do You Want to Chat about?",
"MUX": "Multiplex",
"YHGASP": "You Have Got A Serious Problem",
"OBTW": "Oh, By The Way",
"PEWS": "Post Election Withdrawal Syndrome",
"HOBO": "Itinerant worker",
"SHRINK": "Psychiatrist",
"MVA": "Motor Vehicle Accident",
"TREE": "Marijuana",
"TACP": "Take A Chill Pill",
"TACO": "A Mexican Food Dish| Vagina| Derogatory Term For Spanish Person",
"HAYWIRE": "Crazy",
"DOSHA": "Marijuana",
"^W^": "Happy",
"MVP": "Most Valuable Player",
"MW3": "Modern Warfare 3 (game)",
"TAFF": "Person from Cardiff or Wales generally",
"HOOROO": "Goodbye",
"TAFN": "That's All For Now",
"TTUTT": "To Tell You the Truth",
"MWA": "Kiss",
"MWC": "Married With Children",
"HOMIE": "Homeboy",
"LADYBRO": "A Female Friend",
"TAFW": "Time Away From Work",
"MWF": "Married White Female",
"MWI": "Mad With It",
"BMTH": "Bring Me The Horizon (Band)",
"MWM": "Married White Male",
"OBVI": "Obviously",
"GLAG": "Giggling Like A Girl",
"OBVY": "Obviously",
"OBVZ": "Obviously",
"JUJU": "Magic| luck",
"OBVS": "Obviously",
"TFLMK": "Thanks For Letting Me Know",
"RLAB": "Roll Like A Buffalo",
"TRIP": "Experience when using a hallucinogenic drug",
"IDKWTD": "I Don't Know What To Do",
"ASAIK": "As Soon As I Know",
"SOFT": "Powdered cocaine",
"ASAIC": "As Soon As I Can",
"MYB": "Mind Your Business",
"SMURF": "Experienced gamer posing as a newbie",
"MYF": "Miss Your Face",
"TAHT": "That",
"TURN UP": "Get Hyped| Excited| Wild",
"MYL": "Miss You Loads",
"SWALK": "Sealed With A Loving Kiss",
"IRIE": "Feeling good, alright",
"DODGY": "Of questionable legality, morality, integrity",
"MYT": "Meet You There",
"AK47": "Assault rifle",
"TAGL": "There's a good lad",
"BMUS": "Beam me up, Scotty",
"GETTO": "Ghetto",
"BARBIE": "Barbecue",
"EWBA": "Everything Will Be Alright",
"TL;DR": "TLDR",
"IRLY": "I Really Like You",
"TL;DW": "TLDW",
"IRMC": "I Rest My Case",
"SWANK": "Cool| classy| Single Woman And No Kids| Second Wife And No Kids",
"JDLR": "Just Doesn't Look Right",
"HABBO": "Online community/game",
"PMFJI": "Pardon Me For Jumping In",
"DOUGH": "Money| cash",
"IJWTK": "I Just Want To Know",
"HOLLAND": "Hope Our Love Lasts And Never Dies",
"IJWTS": "I Just Want To Say",
"BROCHACHO": "BRO",
"SOHF": "Sense Of Humor Failure",
"GLHF": "Good luck, have fun.",
"SARKY": "Sarcastic",
"BTOBS": "Be There Or Be Square",
"CARDY": "Cardigan",
"YUPPIE": "Young Urban Professional",
"IAMA": "I Am Mildly Amused",
"CHEVY": "Chevrolet",
"DTAM": "Death To All Men",
"HOLA": "Hello",
"FIFA": "Federation International de Football Association",
"HOLD": "Borrow (money)",
"^^^": "Reference to line above",
"COYOTE": "Person who smuggles immigrants",
"IITYWYBMAD": "If I Tell You Will You Buy Me A Drink",
"PATCH": "Correction applied to a computer program",
"TAKS": "Texas Assessment of Knowledge and Skills",
"WIRED": "High on stimulants| Online",
"^_^": "Happiness| satisfied",
"BANGER": "Great party| Old car| Sausage",
"CUMBERBITCH": "Female Fan Of Benedict Cumberbatch",
"HOMG": "OMG",
"TANK": "Large person| heavily built",
"TANJ": "There Ain't No Justice",
"SOMD": "Sh*t On My Dick",
"PISSED": "Angry| Drunk",
"^_~": "Wink",
"SOML": "Story Of My Life",
"CATFISH": "Individual Who Creates a False Online Identity",
"FIFO": "First In First Out",
"NAB": "NOOB",
"HENRY": "High Earner| Not Rich Yet",
"HOLS": "Holidays| vacation",
"NAA": "Not At All",
"TAMW": "That Awkward Moment When ...",
"NAC": "Not A Chance| Not A Chain",
"NAE": "No",
"NAH": "No",
"FIFY": "Fixed It For You",
"NAL": "Need A Life",
"OTAY": "Okay",
"<_<": "Sarcasm| look left",
"NAK": "Nursing At Keyboard",
"NAP": "Not A Problem",
"TROL": "Police patrol",
"NAO": "Now",
"NAS": "Network Attached Storage",
"NAW": "No",
"NAY": "No",
"IB2D": "I Beg To Differ",
"LOTTA": "Lot of",
"NBA": "National Basketball Association",
"HOOT": "Good time| Funny person",
"NBC": "National Broadcasting Company",
"WALLHAX": "Wall hack",
"NBB": "Never Been Better",
"NBD": "No Big Deal",
"NBG": "Nasty But Good",
"NQAT": "No Question About That",
"TDDWP": "Til Death Do We Part",
"NBF": "New Best Friend",
"PSHAW": "disbelief| scepticism",
"NBK": "Natural Born Killers| Never Been Kissed",
"SOOC": "Straight Out Of Camera |not Photoshopped",
"+_+": "Dead man",
"SOOD": "Cool",
"NBM": "Nil By Mouth| Not Before Midnight",
"SOOK": "A Person Who Whines Or Is Overly Sensitive",
"IITWI": "In It To Win It",
"NBT": "Nothing But Trouble",
"DVDRIP": "Video copied from a DVD",
"EDDRESS": "Email address",
"PWNAGE": "Pure Ownage",
"SAMEFAG": "Person with multiple aliases on forums",
"YELLOW BONE": "Light-skinned black woman",
"NCB": "Never Coming Back",
"ALOTBSOL": "Always Look On The Bright Side Of Life",
"PIKEY": "Offensive term for a Gypsy/traveller",
"BESTY": "Best Friend",
"NCO": "Non-Commissioned Officer",
"HOOD": "Neighborhood| ghetto",
"DCED": "Dissconected",
"NOYDB": "None Of Your Damn Business",
"WOTAM": "Waste of Time and Money",
"IGNTS": "I've Got Nothing To Say",
"NDA": "Non-Disclosure Agreement",
"NDE": "Near Death Experience",
"OTFL": "On The Floor Laughing",
"NDI": "Neither Do I",
"WHEELING": "Picking up women",
"BFF4L": "Best Friend Forever For Life",
"GINGER": "Person with red hair",
"NDN": "Indian",
"CHORTLE": "Chuckle and Snort",
"NDP": "New Democratic Party",
"NE1": "Anyone",
"IDEKY": "I Don't Even Know You",
"NDS": "Nintendo DS",
"NDR": "Non-Delivery Report/Receipt",
"PFFT": "dismissal",
"CUL8R": "See You Later",
"IDKWYM": "I Don't Know What You Mean",
"NEA": "National Education Association",
"SOPA": "Stop Online Piracy Act",
"NED": "Non-Educated Delinquent",
"NEF": "Person who posts pointless messages on forums",
"NEE": "Maiden name indicator",
"NEG": "Negative",
"NEI": "Not Enough Information",
"NEL": "No",
"AAMOF": "As A Matter Of Fact",
"TARD": "Retarded person",
"O&O": "Over and Out",
"LOTRO": "Lord Of The Rings Online",
"NER": "Non-Existent Relationship",
"NET": "Internet",
"NES": "Nintendo Entertainment System",
"AYSOS": "Are You Stupid Or Something?",
"NEY": "No",
"UDWK": "You Don't Wanna Know",
"NFA": "No Further Action",
"TATT": "Tattoo",
"LMGTFY": "Let Me Google That For You",
"TATO": "Alright, OK",
"NFI": "No Freakin Idea",
"NFM": "No Further Message",
"NFL": "National Football League",
"BESOS": "Kisses",
"FINA": "About to do",
"NFN": "Normal For Norfolk",
"NFS": "Not For Sale",
"NFT": "No Further Text",
"NFY": "Not Fixed Yet",
"SKOOL": "School",
"NFZ": "No Fly Zone",
"FILO": "First In, Last out",
"WUUP2": "What are You Up To?",
"NGH": "Not Gonna Happen",
"4EVER": "Forever",
"EEPROM": "Erasable EPROM",
"NGL": "Not Gonna Lie",
"NGO": "Non-Governmental Organization",
"THABTO": "Two Heads Are Better Than One",
"NGU": "Never Give Up",
"TATA": "Goodbye",
"SORN": "Statutory Off Road Notification",
"NHB": "No Holding Back| No Holds Barred",
"FLURGE": "Urge To Be Flaccid",
"NHD": "No Harm Done",
"NHF": "No Hard Feelings",
"TDDUP": "Til Death Do Us Part",
"HOVA": "Jehovah| God",
"NHL": "National Hockey League",
"NHN": "Not Here Now",
"NHS": "National Health Service",
"PWND": "PWNED",
"SOUR": "Strong WEED",
"FINK": "Snitch| informer",
"LWKMD": "Laugh Wan Kill Me Die",
"PSYCHED": "Excited| pumped up",
"FINO": "Failure Is Not an Option",
"NIB": "New In Box",
"TITCH": "Small person",
"NID": "Never In Doubt",
"NIC": "Network Interface Card",
"SOTD": "Song Of The Day",
"TITCR": "This Is The Credited Response",
"SKINNY": "Information| gossip| Thin",
"SOTA": "State Of The Art",
"NIL": "Zero| Nothing",
"NIK": "Now I Know",
"NIN": "Nine Inch Nails",
"NIP": "Derogatory term for a Japanese person",
"NIR": "NOOB In Room",
"NIT": "New In Town| Idiot| nitwit",
"SOTU": "State of the Union",
"NIX": "Destroy| kill| Nothing",
"NAPES": "No",
"GLTY": "Good Luck To You",
"SHADDAP": "Shut up",
"PWOS": "Parents Watching Over Shoulder",
"PWOR": "Proceeding With Orders Recieved",
"WOTLK": "Wrath Of The Lich King",
"DTMS": "Does That Make Sense?",
"BOMBSHELL": "Very attractive woman",
"RLTW": "Rangers Lead The Way",
"FIRM": "Gang of soccer hooligans| Organised criminal gang",
"CHELA": "Beer",
"NKA": "No Known Allergies",
"PREGO": "Pregnant",
"PWNT": "PWNED",
"GLTA": "Good Luck to All",
"NKT": "annoyance",
"DAMHIK": "Don't Ask Me How I Know",
"NKZ": "No Kill Zone",
"I18N": "Internationalization",
"NACKERED": "Tired| worn out",
"NLA": "No Longer Available",
"NLI": "Not Logged In",
">,<": "Angry| annoyed",
"NLM": "No Laughing Matter",
"SKETCH": "Unusual| dubious",
"O-O": "Shock",
"FITB": "Fill In The Blanks",
"FITE": "Fight",
"NLT": "No later than...",
"BOOHOO": "Crying",
"OTOH": "On The Other Hand",
"SOYW": "Something On Your Wall",
"MIMBO": "Male BIMBO",
".COM": "commerce",
"MOTWYW": "Make Of That What You Will",
"NMF": "Not My Fault",
"NME": "New Musical Express (magazine)",
"NMH": "Nodding My Head",
"NMG": "No Mames Guey (OMG)",
"NMJ": "Not My Job",
"NMI": "Need More Info",
"NMN": "No Middle Name",
"NMP": "Not My Problem",
"NMO": "Not My Opinion",
"O.O": "Wide eyed",
"OM NOM": "Sound of eating",
"NMR": "Not My Responsibility",
"NMT": "Not My Type",
"NMS": "Not My Style",
"NMU": "Not Much, You?",
"FISH": "First In Still Here",
"NMW": "No Matter What",
"BNIB": "Brand New In Box",
"NMY": "Nice Meeting You",
"TAZE": "Annoy| irritate",
"AITYD": "And I Think You Do",
"CHUNDER": "Be sick| vomit",
"JEAL": "Jealous",
"DCOI": "Don't Count On It",
"KYFC": "Keep Your Fingers Crossed",
">.<": "Frustrated| angry| upset| in pain",
">.>": "Suspicious| wary",
"CLUBHEAD": "Person who goes to clubs a lot",
"NO1": "No-One",
"OUTTA": "Out Of",
"HOMESLICE": "Homeboy",
"GLWS": "Good Luck With Sale",
"NOD": "Doze off",
"GLWT": "Good Luck With That",
"NOC": "No One Cares",
"NOH": "New Overhauled",
"FITY": "Fifty",
"NOI": "No Offense Intended",
"NOK": "Not Okay| No One Knows",
"-.-": "Annoyance",
"NON": "Now Or Never",
"NOM": "No Offence Meant",
"NOP": "Not Our Problem| No Problem",
"TANSTAAFL": "There Ain't No Such Thing as a Free Lunch",
"NOS": "New Old Stock",
"NOV": "Novice",
"NOU": "No, You",
"FAFWOA": "For A Friend Without Access",
"NOZ": "Nitrous oxide| laughing gas",
"HWSNBN": "He Who Shall Not Be Named",
"NPC": "Non Player Character",
"SHORTY": "Girlfriend| attractive girl",
"NPI": "No Pun Intended",
"NPO": "Never Pull Out",
"NPS": "No Problems",
"NPR": "National Public Radio| No Purchase Required",
"TOUCHE": "Said when admitting opponent has made a good point",
"ISBN": "International Standard Book Number",
"NPW": "No Problem Whatsoever",
"CHICKEN": "Kilogram of cocaine",
"NPZ": "No Problems",
"RETRO": "Old fashioned style",
"DTTC": "Don't Text The Cell",
"CHEEZ": "Cheese",
"ICBSST": "I Can't Believe She Said That",
"NQR": "Not Quite Right",
"DTRT": "Do The Right Thing",
"AMPED": "Excited| fired up",
"NRA": "National Rifle Association",
"NRE": "New Relationship Energy",
"TOEIC": "Test Of English for International Communication",
"NRG": "Energy",
"NRI": "Non-Resident Indian| Non-Repairable Item",
"JEET": "Did you eat?",
"NRL": "National Rugby League",
"NRN": "No Reply Necessary",
"N00B": "Newbie",
"JEEZ": "annoyance| Jesus",
"ISDK": "I Simply Don't Know",
"TBBH": "To Be Brutally Honest",
"ISDN": "Integrated Services Digital Network",
"ORKUT": "Google's social network",
"LEGGO": "Let's Go",
"NSA": "No Strings Attached",
"NSD": "Never Say Die",
"NSF": "Not Sufficient Funds| Not So Fast",
"LYAAF": "Love You As A Friend| Love You Always And Forever",
"IBAN": "International Bank Account Number",
"NSN": "Not So Nice",
"DTTM": "Don't Talk To Me",
"NSM": "Not So Much",
"FIYA": "Fire| The Greatest",
"PS & QS": "Pints and Quarts| Please and Thank Yous",
"NSW": "New South Wales",
"TSBT": "Truth Should Be Told| They Suck Big Time",
"STOKED": "Very happy| excited",
"OTTH": "On The Third Hand",
"NTB": "Not That Bothered| Not Too Bad",
"BNOC": "Big Name On Campus",
"NTD": "Nothing To Do",
"YHTBT": "You Had To Be There",
"NTH": "Nothing",
"NTK": "Need To Know",
"NTM": "Not Too Much| Nothing Much",
"CEREAL": "Serious",
"NTL": "NeverTheLess",
"NTN": "No Thanks Needed",
"NTS": "Nothing To Say| Note To Self",
"NTR": "Nothing To Report",
"NTT": "Name That Tune| Not Touching That!",
"NTW": "Not To Worry",
"NTY": "No Thank You",
"HARD LINES": "Bad luck",
"AKML": "Avada Kedavra My Life (Harry Potter)",
"NUB": "Someone who should know better",
"SPCA": "Society for the Prevention of Cruelty to Animals",
"NUF": "Enough| Nothing",
"NUH": "No",
"NUG": "High quality bud of marijuana",
"SPAG": "Spelling, Punctuation And Grammar",
"NUN": "Nothing",
"NUM": "Tasty",
"TBBT": "The Big Bang Theory (TV Show)",
"UEFA": "Union of European Football Associations",
"NUS": "National Union of Students",
"NUU": "No",
"SPAM": "Unsolicted email",
"SPAZ": "Spastic",
"SHERLOCKIAN": "A Fan Of Sherlock Holmes",
"WHIPPED": "Controlled by girlfriend/wife",
"TBCH": "To Be Completely Honest",
"SPAT": "Fight| argument| tiff",
"DILLIGAD": "Do I Look Like I Give A Darn?",
"NVM": "Nevermind",
"NVN": "Not Very Nice",
"BOOZE": "Alcoholic beverages",
"DTYT": "Don't Take Your Time",
"GMBH": "Gesellschaft mit Beschr",
"ISHK": "I Should Have Known",
"MPREG": "Male Pregnancy",
"JEJE": "Hispanic LOL",
"FLOSS": "Show off",
"SPED": "Someone who needs special education",
"TBFU": "Too Bad For You",
"TTTTY": "Time To Talk To You",
"GOOGLE": "An Internet Search Engine",
"NWJ": "No Way Jose",
"WKND": "Weekend",
"NWM": "No Worries Mate",
"GMAB": "Give Me A Break",
"NWO": "New World Order",
"NWT": "New With Tags",
"NWS": "Not Work Safe",
"PRESSIE": "Present| gift",
"TOEFL": "Test Of English as a Foreign Language",
"BINGO": "Here it is| Voila",
"HIT A LICK": "Get a lot of money very quickly",
"IBID": "Ibidem| In the same place",
"LEGIT": "Real| legal",
"NYB": "Not Your Business",
"NYD": "New Year's Day",
"NYC": "New York City",
"BURNED": "Insulted",
"NYF": "Not Your Fault",
"NYE": "New Years Eve",
"REDONCULOUS": "Ridiculous",
"MULLET": "Hairstyle",
"NYK": "Not Yet Known| Now You Know",
"NYP": "Not Your Problem",
"NYERK": "Replacement swearword",
"NYT": "Not Your Type| New York Times",
"NYU": "New York University",
"TSIA": "Title Says It All",
"BUTTER FACE": "Girl who is good looking except for her face",
"AKTF": "Always Keep the Faith",
"PERVY": "Perverted| creepy",
"CHEAP SKATE": "Mean| stingy person",
"DITZY": "Acting dumb and innocent",
"SPIC": "Offensive term for a Hispanic person",
"YUPPERZ": "Yes",
"ISLY": "I Still Love You",
"ROTFL": "Rolling On The Floor Laughing",
"TWONK": "Idiot| fool",
"BNWT": "Brand New With Tags",
"ILYAAF": "I Love You As A Friend",
"COMMANDO": "Without underwear",
"DEUCES": "Peace",
"WICKED": "Very| extremely| Cool| great",
"ISCSI": "Internet SCSI",
"ROZZERS": "Police",
"NKOTB": "New Kid On The Block",
"YARLY": "Yeah Really",
"SPIM": "Instant Messaging SPAM",
"KYSO": "Knock Yourself Out",
"ROTGL": "Rolling On The Ground Laughing",
"^~^": "Mischievous Smile",
"DUBS": "Twenty inch wheels",
"MOOLAH": "Money",
"ITWYM": "Is That What You Mean?",
"KHUF": "Know How You Feel",
"PGDA": "Pretty Goddamn Awesome",
"TBNT": "Thanks, But No Thanks",
"JERK": "Idiot",
"IIFYM": "If It Fits Your Macros",
"BLUEBERRY": "Form of marijuana",
"TTFAF": "Through The Fire And Flames",
"TBMS": "To Be More Specific",
"OAO": "Over And Out",
"OUAT": "Once Upon A Time",
"OAN": "On Another Note",
"OAP": "Old Age Pensioner",
"DUDS": "Casual clothes",
"TBPH": "To Be Perfectly Honest",
"WKYP": "Will Keep You Posted",
"YGTBK": "You've Got To Be Kidding",
"DITTO": "The same| me too| I agree",
"CRAB": "Derogatory word for a CRIP",
"SFTBC": "Sorry For The Broadcast",
"OBE": "Overcome By Events| Out of Body Experience| Order of the British Empire",
"DUFUS": "DOOFUS",
"SPOC": "Single Point of Contact",
"OBO": "Or Best Offer",
"OBQ": "Oh, Be Quiet",
"OBS": "Obviously",
"OBV": "Obviously",
"4EVA": "Forever",
"OBX": "Outer Banks North Carolina",
"TELNET": "Network Virtual Terminal Protocol",
"DUCY": "Do you see why?",
"OUCH": "Interjection said when experiencing pain",
"OCD": "Obsessive Compulsive Disorder",
"BALLSY": "Courageous| manly",
"DUDE": "A name for anyone",
"OCH": "Oh",
"CRAY": "Crazy",
"OCO": "Oh, Come On",
"OCN": "Of Course Not",
"JEST": "Joke",
"OCR": "Optical Character Recognition",
"ISTM": "It Seems To Me",
"OCT": "On Company Time",
"SWEETY": "SWEETIE",
"KHYF": "Know How You Feel",
"WIUWY": "What Is Up With You?",
"XOXOXO": "Hugs and Kisses",
"ISTR": "I Seem To Recall",
"TSTB": "The Sooner The Better",
"BFFWB": "Best Friend Forever With Benefits",
"ODF": "Own Damn Fault",
"TSTL": "Too Stupid To Live",
"YAAR": "Dude| mate",
"NABD": "Not A Big Deal",
"ODEE": "OD",
"ODG": "Oh Dear God",
"ODL": "Oh Dear Lord",
"SPQR": "Senatus Populusque Romanus| the Senate and the people of Rome",
"GMOD": "Garry's Mod",
"CRBT": "Crying Real Big Tears",
"ODT": "On Dis Ting",
"CAAC": "Cool As A Cucumber",
"TBQH": "To Be Quite Honest",
"SPOT": "Lend money",
"TBQF": "To Be Quite Frank",
"ROCKS": "Crack cocaine",
"ISSN": "International Standard Serial Number",
"SPRUNG": "Obsessed with someone| in love",
"NAMBLA": "North American Man/Boy Love Association",
"P&C": "Point and Click",
"OED": "Oxford English Dictionary",
"OEM": "Original Equipment Manufacurer",
"P&L": "Profit And Loss| Peace And Love",
"P&P": "Party And Play| Postage And Packaging",
"TT4N": "Ta Ta For Now",
"ISTG": "I Swear To God",
"YACC": "Yet Another Compiler-Compiler",
"AREA 51": "Top secret military base in Nevada",
"MOBA": "Multiplayer Online Battle Arena",
"TBTH": "To Be Totally Honest",
"WTDTA": "Where They Do That At?",
"TBTG": "Thanks Be To God",
"TBTF": "Too Big To Fail",
"MOBO": "Motherboard",
"\\//": "Peace| Vulcan salute",
"OFC": "Of Course",
"TBTP": "That's Beside The Point",
"OFH": "Old Folks' Home",
"NADA": "Nothing",
"SPST": "Same Place, Same Time",
"BOAT": "Bust Out Another Thousand",
"JWTLYK": "Just Wanted To Let You Know",
"MOAB": "Mother of All Bombs| Massive Ordnance Air Blast",
"TBSS": "Too Bad, So Sad",
"MOAM": "Man On A Mission",
"IBTD": "In Before The Delete| I Beg To Differ",
"MOAS": "Mother Of All Secrets",
"OGK": "Only God Knows",
"IBTL": "In Before the Lock",
"CHINWAG": "Chat",
"OGM": "misspelling of OMG",
"MOAR": "More",
"OGN": "Online Gaming Network",
"CREASING": "Laughing",
"CREY": "Cry",
"TSUP": "SUP",
"}": "-):Devilish smile",
"ALAS": "regret| sorrow",
"ALAP": "As Long As Possible",
"LLAP": "Live Long and Prosper",
"SPUN": "Under the influence of methamphetamine",
"OHK": "Oh,Ok",
"OHN": "Oh Hell, No!",
"SONAR": "SOund Navagation And Ranging",
"NAFF": "Poor quality| uncool| Substitute swear word",
"OHT": "One Handed Typing",
"OHS": "Operator Head Space",
"OHW": "One Hit Wonder",
"WHITEWASH": "Act like a white person",
"NRFB": "Never Removed From Box",
"AIRHEAD": "Stupid person",
"SHILL": "Decoy| person in on a scam",
"OIC": "Oh, I See",
"YADA": "And so on, blah",
"OIK": "Unpleasant| unpopular person",
"BOED": "Book of Exalted Deeds (MMORPG)",
"UKWIM": "You Know What I Mean",
"IMSMR": "If Memory Serves Me Right",
"OIY": "Hey",
"SHIOK": "happiness| pleasure",
"CRIT": "Critical hit (gaming)",
"OJD": "Obsessive Jonas Disorder",
"ALCO": "Alcoholic",
"CRIP": "Community Restoration in Progress| Gang member",
"YAGB": "You Ain't Grizz Baby (COD)",
"EXPO": "Exposition| large-scale public exhibition",
"IBYP": "I Beg Your Pardon",
"LOLING": "The act of laughing out loud",
"CRIB": "House| residence",
"OJT": "On Job Training",
"WHALE": "Punch| beat",
"OKC": "Oklahoma City",
"IDHTT": "I Don't Have The Time",
"BOGO": "Buy One Get One",
"VACAY": "Vacation",
"WLAN": "Wireless LAN",
"OKI": "Okey Dokey| OK",
"DULM": "Do You Love Me?",
"GMTA": "Great Minds Think Alike",
"EPIC FAIL": "Total failure",
"WIDOW": "WW",
"NAGL": "Not A Good Look",
"YJLTG": "You Just Lost The Game",
"BOHO": "Bohemian",
"UKWUR": "You Know Who You Are",
"JWAS": "Just Wait And See",
"VIBE": "Atmosphere| ambience",
"OMA": "Oh My Allah",
"ASAYC": "As Soon As You Can",
"OMB": "On My Break",
"CAKE": "Kilo of cocaine",
"OMD": "Oh My Days",
"OMG": "Oh My God",
"OMJ": "Oh My Jonas",
"OML": "Oh My Lord",
"TBYS": "Think Before You Speak",
"IDEWTK": "I Don't Even Want To Know",
"OMS": "Oh My Science",
"OMT": "One More Thing",
"OMW": "On My Way",
"ON9": "Stupid| Online",
"OMZ": "Oh My Zeus",
"MOJO": "Sex appeal| Charm| spell",
"ONE": "One Love| goodbye",
"HOOK ME UP": "Share with me",
"ONL": "Online",
".-.": "Sad| unhappy",
"ONM": "Oh, Never Mind",
"ONO": "Or Nearest Offer",
"SHELL-LIKE": "Ear",
"HOOAH": "HUA",
"YALA": "You Always Live Again",
"IIGHT": "Alright| OK",
"ONS": "One Night Stand",
"P/U": "Pick Up",
"FELLA": "Man| boyfriend",
"P/X": "Part exchange",
"STANDARD": "Of course| goes without saying",
"YOKED": "Well built| muscular| ripped",
"OOA": "Out Of Ammo",
"DUPA": "Ass",
"OOC": "Out Of Control/Character/Context",
"HEADS": "People",
"DUPE": "Duplicate",
"OOF": "Out Of Office",
"RSPCA": "Royal Society for the Prevention of Cruelty to Animals",
"OOI": "Out Of Interest",
"OOH": "Out Of Here",
"SWEET": "Awesome",
"CAMB": "Come At Me Bro",
"OOM": "Out Of Mana",
"...": "Silence| A Trailing Thought",
"OOO": "Out Of Office",
"P12": "Childish person",
"OOP": "Out Of Place",
"YUMMY MUMMY": "Attractive mother",
"OOS": "Out Of Style",
"OOT": "Out Of Town",
"AYBABTU": "All Your Base Are Belong To Us",
"BOKE": "Gag| almost vomit",
"DDOS": "Distributed Denial of Service (network attack)",
"SMHID": "Shaking My Head In Despair",
"OPB": "Other People's Business",
"OPA": "Greek exclamation",
"NANA": "Grandmother",
"YAMP": "Young tramp",
"MOLL": "Gangster's GF",
"CROW": "Marijuana",
"NANG": "Cool| wicked| good",
">O<": "Yelling",
"CLASS": "Cool| excellent",
"OPK": "Other People's Kids",
"OPM": "Other People's Money",
"BOLO": "Be On The Lookout",
"DUSS": "Run away",
"DUST": "Angel dust| PCP",
"YANA": "You Are Not Alone",
"OPS": "Operations| Oops",
"PINOY": "Filipino man",
"ALIE": "Init",
"CAMO": "Camoflauge",
"NASDAQ": "National Association of Securities Dealers Automated Quotation",
"YALL": "You All",
"SKIDADDLE": "Go away",
"YRMV": "Your Results May Vary",
"P2P": "Peer To Peer| Pay To Play",
"ITAI": "I'll Think About It",
"BOMM": "BOOMM",
"DURS": "Damn, You Are Sexy",
"P2W": "Pay To Win",
"BFFLE": "Best Friends For Like Ever",
"BOMB": "Something really bad| Something really good",
"KIFF": "Cool",
"CAPO": "Captain| Mafia boss",
"ICBI": "I Can't Believe It",
"ORG": "Online Reality Games| Organization",
"ICBM": "Intercontinental Ballistic Missile",
"ORN": "Oh Really Now",
"WAGWAN": "What's Going On?",
"ICBW": "It Could Be Worse",
"ORO": "confusion",
"P45": "Employment severance form",
"ODST": "Orbital Drop Shock Trooper",
"EPROM": "Electronically Programmable ROM",
"ORZ": "Frustration| despair",
"BONG": "Water pipe for smoking dope",
"MOOK": "Idiot| JERK",
"MOMO": "Annoying person| idiot",
"YANO": "You Know",
"ICAM": "I Couldn't Agree More",
"OSM": "Awesome",
"OSO": "Other Significant Other",
"YANK": "American",
"BOOT": "Vomit| Inexperienced person",
"P4P": "Pound For Pound",
".22": "A handgun",
"OSS": "Open Source Software",
"SWEAT": "Like, love",
"OT3": "One True Threesome",
"ICAY": "I Care About You",
"VA JAY JAY": "Vagina",
"BRILL": "Brilliant| cool",
"OST": "Original Sound Track",
"BOON": "Someone who has been a NOOB for ages",
"BOOM": "Great| amazing",
"BOOK": "Cool (predictive text )",
"CARTY": "Party in a car",
"ICBB": "I Can't Be Bothered",
"OTB": "Off The boat",
"OTA": "Over The Air",
"CAPS": "Bullets|Mushrooms",
"OTD": "On The Dot| Out The Door",
"MONG": "Complete idiot",
"OTC": "Over The Counter",
"PHISHING": "Scamming method used to get personal information",
"OTF": "Off Topic Forum",
"OTE": "Over The Edge",
"OTH": "Off The Hook",
"NARF": "Random interjection",
"OTG": "On The Go| Off The Ground",
"MOPO": "Master Of the Painfully Obvious",
"OTL": "like ORZ stick figure on hands and knees| Out To Lunch",
"OTK": "Over The Knee",
"NARK": "Informer",
"OTN": "On That Note",
"ALIHAL": "At Least I Have A Life",
"OTM": "Of The Moment| One Track Mind",
"OTP": "One True Pairing",
"OTO": "One Time Only",
"OTR": "On The Run",
"CARE": "I don't care",
"OTT": "Over The Top",
"CARD": "Amusing| eccentric person",
"OTS": "On The Side",
"SPANNER": "Idiot| fool",
"NARU": "Not A Registered User",
"OTW": "On The Whole",
"YARD": "Home",
"OTZ": "ORZ",
"NARP": "Non-Athletic Regular Person",
"OTY": "Over To You",
"IKMTY": "I Know More Than You",
"KIKE": "Offensive term for a Jew",
"FO SHO": "For Sure",
"?4U": "Question For You",
"ICCL": "I Couldn't Care Less",
"TCBY": "The Country's Best Yogurt",
"OUO": "Official Use Only",
"CASJ": "Casual",
"YEZZER": "Yes Sir",
".44": ".44 Magnum",
"WTHIGO": "What The Hell Is Going On?",
"OVA": "Original Video Animation",
"SYMBIAN": "Mobile phone OS",
"NOOB TUBE": "Grenade launcher attachment in COD4",
"TTGG": "Time To Go Girl",
"NATM": "Not At The Moment",
"OVO": "Obviously",
"NATO": "North Atlantic Treaty Organization| No Action Talk Only",
"TCFC": "Too Close For Comfort",
"NATI": "Cincinnati",
"TCFN": "Take Care For Now",
"MOSH": "Push/shove at rock/punk concert",
"ALOL": "Actually Laughing Out Loud",
"YATK": "You Are Too Kind",
"TCFS": "Too Cool For School",
"YATB": "You Are The Best",
"TTFL": "Ta Ta For Later",
"OWK": "Other World Kingdom",
"NASA": "National Aeronautics and Space Administration",
"CARPE": "Vomit| throw up",
"OWN": "Beat| dominate",
"NASL": "Name, Age, Sex, Location",
"OWS": "Occupy Wall Street",
"OWT": "Anything",
"TTFN": "Ta-Ta For Now",
"SNOOKIE": "Pet name for BF or GF",
"BOSS": "Cool| awesome",
"MORF": "Male or Female",
"TTFW": "Too Tacky For Words",
"LLOL": "LOL",
"MYODB": "Mind Your Own Damn Business",
"BOTW": "Blog Of The Week",
"OXY": "OxyContin| painkiller",
"CAVE": "Give up| give in",
"BOTS": "Robots| automated processes",
"BOTN": "Back Of The Net",
"HIMYM": "How I Met Your Mother (TV show)",
"OYB": "On Your Bike",
"OYE": "Listen up",
"WING MAN": "Person who looks out for you",
"OYG": "Oh Your God",
"WANNA": "Want to",
"OYO": "On Your Own",
"MOSS": "Relax| chill",
"MOTA": "Marijuana| weed",
"BURNT": "Out of it| having smoked too much pot",
"TCGB": "Take Care, God Bless",
"BOUT": "About",
"MOTB": "Mother Of The Bride",
"MOTD": "Message Of The Day| Match Of The Day",
"MOTO": "Master Of The Obvious",
"BOUL": "Bull",
"KINO": "Physical flirting, touching",
"GNFB": "Good Night Facebook",
"WIGGER": "White person pretending to be black",
"PCMIA": "Personal Computer Manufacturer Interface Adaptor",
"SWIZZLE": "Sweet",
"GJWHF": "Girls Just Wanna Have Fun",
"ITMA": "It's That Man Again",
"YAVW": "You Are Very Welcome",
"SUPER": "Very, really",
"BFFAE": "Best Friends Forever And Ever| Best Friends For All Eternity",
"NAWF": "North",
"BRICK": "1 kilogram of a drug| Very cold",
"POPPERS": "Amyl nitrate (recreational drug)",
"NAWL": "No",
"NAWM": "Know What I Mean?",
"HMIHY": "How May I Help You?",
"MASH UP": "Song made from bits of other songs",
"GROUSE": "Great| amazing",
"NAWT": "Not",
"BOWS": "Elbows",
"YAWN": "Boring",
"BURMA": "Be Upstairs Ready My Angel",
"YAWA": "Person who always uses the 'Away' status in IM systems",
"BOWL": "Part of a marijuana pipe",
"TRICKIN": "Spending money on a girl",
"MOXY": "Courage| confidence",
"XOXO": "Hugs & Kisses",
"PROZZIE": "Prostitute",
"PHAT": "Pretty Hot And Tempting",
"REEFA": "REEFER",
"KISA": "Knight In Shining Armour",
"DOWNS": "Downs-Syndrome",
"DROVE": "Mad| angry",
"KISS": "Keep It Simple, Stupid",
"ICMP": "Internet Control Message Protocol",
"PIGEON": "Woman using a man for personal gain",
"WLTM": "Would Like To Meet",
"TTLY": "Totally",
"ILYSDM": "I Love You So Damn Much",
"TOFTT": "Take One For The Team",
"KIRF": "Keeping It Real Fake",
"SIM CARD": "Subscriber Identity Module for a mobile phone",
"YAYO": "Cocaine",
"TTML": "Talk To Me Later",
"BOYO": "Friend| mate",
"ROSES": "Money",
"ALWP": "Actual Laughter Was Produced",
"BOZO": "Incompetent person| idiot",
"O_O": "Confused",
"BAIRN": "Child| baby",
"LAMER": "Annoying person",
"KITE": "Mail received in jail",
"DEAR": "Drop Everything And Read",
"LAMEO": "Lame Person",
"BUCKS": "Dollars| money",
"WTTIR": "When The Time Is Right",
"PAD": "House| room",
"PAC": "Gangsta rapper Tupac Shakur",
"PAF": "Phone A Friend",
"ITOY": "I'm Thinking Of You",
"KAWAII": "Cute| pretty",
">_<": "Frustrated",
"PAN": "Personal Area Network",
">_>": "Looking left| shifty look",
"MMOFPS": "Massively Multiplayer Online First Person Shooter",
"TOPPER": "Person who always has a better story",
"PAP": "Post A Picture",
"DVDR": "DVD Recordable",
"PAR": "Disrespect",
"MESOS": "Currency in the game Maple Story",
"PAU": "Finished| done",
"PAX": "Peace",
"PAW": "Parents are watching",
"ASCII": "American Standard Code for Information Interchange",
"YMCMB": "Young Money Cash Money Billionaires",
"PHEW": "relief",
"WLYK": "Will Let You Know",
"DEBO": "Steal",
"PBB": "Pirate Bulletin Board",
"KIWI": "A person from New Zealand| A small brown bird from New Zealand| Kiwifruit",
"JFTR": "Just For The Record",
"PBJ": "Peanut Butter and Jelly",
"-_-": "Annoyed| tired",
"PBN": "Paintball Nation",
"PBQ": "Please Be Quiet",
"NSAW": "Not Saying A Word",
"PBP": "Please Be Patient",
"PBS": "Public Broadcasting Service",
"PBR": "Pabst Blue Ribbon (beer)",
"TCOB": "Taking Care of Buisness",
"JWTK": "Just Wanted To Know",
"LOWRIDER": "Vehicle with modified suspension often hydraulically controlled",
"SIKED": "Psyched, excited about",
"CACHE": "Store for temporary computer files",
"PCB": "Printed Circuit Board| Please Come Back",
"BITTIE": "Girl",
"PCD": "Pussycat Dolls",
"PCE": "Peace| Goodbye",
"DECK": "Cool| Punch hard",
"WLYB": "Who Loves You Baby",
"PCI": "Peripheral Component Interconnect",
"PCM": "Please Call Me",
"RIFRAF": "Common people| trailer trash",
"PCP": "Phencyclidine| Angel dust",
".EDU": "education",
"TCOY": "Take Care Of Yourself",
"WYATB": "Wish You All The Best",
"CRAY CRAY": "Really crazy",
"DEEC": "Decent",
"PDA": "Personal Digital Assistant",
"ICSL": "I Couldn't/Can't Stop Laughing",
"PDG": "Pretty Damn Good",
"PDF": "Portable Document Format",
"TTTH": "Talk To The Hand",
"NAPSTER": "File sharing site",
"TTTE": "Thomas The Tank Engine",
"ROCKBAND": "Music video game",
"PDQ": "Pretty Damn Quick",
"PDP": "Pretty Darn Precious",
"HOP OFF": "Leave me alone",
"ITSK": "It's OK",
"PDT": "Pacific Daylight Time",
"ITSO": "In The Style Of",
"ATEOTD": "At The End Of The Day",
"SYITM": "See You In The Morning",
"DEEZ": "These",
"SWTOR": "Star Wars: The Old Republic (game)",
"SWEETIE": "Pet name for BF or GF",
"Q&A": "Question and Answer",
"FLAME": "Insult someone over the internet",
"BLIND": "In love",
"PEN": "Penitentiary| jail| prison",
"BLINK": "Blink 182 (band)",
"ZECH": "Energy| life",
"WAMHAS": "With All My Heart And Soul",
"BLING": "Flashy Jewelry",
"COOLIN": "CHILLing",
"CBBC": "Children's BBC",
"FANTABULOUS": "Fantastic and fabulous",
"GAL PAL": "Man who hangs around with girls",
"CWTCU": "Can't Wait To See You",
"PFA": "Please Find Attached",
"PFC": "Private First Class",
"PFB": "Please Find Below",
"DEFS": "Definitely",
"PFF": "disagreement",
"FLAKY": "Unreliable",
"DEFO": "Definitely",
"N1C3": "Nice",
".CO": "Alternative to .COM",
"PFP": "Picture For Proof",
"CWTCH": "Hug or cuddle",
"DEFF": "Definitely",
"PFT": "An exlamation of disbelief",
"MPAA": "Motion Picture Assosciation of America",
"TTTT": "These Things Take Time",
"FLAKE": "Unreliable person",
"PGA": "Professional Golfers' Association",
"PUNTER": "Customer",
"TTUL": "Talk To You Later",
"DEGT": "Don't Even Go There| I don't want to talk about it",
"INDIE": "Independent music",
"ILYSMM": "I Love You So Much More",
"PGM": "Pro Gamer",
"GOTCHA": "I got you| I understand",
"ILYSMB": "I Love You So Much Baby",
"PGP": "Pretty Good Privacy",
"TTUT": "Talk To You Tomorrow",
"TCTC": "To Close To Call",
"TTUS": "Talk To You Soon.",
"AMAP": "As Much As Possible",
"YELLO": "Hello",
"LMAD": "Let's Make A Deal",
"PHB": "Pointy-Haired Boss",
"IMAHO": "In My Absolutely Honest Opinion",
"LMAO": "Laughing my ass off",
"CRUSH": "Infatuation| attraction",
"PHD": "Doctor of Philosphy",
"LMAM": "Leave Me A Message",
"BUTCH": "Masculine",
"TWEETER": "Person who posts on Twitter",
"AMAA": "Ask Me Almost Anything",
"LMAP": "Leave Me Alone Please",
"T2UL": "Talk To You Later",
"GNSD": "Good Night, Sweet Dreams",
"PHP": "Personal Home Page",
"ITYF": "I Think You'll Find",
"ITYL": "I'll Tell You Later",
"PHU": "Potential Hook Up",
"ITYM": "I Think You Mean",
"NSFA": "Not Safe For Anyone",
"CBGB": "Country BlueGrass Blues",
"PID": "Stupid",
"PIC": "Picture| Partner In Crime",
"PIF": "Marijuana",
"JFYI": "Just For Your Information",
"NSFL": "Not Safe For Life",
"PIG": "Police officer",
"NSFV": "Not Safe for Viewing/Vegetarians/Vegans",
"SQTM": "Smiling Quietly To Myself",
"NSFW": "Not Safe For Work",
"PIN": "Personal Identification Number",
"PIR": "Parent In Room",
"WAGWUN": "What's Going On?",
"PIQ": "Person In Question",
"PIU": "Pump It Up",
"PIX": "Pictures",
"COUCH POTATO": "Lazy person who sits all day watching TV",
"DEKE": "Avoid| fake| decoy",
"LMCO": "Laughing My Cats Off",
"WUWTB": "What You Wanna Talk 'Bout?",
"WUWTA": "What You Wanna Talk About?",
"PJS": "Pajamas",
"SMOOCH": "Kiss",
"MPEG": "Moving Picture Experts Group| Audio/visual file format",
"PKB": "Pot, Kettle, Black",
"ITYS": "I Told You So",
"TTYF": "Talk To You Forever",
"PKI": "Public Key Infrastructure",
"NAACP": "National Association for the Advancement of Colored People",
"TTYN": "Talk To You Never",
"TTYM": "Talk to You Ma",
"TTYL": "Talk To You Later",
"PKS": "Painkillers",
"TTYT": "Talk To You Tomorrow",
"TTYS": "Talk To You Soon",
"AMEX": "American Express",
"FIVE O": "Police",
"PLA": "People's Liberation Army",
"FLAFF": "Foot Loose And Fancy Free",
"PLC": "Public Limited Company",
"PLD": "Poor Life Decision",
"TWISTED": "High and drunk",
"PLI": "Potential Love Interest",
"OVNO": "Or Very Near Offer",
"PLH": "Peace, Love and Happiness",
"PLL": "Pretty Little Liar",
"CRUNK": "Crazy and drunk",
"PLP": "People Lazily Procrastinating",
"PLS": "Please",
"PLR": "Private Label Rights",
"T2YL": "Talk To You Later",
"PLU": "People Like Us",
"PLX": "Please",
"JOHN DOE": "General term for a man",
"PLZ": "Please",
"PMB": "Private Message Box",
"PMA": "Positive Mental Attitude",
"PMD": "Put Me Down",
"PMC": "Post Meal Cigarette",
"PMF": "Pardon My French",
"NBIF": "No Basis In Fact",
"PMT": "Pre-Menstrual Tension",
"Q/A": "Question and Answer",
"PNC": "Point and Click| Partner in Crime",
"E&OE": "Errors and omissions excluded",
"PNG": "Image file format",
"MUAHZ": "Kisses",
"PNL": "Peace And Love",
"@.@": "Confused| surprised",
"PNP": "Party and Play",
"BOOMTING": "Good looking girl",
"PNR": "Passenger Name Record",
"PNW": "Pacific North West",
"ILYWAMH": "I Love You With All My Heart",
"BLICK": "A very black person",
"POA": "Plan Of Action",
"POD": "Passed Out Drunk",
"POF": "Plenty of Fish",
"POI": "Point(s) Of Interest| Person of Interest",
"PON": "On| PWN",
"POM": "Australian name for an English person",
"POP": "Popular (music)",
"POT": "Mariujana",
"MUAHA": "An evil laugh",
"POV": "Point Of View",
"POW": "Prisioner of War",
"STEP OFF": "Leave",
"POZ": "HIV positive",
"PPC": "Pay Per Click",
"PPB": "Parts Per Billion",
"PPD": "Post Potter Depression",
"ADGTH": "All Dogs Go To Heaven",
"PPM": "Parts Per Million",
"SHEERIO": "An Ed Sheeran Fan",
"PPL": "People",
"PPS": "Post-Post Script",
"PPU": "Please Pick Up",
"PPT": "Microsoft PowerPoint",
"PINCHED": "Caught| arrested",
"PPV": "Pay Per View",
"DEPT": "Department",
"DERP": "Reply to stupid comment or action",
"DERO": "Derelict| loser",
"OWLING": "Crouching and staring like an owl",
"GHOST": "Leaving",
"DERE": "There",
"LMHO": "Laughing My Head Off",
"RESTECPA": "Respect",
"IDBI": "I Don't Believe It",
"PRC": "People's Republic of China",
"PRI": "Pretty",
"DESI": "Someone from the Indian subcontinent",
"HUNNED": "Hundred dollars",
"ZERG": "Defeat enemy by outnumbering rather than strategy or skill",
"PS2": "PlayStation 2",
"IDBY": "I Don't Believe You",
"PS4": "PlayStation 4",
"PS3": "PlayStation 3",
"PRW": "Parents Are Watching",
"PRY": "Probably",
"4GET": "Forget",
"DERV": "Diesel Engine Road Vehicle",
"BLOTTO": "Drunk",
"DERR": "DUH",
"PSB": "Please See Below",
"MPOG": "Multiplayer Online Game",
"PSA": "Publice Service Announcement",
"PSD": "Photoshop Data file",
"PSH": "Dismissive expression",
"PSI": "Pounds per Square Inch (pressure)|Greek letter",
"PSN": "Playstation Network",
"IWOOT": "I Want One Of Those",
"PSM": "Personal Short Message",
"PSP": "PlayStation Portable",
"TTEOT": "Till The End of Time",
"PSR": "Pretty Standard, Really",
"IIWII": "It Is What It Is",
"PST": "Please Send Tell| Pacific Standard Time",
"PSU": "Power Supply Unit",
"Q4U": "Question For You",
"PSX": "Playstation X",
"BDRIP": "Blu-ray Disc RIP",
"BALLER": "Successful ball player",
"DESU": "It is",
"PTC": "Popping The Collar",
"PTB": "Powers That Be| Pass The Buck| Please Text Back",
"MIXOLOGY": "Study of mixing drinks",
"PTE": "Private",
"PTI": "Pardon The Interruption",
"SMGDH": "Shaking My God Damn Head",
"PTM": "Please Tell Me",
"PEACE": "Goodbye",
"PTL": "Praise The Lord",
"DIDDO": "DITTO",
"PTO": "Paid Time Off",
"PTP": "Pay To Play| Peer To Peer",
"PTT": "Push-To-Talk",
"PTW": "Play To Win| Professional Time Waster",
"PTZ": "Pan-Tilt-Zoom",
"SAUCE": "source",
"UGGS": "Brand of boots",
"IDEC": "I Don't Even Care",
"JORTS": "Jean shorts",
"PUB": "bar",
"PUA": "Pick Up Artist",
"UGGO": "Ugly person",
"NO BIGGIE": "No big deal",
"MPOS": "Multiple Positions",
"PUG": "Pick Up Group",
"DEVO": "Devastated",
"TUDE": "Bad attitude",
"NALOPKT": "Not A Lot Of People Know That.",
"PURPLE": "Strong marijuana",
"SHOOTER": "Gun| Person who fires a gun",
"PUV": "Public Utility Vehicle",
"PIMPL": "Pee In My Pants Laughing",
"PVC": "Polyvinyl Chloride| vinyl",
"PVE": "Player versus Environment",
"HACK": "Break into computer systems or software",
"NBTD": "Nothing Better To Do",
"PVM": "Player Versus Monster",
"ROAR": "Right Of Admission Reserved",
"PVP": "Player v Player",
"PVR": "Personal Video Recorder",
"PVT": "Private",
"SACK": "Bed",
"TFMTK": "That's For Me To Know",
"NBTW": "Now Back To Work",
"WHACK": "Bad",
"YBTJ": "You Be The Judge",
"MURSE": "Man purse| Male nurse",
"AMOG": "Alpha Male Of Group",
"PWD": "Password",
"AMOF": "As a Matter of Fact",
"IDGI": "I Don't Get It",
"IDEM": "The same, me too",
"IDEK": "I Don't Even Know",
"SOMSW": "Someone Over My Shoulder Watching",
"PWN": "Own| dominate",
"TUFF": "Rugged",
"NBSB": "No Boyfriend Since Birth",
"OKIES": "Okay",
"PWM": "Professional White Male",
"PWP": "Plot? What Plot?",
"PWT": "Poor White Trash",
"LMOA": "LMAO",
"REDNECK": "Unsophisticated rural person from Southeast USA",
"BALLIN": "Playing basketball| Living the good life",
"GOAT": "Greatest Of All Time",
"IDFG": "I Don't Feel Good",
"IDHAC": "I Don't Have A Clue",
"MPUA": "Master Pick Up Artist",
"FLAG": "Bandana",
"PC 4 PC": "Picture Comment For Picture Comment",
"MAPOTI": "Most Annoying People On The Internet",
"HAFL": "Heart Attack From Laughing",
"LMPO": "Laughing My Pants Off",
"JOCKER": "copies another's style",
"PYP": "Play Your Position",
"PYT": "Pretty Young Thing",
"YSVW": "You're So Very Welcome",
"QANTAS": "Queensland and Northern Territory Aerial Service",
"HAGO": "Have A Good One",
"HAGN": "Have A Good Night",
"HAGS": "Have A Great Summer",
"YSYL": "You Snooze, You Loose",
"HAGT": "Have A Great Time",
"SAGE": "Word added to a post to stop it getting bumped",
"HOMIES": "Group Of Close Friends",
"STEAMING": "Drunk| intoxicated",
"CHAMPION": "Good| great",
"IDKE": "I Don't Know Either",
"ROFC": "Rolling On Floor Crying",
"/": "):Flirtation |Sign Of Suspicion",
"SAFE": "Something or someone good",
"SAFM": "Stay Away From Me",
"FLBP": "Future Lower Back Problems",
"HAGE": "Have A Good Evening",
"YBWM": "You Belong With Me",
"HAGD": "Have A Good Day",
"WYBMV": "Will You Be My Valentine?",
"SLUGS": "Bullets",
"LMSO": "Laughing My Socks Off",
"HAGL": "Have A Good Life",
"IDLT": "I Don't Like That",
"IDLY": "I Don't Love/Like You",
"HAIZ": "Sigh",
"HONKY": "HONKIE",
"GOTTA": "Got to ...| Have you got a ...?",
"DWAG": "Misspelling of DAWG",
"FLEX": "Show off",
"PIBE": "Play It By Ear",
"DWAI": "Don't Worry About It",
"TUNE": "Flirt",
"TEOTWAWKI": "The End Of The World As We Know It",
"IDKW": "I Don't Know Why",
"ROFL": "Rolling On Floor Laughing",
"IDKY": "I Don't Know Why",
"SAHD": "Stay At Home Dad",
"SAHN": "Son",
"SAHM": "Stay At Home Mom",
"IDLE": "Inactive",
"IDLI": "I Don't Like It",
"SAHW": "Stay At Home Wife",
"TROLOL": "LOLING at somebody who is TROLLING",
"DINERO": "Money",
"LIKKLE": "Little",
"FELLER": "Fellow| man",
"HALP": "Help",
"YIKES": "shock| surprise",
"SMIZE": "Smile with your eyes",
"DWBH": "Don't Worry, Be Happy",
"DWBI": "Don't Worry 'Bout It",
"IDNK": "I Do Not Know",
"THANQ": "Thank You",
"CHICK": "Girl",
"?_?": "Confused| lost",
"@@@": "Warning of parents nearby",
"QAP": "Quick As Possible| Quickly As Possible",
"HAND": "Have A Nice Day",
"FLIR": "Forward Looking Infra Red",
"ROLF": "Rolling On Laughing Floor",
"PUMPED": "Exhilarated| fired up",
"TURF": "Area| part of the neighborhood",
"HANG": "Hang out| spend time with",
"TTJASI": "Take This Job And Shove It",
"KTBFFH": "Keep The Blue Flag Flying High!",
"HANL": "Have A Nice Life",
"WDYDT": "Why Did You Do That?",
"GRODY": "Disgusting| nasty| gross",
"BTAIM": "Be That As It May",
"ROLL": "Experience the effects of Ecstasy",
"GOMB": "Get Off My Back",
"PIFF": "Purple weed",
"._.": "Emoticon Representing Apathy",
"GOMD": "Guy/Girl Of My Dreams",
"NEWFAG": "Newcomer",
"SALA": "Idiot| moron",
"OFAY": "Offensive term for a white person",
"BOXORZ": "Boxers",
"TDOT": "Toronto",
"WMYB": "What Makes You Beautiful",
"FRESHIE": "Imigrant",
"PIGS": "The police",
"LAPPY": "Laptop computer",
"HAPA": "Mixed race",
"/B/": "4chan's random image board",
"IDST": "If Destroyed, Still True",
"GOMF": "Get Outta My Face",
"DQMOT": "Don't Quote Me On This",
"PJ'S": "Pyjamas| night wear",
"GOML": "Get On My Level",
"SANK": "Something",
"BSOFN": "Big Smile On Face Now",
"IDRC": "I Don't Really Care",
"IUSS": "If You Say So",
"R&B": "Rhythm And Blues",
"NCAA": "National Collegiate Athletic Association",
"QED": "Quod Erat Demonstrandum| it has been proved",
"R&D": "Research and Development| Replicate and Duplicate",
"SANS": "Without",
"IDRK": "I Don't Really Know",
"HAXOR": "Hacker",
"R&R": "Rest and Relaxation",
"WWOTW": "Wicked Witch Of The West",
"TDTF": "Too Drunk to Function",
"HARF": "Vomit| throw up",
"QFE": "Quoted For Emphasis",
"OSSIM": "Awesome",
"IYBTYBA": "If You Believe That, You'll Believe Anything",
"QFP": "Quoted For Posterity",
"NUNYA": "None of Your business",
"QFT": "Quoted For Truth",
"TUVM": "Thank You Very Much",
"A&E": "Accident & Emergency",
"GOOH": "Get Out Of Here",
"A&F": "Always And Forever| Abercrombie & Fitch",
"FAT32": "32 bit File Allocation Table",
"GOOG": "Ecstasy pill",
"GOOK": "Offensive term for a Korean",
"GOON": "Stupid| Tough guy",
"A&R": "Artists & Repertoire",
"WDYWFM": "What Do You Want From Me?",
"IDTS": "I Don't Think So",
"GORM": "Gormless person",
"SASE": "Self-Addressed, Stamped Envelope",
"IDWK": "I Don't Wanna Know",
"NOMNOM": "Sound of eating",
"COMMY": "Communist",
"IDWT": "I Don't Want To",
"SATC": "Sex And The City (TV show)",
"GOSH": "Exclamation of suprise or shock",
"SATA": "Serial Advanced Technology Attachment",
"TTYRS": "Talk To You Real Soon",
"CTHU": "Crack The Hell Up",
"OFGS": "Oh, For God's Sake",
"NOMMY": "Good| delicious",
"HASH": "Marijuana",
"NTFS": "New Technology File System",
"ROFLMFAO": "Rolling On Floor Laughing My F***ing A** Off",
"GORG": "Gorgeous",
"FLOG": "Sell",
"WAPCE": "Women Are Pure Concentrated Evil",
"RIPPED": "Well defined muscles| Stoned",
"LIGIT": "Legitimate",
"NADDA": "Nothing",
"ROTF": "Rolling On The Floor (laughing is implied)",
"ROTC": "Reserve Officer Training Corps",
"DEALING": "Acting like boyfriend/girlfriend| Selling drugs",
"GOTY": "Game Of The Year",
"ROTM": "Run Of The Mill",
"ROTK": "Return of the King (Tolkien)",
"PING": "Send a packet to a computer and wait for its return (Packet INternet Groper)",
"DWMT": "Don't Waste My Time",
"MO PO": "Mounted Police",
"EZPZ": "Easy peasy| very easy",
"DFLI": "Don't Feel Like It",
"CCIE": "Cisco Certified Internetwork Expert",
"GOSU": "Skillful person (Korean)",
"SATS": "Standard Attainment Tests",
"TDWP": "The Devil Wears Prada (Band)",
"BUDDY": "Friend",
"CADDY": "Cadillac",
"RAINCHECK": "Postponement",
"SAWP": "SUP",
"OWNM": "Oh Well, Never Mind",
"MIRIN": "Admiring",
"QLD": "Queensland",
"KNOCK IT OFF": "Stop it",
"WIWWY": "What Is Wrong With You?",
"DISSING": "Being disrespectful",
"IWALY": "I Will Always Love You",
"GRONK": "Idiot| fool",
"LIEING": "lying",
"PIPA": "Protect IP Act",
"WIWWU": "Wish I Was With You",
"DYHWIH": "Do You Hear What I Hear?",
"FELLAS": "Close friends",
"PIPS": "Easy| not difficult",
"GOWM": "Go Out With Me",
"OWOA": "One Way Or Another",
"D-DAY": "6th June 1944| Designated day when operations are due to commence",
"HAWK": "Knife",
"GROOL": "Great and Cool",
"IWALU": "I Will Always Love You",
"HAWT": "Hot| Having A Wonderful Time",
"NCIS": "Naval Criminal Investigative Service (TV show)",
"NUFFIN": "Nothing",
"HAZE": "Type of marijuana",
"QNA": "Question and Answer",
"NTMU": "Nice To Meet You",
"OFOC": "Overwhelming Feelings of Concern",
"QNO": "Quiet Night Out",
"YTML": "You'll Thank Me Later",
"NTMY": "Nice To Meet You",
"AIGHT": "All right",
"IMUSM": "I Miss You So Much",
"HAYD": "How Are You Doing?",
"P911": "Parent Alert",
"CREPS": "Trainers",
"QOL": "Quality Of Life",
"0.0": "Surprise",
"P90X": "Type of workout schedule",
"QOS": "Quantam Of Solace",
"HAYT": "How Are You Today?",
"TYFYT": "Thank You For Your Time",
"WUCIWUG": "What You See Is What You Get",
"FAUX PAS": "Social blunder",
"A/N": "Author's Note",
"FASHO": "For sure",
"A/W": "Anyway",
"CCNA": "Cisco Certified Network Associate",
"LMTAL": "Let Me Take A Look",
"AUSSIE": "A Person From Australia",
"0.O": "Raised eyebrow",
"PITR": "Parent In The Room",
"BLAZING": "Smoking weed",
"4YEO": "For Your Eyes Only",
"IBUWU": "I'm Breaking Up With You",
"YCMU": "You Crack Me Up",
"NCNR": "Non-Cancelable, Non-Returnable",
"NCNS": "No Call, No Show",
"IYSWIM": "If You See What I Mean",
"GROSS": "Unpleasant| disgusting",
"/O/": "Happy| arms in the air",
"NCMO": "Non-Committal Make Out",
"LNIB": "Like New In Box",
"GASSED": "Have a high opinion of yourself",
"ITIKY": "I Think I Know You",
"SHADES": "Sunglasses",
"RIDIC": "Ridiculous",
"DWTH": "Don't Work Too Hard",
"N2MU": "Nice To Meet You",
"QSL": "Confirmed",
"DFTD": "Done For The Day",
"CTRN": "Cant Talk Right now",
"DWTS": "Dancing With The Stars",
"CTRL": "Control (key)",
"KOOLS": "Mentholated cigarettes",
"HONKIE": "White person",
"SSBB": "Super Smash Brothers Brawl",
"ALPHA": "Head person of a group",
"/R/": "Requesting",
"QTR": "Quarter",
"QTS": "Cuties",
"TTYVS": "Talk To You Very Soon",
"WYLSCWTW": "Would You Like Some Cheese With That Whine?",
"QTY": "Quantity",
"DFTT": "Don't Feed The Troll",
"DFTS": "Don't Forget to Smile",
"IEEE": "Institute of Electrical and Electronics Engineers",
"ITILY": "I Think I Love You",
"QUE": "What?",
"ITILU": "I Think I Love You",
"HSBC": "Hong Kong and Shanghai Banking Corporation",
"SCHWAG": "Low grade marijuana",
"TECH": "Technician| Technology",
"NTSC": "National Television System Committee video standard",
"DOUGIE": "Swagger| Dance style",
"NTSH": "Nothing To See Here",
"QVC": "Quality Value Convenience",
"CHILL OUT": "CHILL",
"OLD BAG": "Grumpy old woman",
"SSDD": "Same Stuff Different Day",
"HBCU": "Historically Black Colleges and Universities",
"ANON": "Unknown person",
"INCYDK": "In Case You Didn't Know",
"TTYTT": "To Tell You The Truth",
"MTFBWY": "May The Force Be With You",
"KKKK": "Korean LOL",
"NTTU": "Not Talking To You",
"MTFBWU": "May The Force Be With You",
"A7A": "Frusration, anger",
"GANJA": "Marijuana| pot| cannabis| weed",
"TEEF": "Steal| Teeth",
"CCTV": "Closed-Circuit Television",
"TEEN": "Teenager| teenage",
"YTTM": "You Talk Too Much| You Talking To Me?",
"A7X": "Avenged Sevenfold (band)",
"WMMOWS": "Wash My Mouth Out With Soap!",
"RED BONE": "Light Skinned Colored Person| Red Undertones In Skin and Hair",
"TWAIN": "Technology Without An Interesting Name",
"PAPARAZZI": "Photo-journalists",
"ANFAWFOS": "And Now For A Word From Our Sponsor",
"CWTSY": "Can't Wait To See You",
"SLOSHED": "Drunk",
"TEHO": "To Each His/Her Own",
"OTTOMH": "Off The Top Of My Head",
"NTWF": "Neopian Times Writers Forum",
"CREEP": "Weird| undesirable person",
"NIMROD": "Stupid person| idiot",
"ENNIT": "agreement",
"FMCG": "Fast Moving Consumer Goods",
"HSIK": "How Should I Know?",
"BOBBA": "Replacement swear word",
"HSIT": "How Sad Is That",
"ANSI": "American National Standards Institute",
"SSID": "Service Set Identifier",
"ICYDK": "In Case You Didn't Know",
"KINDA": "Kind Of",
"IDKFA": "Cheat code for Doom",
"GENIOUS": "genius",
"GNARLY": "Extreme",
"ANTM": "America's Next Top Model",
"ANTI": "Against",
"WOTCHER": "What are you up to? WU?",
"SMOOSH": "Hug| squeeze",
"TARFU": "Things Are Really Fouled Up",
"IRCOP": "Internet Relay Chat Operator",
"PLIES": "American rapper",
"GASSING": "Talking",
"MOTKU": "Master Of The Known Universe",
"WNTT": "We Need To Talk",
"MHOTY": "My Hat's Off To You",
"DRINKY POO": "Alcoholic drink",
"WNWY": "What's New With You?",
"SOD OFF": "Go away| get lost",
"KINGS": "Drinking game",
"CHYEA": "Yes| yeah",
"WDYWT": "What Did You Wear Today?",
"HUBBY": "Husband",
"TEMP": "Temporary",
"RAD": "Radical| cool",
"RAC": "Rock Against Communism",
"RAH": "Pompous or superior person",
"ANY1": "Anyone",
"RAK": "Random Act of Kindness",
"RAM": "Random Access Memory",
"AMIRITE": "Am I Right?",
"RAP": "Music genre",
"@_@": "Hypnotized| dazed",
"RAR": "File compression type",
"RAT": "Snitch| telltale",
"RAS": "Remote Access Server",
"CUAD": "Curl Up And Die",
"DINGBAT": "Stupid person| idiot",
"SENIOR MOMENT": "Lapse of memory| logic or mental function",
"BISOU": "Kiss",
"RBG": "Revolutionary But Gangsta",
"AAA": "American Automobile Association| Battery size",
"AAB": "Average At Best",
"AAC": "Apple Audio Compression",
"BAHAHA": "Evil laugh",
"AAF": "Always and Forever",
"RBV": "Red Bull and Vodka",
"AAK": "Alive And Kicking",
"AAP": "Always A Pleasure",
"HOOPS": "Basketabll",
"AAR": "At Any Rate",
"AAT": "And Another Thing| At All Times",
"RCH": "A very small unit of measurement",
"SAVVY": "Do you understand?| Knowledgeable| well informed",
"TIGHT": "Close| Miserly",
"ABC": "American Born Chinese",
"RCT": "Roller Coster Tycoon",
"ABD": "Already Been Done",
"DOWNER": "Someone or something that brings the mood down",
"ABF": "All But Face",
"ABG": "Asian Baby Girl",
"ABH": "Actual Bodily Harm",
"GLITCH": "Error| mistake",
"ABO": "Aborigine",
"CONVO": "Conversation",
"ABP": "Already Been Posted",
"RDC": "Really Don't Care",
"ABS": "Absolutely",
"YASNY": "You Ain't Seen Nothing Yet",
"ABT": "About",
"ABU": "Anyone but Manchester United",
"RDJ": "Robert Downey Jr.",
"RDM": "Random Death Match",
"FMLA": "Family and Medical Leave of Absence",
"RDO": "Regular Day Off",
"ACC": "Actually",
"HOOCHIE": "Over made up woman with trashy dress sense",
"RDR": "Red Dead Redemption (Game)",
"ACE": "Excellent| great",
"RDV": "Rendez-Vous",
"RDY": "Ready",
"BRAH": "Bro| friend",
"ACK": "Disgust| frustration| Acknowledgement",
"H1N1": "Swine flu virus",
"POPPING": "Dance style",
"ACP": "Automatic Colt Pistol",
"NUBE": "NOOB",
"S&D": "Search and Destroy",
"REC": "Recreation",
"ACT": "SAT type test",
"REF": "Refugee",
"ACU": "Army Combat Uniform",
"REG": "Regular",
"REL": "Relative| Irrelevant",
"REM": "Rapid Eye Movement| Rock band",
"S&M": "Sadism and Masochism",
"REP": "Represent| Reputation| Repetition",
"S&P": "Salt and Pepper",
"REQ": "Request| Required",
"ADD": "Attention Deficit Disorder",
"RES": "Resin",
"REV": "Make an engine go faster| Reverend",
"BRBK": "BRB",
"REZ": "Indian reservation",
"CONTD": "Continued",
"ADL": "All Day Long",
"ADM": "Ay Dios Mio| OMG",
"ADN": "Any Day Now",
"RFC": "Request For Comment",
"ADR": "All Due Respect",
"ADS": "Aim Down Sights",
"RFI": "Request For Information",
"MAAB": "Male-Assigned At Birth",
"BRAT": "Annoying| spoilt child",
"RFQ": "Request For Quotation",
"RFP": "Request for Proposal",
"B&B": "Bed & Breakfast",
"B&D": "Bondage And Discipline| Bondage And Domination",
"B&E": "Breaking And Entering",
"RFT": "Right First Time",
"B&M": "Bricks and Mortar",
"RGB": "Red, Green, Blue",
"CUFF": "Overprotect| smother",
"YUCK": "disgust",
"BAAS": "Boss",
"HEAVY": "Awesome| good| Serious| intense",
"GPOY": "Gratuitous Picture Of Yourself",
"B&W": "Black and White",
"RGR": "Roger",
"BISLY": "But I Still Love You",
"AFC": "Away From Computer",
"AFF": "Affirmative",
"AFI": "A Fire Inside (Band)",
"BREDRINS": "Close friends| brothers",
"AFK": "Away From Keyboard",
"HSV2": "Herpes Simplex Virus 2",
"AFL": "Australian Football League",
"AFN": "All For Now",
"TEVS": "Whatever",
"SBSP": "SpongeBob SquarePants",
"AFP": "Away From Phone",
"GPRS": "General Packet Radio Services",
"LOAF": "Head",
"FMOT": "Follow Me On Twitter",
"PURDY": "Pretty",
"AFS": "Always, Forever and Seriously",
"RHD": "Right Hand Drive",
"RHI": "Rumour Has It",
"WOAH": "WHOA",
"OGIM": "Oh God It's Monday",
"WUGOWM": "Will You Go Out With Me?",
"YDEK": "You Don't Even Know",
"HBTU": "Happy Birthday To You",
"RHS": "Right Hand Side",
"AGC": "Automatic Gain Control",
"GITMO": "Guantanamo Bay",
"HBTY": "Happy Birthday To You",
"KLUTZ": "Clumsy or stupid person",
"MACK": "Flirt",
"KILLA": "Good quality weed",
"AGG": "Cool",
"AGH": "Ain't Gonna Happen",
"NUFF": "Enough",
"NO-BRAINER": "no thought before deciding",
"AGM": "Annual General Meeting",
"AGP": "Accelerated Graphics Port",
"RIB": "Make fun of",
"BUSKING": "Playing music in public for money",
"BREH": "Brethren| brothers",
"NGUNS": "Never Give Up| Never Surrender",
"AGT": "America's Got Talent (TV show)",
"RIF": "Reduction In Force",
"RIH": "Rot In Hell",
"AGW": "All Going Well",
"BACS": "Bankers Automated Clearing System",
"INWLY": "I Never Wanna Lose You",
"RIP": "Convert music/video on CD to MP3s| Rest In Peace",
"SSTS": "Snickering Softly To Self",
"AHA": "discovery| realization",
"SELFIE": "Picture of yourself, usually taken by yourself",
"RIS": "Received In Swap",
"AHO": "Idiot",
"TEXT": "An SMS message sent between cell phones",
"SSWA": "So Say We All",
"AHT": "Out",
"SBUX": "Starbucks",
"CHIZZ": "Chill out| relax",
"COUPLE": "Two people in a relationship",
"NSFTBC": "Not Sorry For The Broadcast",
"AIB": "Am I Boverd?",
"BREW": "Beer| Tea| Brother",
"AII": "Alright",
"WOAT": "Worst Of All Time",
"AIM": "AOL Instant Messanger",
"HBU?": "How About You?",
"AIO": "All In One",
"AIR": "Blanked| ignored",
"AIT": "Alright",
"MADD": "Mothers Against Drunk Driving",
"MAGGOT": "Fan of the band Slipknot",
"NUKE": "To attack with a nuclear weapon",
"DOOFUS": "Stupid person",
"NOTTIE": "Unattractive person| not a HOTTIE",
"VLAN": "Virtual Local Area Network",
"RLF": "Real Life Friend",
"NATCH": "Naturally",
"KNOCKED": "Caught",
"BAFO": "Best And Final Offer",
"AKA": "Also Known As",
"FULL MONTY": "Whole way| whole thing",
"RLY": "Really",
"AKI": "Autumn| fall",
"MESSRS": "Messieurs| Mister",
"RLZ": "Rules",
"RMA": "Return Merchandise Authorization",
"AKS": "Ask",
"CULO": "Ass",
"RME": "Rolling My Eyes",
"RML": "Read My Lips",
"ALA": "In the style of",
"RMR": "Remember",
"CULA": "See You Later, Alligator",
"RMT": "Real-Money Trading",
"BAGL": "Bust A Gut Laughing",
"RMV": "Results May Vary",
"ALF": "Animal Liberation Front",
"ALG": "Ain't Life Grand",
"ALI": "Best Friend",
"ALK": "Alcohol",
"RNA": "Ribonucleic Acid",
"MUCHO": "Much| very",
"RNB": "Rhythm And Blues",
"S/B": "Should Be",
"ALT": "Alternative character in RPG",
"ALU": "Arithmetic Logic Unit",
"RNG": "Random Number Generator",
"ALV": "Asta La Vista| goodbye",
"HAFTA": "Have To",
"S/O": "Shout Out| Significant Other",
"TROLLING": "The act of being a TROLL",
"S/N": "Serial Number",
".GOV": "governments",
"AMA": "Ask Me Anything| Against Medical Advice",
"BTBAM": "Between The Buried And Me (band)",
"AMC": "American Motors Corportation",
"RNR": "Rock 'N Roll",
"AMD": "Advanced Micro Devices",
"SWEXY": "Sweet and sexy",
"S/T": "Self Titled",
"AMG": "Ah My God",
"ELITE": "Skillful| quality| professional",
"AML": "All My Love",
"BOTTS": "Back On To The Subject",
"GGNORE": "Good Game No Rematch",
"AMN": "Any Minute Now",
"AMP": "Amplifier| Ampere",
"AMT": "Alpha-Methyltryptamine",
"ROF": "Rate Of Fire| ROFL",
"ROE": "Rules Of Engagement",
"AMV": "Anime Music Video",
"CDMA": "Code Division Multiple Access",
"ROH": "Ring of Honor (wrestling)",
"AMW": "America's Most Wanted (TV show)",
"ROG": "Really Old Git",
"ROI": "Return on Investment",
"AMZ": "Amazing",
"ROM": "Read-Only Memory",
"BAIT": "Blatant| obvious",
"ROR": "Raughing out Roud",
"ANC": "African National Congress",
"B/C": "Because",
"DGOT": "Don't Go Over There",
"ROS": "Right On Schedule",
"ROV": "Remotely Operated Vehicle",
"ANF": "Abercrombie & Fitch",
"ROX": "Rocks",
"ROW": "Rest Of the World",
"BAIL": "Leave",
"ROY": "Relative of yours?",
"ANO": "Anorexic",
"RPC": "Role Playing Chat",
"ANS": "Answer",
"BAKE": "Smoke marijuana",
"RPG": "Rocket Propelled Grenade| Role Playing Game",
"RPF": "Real Person Fiction",
"B/W": "Between| Black and White",
"BAKA": "Fool| idiot",
"IWBB": "I Will Be Back",
"RPM": "Revolutions Per Minute",
"CUOL": "See You On Line",
"IITYWIMIWHTKY": "If I Tell You What It Means I Will Have to Kill You",
"AOB": "Any Other Business",
"RPS": "Rock Paper Scissors (game)",
"AOD": "All Of Day",
"AOE": "Area Of Effect",
"BEASTLY": "Being good at something| cool",
"YDMS": "You Don't Make Sense",
"YDMT": "You Don't Mean That",
"AOK": "Age of Kings (game)| All OK",
"AOL": "American Online",
"AOM": "Age of Mythology (game)",
"AON": "All Or Nothing| As Of Now",
"AOS": "Ahead Of Schedule",
"AOT": "As Opposed To| Among Other Things",
"AOW": "All Out War",
"S2G": "Swear To God",
"BANTER": "Playful| witty chat",
"S2R": "Send To Receive",
"APB": "All Points Bulletin",
"APC": "Armoured Personnel Carrier",
"S2S": "Sorry To Say| Safe To Say",
"APE": "Mad| crazy",
"S2U": "Same To You",
"API": "Application Programming Interface",
"APM": "Actions Per Minute",
"APN": "As Per Normal",
"APO": "Authorized Personnel Only",
"APP": "Application (computer program)",
"APR": "Annual Percentage Rate",
"OWNED": "Dominated",
"APT": "Apartment",
"RRD": "Rick Rolled",
"APU": "As Per Usual",
"BOTOH": "But On The Other Hand",
"BFITW": "Best Friend In The World",
"SPARK": "Set light to",
"BAMA": "Person lacking style",
"DHTML": "Dynamic HTML",
"RRP": "Recommended Retail Price",
"10Q": "Thank you",
"B2B": "Back To Back| Business To Business",
"B2C": "Business To Consumer",
"CHIRP": "Insult",
"MAMI": "Sexy, Baby",
"RRR": "Reading, 'Riting, 'Rithmetic",
"ROTFLOL": "Rolling On The Floor Laughing Out Loud",
"AQF": "Ay Que Funny| LOL",
"10X": "Thanks",
"CQTMS": "Chuckle Quietly To Myself",
"JIFF": "Cocaine",
"RSA": "An encryption algorithm",
"B2W": "Back To Work",
"RSI": "Repetitive Strain Injury",
"RSN": "Real Soon Now",
"ARD": "Alright",
"RSS": "Rich Site Summary| Really Simple Syndication",
"S4S": "Support For Support",
"ARF": "Laugh",
"ARG": "Alternate Reality Gaming",
"ARK": "Act of Random Kindness",
"BAMN": "By Any Means Necessary",
"RTA": "Road Traffic Accident",
"LOML": "Love Of My Life",
"RTB": "Return To Base",
"RTE": "National broadcaster in Ireland",
"STBE": "Soon-To-Be Ex",
"ART": "Alright",
"RTD": "Ready To Drink",
"RTG": "Ready To Go",
"RTF": "Return The Favour",
"LOMO": "Lights Out, Missionary Only",
"RTM": "Read The Manual",
"BAKED": "Stoned| Wasted",
"RTO": "Radio Telephone Operator",
"CUSS": "Curse| swear",
"ASA": "And So On",
"RTS": "Real Time Strategy",
"RTR": "Roll Tide Roll",
"STBX": "Soon-To-Be Ex",
"ASD": "Sequence of letters on many keyboards",
"WOMP": "Low bass sound",
"ASF": "And So Forth",
"RTW": "Ready To Wear| Round The World",
"ASG": "Asgarv| LOL",
"RTV": "Rock The Vote",
"CHIPS": "Money| Weak| corny",
"CUSH": "High grade marijuana",
"YUSH": "Yes",
"ASL": "Age, Sex, Location",
"SCAM": "Rip off| Make out",
"ASM": "Assembly computer language",
"ASN": "Any Second Now",
"B4N": "Bye For Now",
"ASO": "I see",
"ASP": "At Some Point",
"STBY": "Sucks To Be You| Sorry To Bother You",
"RUA": "Are You Alone?",
"LOLL": "Laugh Out Loud Literally",
"NASCAR": "National Association for Stock Car Auto Racing",
"LOLK": "LOL+OK",
"B4U": "Before you",
"HESHE": "Person of unidentifiable gender",
"LOLO": "Lowrider| Crazy",
"LOLN": "LOL, Not",
"CUTY": "Cutie",
"STAN": "Stalker Fan",
"MANC": "Person from Manchester",
"LOLX": "LOL",
"PAYCE": "Peace",
"RUO": "Weak",
"ATA": "Actual Time of Arrival",
"LOLV": "Lots Of Love",
"MANG": "Man",
"ATB": "All The Best",
"ATC": "Air Traffic Control",
"RUT": "Are You There",
"ATD": "Attention to Detail",
"RUS": "Are You Serious?",
"LOLZ": "Laugh Out Loud",
"STAT": "Immediately",
"ATF": "All Time Favorite",
"STAR": "Friend| buddy",
"TFCD": "Time For Compact Disc",
"ATH": "All Time High",
"ATI": "Array Technology Inc.",
"WOLS": "Slow",
"ATK": "Attack",
"MANU": "MAN U",
"ATL": "Atlanta, Georgia",
"ATM": "At The Moment| Automated Teller Machine",
"ATN": "Any Time Now",
"LOMB": "Lying On My Bed",
"CUTE": "Someone/thing that is attrative in an innocent way",
"ATQ": "Answer The Question",
"HTBT": "Had To Be There",
"RVB": "Red Versus Blue",
"ATT": "At This Time| All The Time",
"ATV": "All Terrain Vehicle",
"LOOL": "Laughing Outlandishly Out Loud",
"ATW": "All The Way",
"COCK UP": "Mistake| mess",
"ATX": "Austin, Texas",
"LOOT": "Money",
"IWHI": "I Would Hit It",
"RVR": "Realm Versus Realm (MMORPG)",
"AUD": "Australian Dollar",
"CHINK": "Offensive term for an Asian or Chinese person",
"DOSSER": "Lazy person| Vagrant| Tramp",
"WOOP": "WOOT",
"IFFY": "Dodgy| unreliable",
"WOOT": "Hurrah| Want One Of Those",
"SPROG": "Child",
"AUO": "I don't know",
"AUP": "Another Useless Post| Acceptable Use Policy",
"LOPL": "LOL",
"WOOK": "Dirty hippy",
"RWD": "Rear Wheel Drive",
"AUS": "Are You Serious?",
"THURR": "There",
"KLKL": "Cool Cool",
"NETWORK": "Two or more connected computers",
"YUTE": "Youth",
"BARN": "Barnacle| person who latches on",
"RWL": "Roaring With Laughter",
"GAMER": "Person who plays video games",
"NUTZ": "Nuts| crazy| insane",
"RWP": "Random Wall Post",
"NNITO": "Not Necessarily In That Order",
"AVA": "Angels and Airwaves (Band)",
"BARE": "Lots of| very",
"BARF": "Throw up| vomit",
"AVG": "Average",
"ARPANET": "Advanced Research Projects Agency Network",
"AVI": "Audio Video Interleaved (media format)",
"CONDO": "Apartment",
"AVO": "Afternooon",
"WONK": "Expert",
"ROFLCOPTER": "LOL, ROFL etc",
"BB4N": "Bye-Bye For Now",
"MASC": "Masculine",
"TWIT": "Foolish person| That's What I Thought",
"AWA": "As Well As",
"AWD": "All Wheel Drive",
"AWK": "Awkward",
"CHILL": "Relax| hang out",
"BRUDDA": "Brother",
"FNAR": "For No Apparent Reason| Snigger",
"AWP": "Arctic Warfare Police sniper rifle",
"AWS": "As We Speak",
"RYC": "Regarding Your Comment",
"LORL": "Laugh Out Really Loud",
"AWW": "Cute",
"BATO": "Dude",
"TFFW": "Too Funny For Words",
"RYN": "Read Your Note| Regarding Your Note",
"RYO": "Roll Your Own",
"TWHS": "That's What He Said",
"IWID": "It's What I Do",
"RYT": "Right| alright",
"BATE": "Obvious",
"BRUV": "Mate",
"MARK": "Target| sucker",
"STEM": "Mix of STUD and FEM",
"IWIK": "I Wish I Knew",
"STET": "Let it stand",
"BRUK": "Broken",
"TWID": "That's What I Do",
"LIPSING": "Kissing",
"BRUH": "Brother",
"SHIZZLE": "Sure",
"KTHXBI": "Okay, Thanks, Bye",
"MAUH": "Kiss",
"AOTA": "All Of The Above",
"STHG": "Something",
"STHU": "Shut The Hell Up",
"AYB": "All Your Base",
"AYC": "At Your Convenience",
"AYE": "Yes",
"LOTD": "Link Of The Day",
"CRACKA": "Offensive term for a white person",
"AYK": "As You Know",
"AYL": "Are You Listening?",
"AYO": "Hey, You",
"AYR": "Are You Ready?",
"AYS": "Are You Serious?",
"LOTI": "Laughing On The Inside",
"AYT": "Are You There?",
"AYW": "As You Wish",
"MATE": "Friend",
"VLOG": "Video Log",
"GR&D": "Grinning, Running & Ducking",
"CUZZ": "Friend| cousin",
"NATTY": "Natural Light beer| Smart, good, cool",
"AOTS": "Attack of the Show (TV show)",
"AZN": "Asian",
"YDWK": "YDWTK",
"WORM": "Malicious computer program",
"HCIT": "How Cool Is That?",
"TFLN": "Texts From Last Night",
"DYAC": "Damn You Auto Correct",
"SWAMBO": "She Who Always Must Be Obeyed",
"HEADS UP": "Advance notice",
"LOVL": "Laughing Out Very Loud",
"WOTD": "Word Of The Day",
"LOTR": "Lord of the Rings",
"DELISH": "Delicious",
"SIWOTI": "Someone Is Wrong On The Internet",
"LOUD": "Good quality marijuana",
"NAMASTE": "Sanskrit greeting",
"WOTS": "Word On The Street",
"TWOP": "Television Without Pity",
"WIFEY": "Serious girlfriend| wife material",
"CAUSE": "Because",
"HTML": "Hypertext Markup Language",
"MAYN": "Man",
"BOTCH": "Mess up",
"GAMMD": "Go Ahead Make My Day",
"TWOT": "Total Waste Of Time",
"10CHAR": "Padding where at least 10 characters are required",
"SOOTB": "Straight Out Of The Box",
"STMF": "Stay Thirsty My Friends",
"OHAC": "Own House And Car",
"SYTYCD": "So You Think You Can Dance",
"VLSI": "Very Large-Scale Integration",
"H2IK": "Hell If I Know",
"&HEARTS;": "HTML code for a heart",
"BOTDF": "Blood On The Dance Floor (band)",
"TWOC": "Taken Without Owner's Consent",
"LOWE": "Don't bother| forget it",
"SAB": "Smoking A Blunt",
"SAE": "Stamped/Self Addressed Envelope",
"SAD": "Pathetic| Seasonal Adjustment Disorder",
"SAG": "Wear trousers low around the waist",
"SAF": "Single Asian Female",
"XAXA": "Ha Ha",
"SAM": "Stop Annoying Me",
"SAN": "Japanese name suffix, Mr or Ms",
"SAP": "Sad And Pathetic| Fool",
"SAS": "Special Air Service",
"LOYOL": "Live Out Your Own Life",
"SAT": "Scholastic Assessment Test",
"STNY": "Same Time Next Year",
"SBD": "Silent But Deadly",
"SBF": "Single Black Female",
"BROSKI": "Bro",
"SBJ": "Sweet Baby Jesus",
"AOYP": "Angel on Your Pillow",
"SBI": "Surrounded By Incompetence",
"DYEL": "Do You Even Lift?",
"SBM": "Single Black Male",
"0_0": "Shocked",
"BAC": "Back At Computer| Blood Alcohol Content| By Any Chance",
"SBT": "Sorry 'Bout That",
"BAD": "Good",
"OHAI": "Oh, Hi",
"SBS": "Step By Step",
"BAE": "Baby",
"STML": "Short Term Memory Loss",
"BAG": "Get",
"BAH": "Bored As Hell",
"BAI": "Bye",
"BAK": "Back At Keyboard",
"BAM": "Below Average Mentality| Exclamation of happiness",
"DHCP": "Dynamic Host Configuration Protocol",
"BAO": "Be Aware Of",
"SCA": "Scandal",
"BAU": "Business As Usual",
"SCH": "School",
"BAY": "Baby",
"SCO": "San Fransisco"
}

In [66]:
# Function to change slang word to original words

def chat_convert(text):
    new_txt = []
    for word in text.split():
        if word.upper() in chat_words:
            new_txt.append(chat_words[word.upper()])
        else:
            new_txt.append(word)
    return " ".join(new_txt)

In [67]:
df['text_updated'] = df['text_'].apply(chat_convert)

In [68]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

Number of rows affected: 35415


In [69]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Very, so made, sturdy, and very com..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I'..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very National Institute of Clinical Excellence...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,You can wear the Neighborhood| ghetto by itsel...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...


In [70]:
# Convert text into lower case
df['text_'] = df['text_'].str.lower()
# Remove numbers
df['text_'] = df['text_'].str.replace(r'\d+', '', regex=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"love this! very, so made, sturdy, and very com..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. i'..."
2,Home_and_Kitchen_5,5.0,CG,this pillow saved my back. i love the look and...
3,Home_and_Kitchen_5,1.0,CG,"missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,very national institute of clinical excellence...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,i had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,i wasn't sure exactly what it would be. it is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,you can wear the neighborhood| ghetto by itsel...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,i liked nothing about this dress. the only rea...


In [71]:
# Function to detect HTML tags

def change_html_tags(text):
    html_content = r'<[^>]+>(.*?)</[^>]+>'
    matches = re.findall(html_content, text)
    return ' '.join(matches) if matches else text

In [72]:
df['text_updated'] = df['text_'].apply(change_html_tags)

In [73]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

Number of rows affected: 195


In [74]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"love this! very, so made, sturdy, and very com..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. i'..."
2,Home_and_Kitchen_5,5.0,CG,this pillow saved my back. i love the look and...
3,Home_and_Kitchen_5,1.0,CG,"missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,very national institute of clinical excellence...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,i had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,i wasn't sure exactly what it would be. it is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,you can wear the neighborhood| ghetto by itsel...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,i liked nothing about this dress. the only rea...


In [75]:
# Function to detect links(https, http, www) tags

def remove_links(text):
    link_pattern = r'(http://\S+|https://\S+|www\.\S+)'
    return re.sub(link_pattern, '', text).strip()

In [76]:
df['text_updated'] = df['text_'].apply(remove_links)

In [77]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

Number of rows affected: 245


In [78]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"love this! very, so made, sturdy, and very com..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. i'..."
2,Home_and_Kitchen_5,5.0,CG,this pillow saved my back. i love the look and...
3,Home_and_Kitchen_5,1.0,CG,"missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,very national institute of clinical excellence...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,i had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,i wasn't sure exactly what it would be. it is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,you can wear the neighborhood| ghetto by itsel...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,i liked nothing about this dress. the only rea...


In [79]:
# Function to remove punctuation

def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [80]:
df['text_updated'] = df['text_'].apply(remove_punctuation)

In [81]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

Number of rows affected: 40115


In [82]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love this very so made sturdy and very comfort...
1,Home_and_Kitchen_5,5.0,CG,love it a great upgrade from the original ive ...
2,Home_and_Kitchen_5,5.0,CG,this pillow saved my back i love the look and ...
3,Home_and_Kitchen_5,1.0,CG,missing information on how to use it but it is...
4,Home_and_Kitchen_5,5.0,CG,very national institute of clinical excellence...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,i had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,i wasnt sure exactly what it would be it is a ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,you can wear the neighborhood ghetto by itself...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,i liked nothing about this dress the only reas...


In [83]:
# Function to remove stop words

def remove_stop_words(text):
    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [84]:
df['text_updated'] = df['text_'].apply(remove_stop_words)

In [85]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

Number of rows affected: 40319


In [86]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love made sturdy comfortable love itvery...
1,Home_and_Kitchen_5,5.0,CG,love great upgrade original ive mine tw...
2,Home_and_Kitchen_5,5.0,CG,pillow saved back love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,missing information use great product ...
4,Home_and_Kitchen_5,5.0,CG,national institute clinical excellence set g...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,read reviews saying bra ran small orde...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,wasnt sure exactly would little large ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,wear neighborhood ghetto wear hood we...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,liked nothing dress reason gave stars ...


In [87]:
# Function to change emoji in meaningful text

def emoji_to_text(text):
  return emoji.demojize(text, language="en")

In [88]:
df['text_updated'] = df['text_'].apply(emoji_to_text)

In [89]:
affected_rows = (df['text_'] != df['text_updated']).sum()
print(f"Number of rows affected: {affected_rows}")

# Emoji is not there!

Number of rows affected: 0


In [90]:
df['text_'] = df['text_updated']
df.drop(columns=['text_updated'], inplace=True)

In [91]:
# Function of tokenization

nlp = spacy.load("en_core_web_sm")

def tokenize_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [92]:
df['text_'] = df['text_'].apply(tokenize_text)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"[love, , made, sturdy, , comfortable, , ..."
1,Home_and_Kitchen_5,5.0,CG,"[love, , great, upgrade, , original, i, ve..."
2,Home_and_Kitchen_5,5.0,CG,"[ , pillow, saved, , back, , love, , look, ..."
3,Home_and_Kitchen_5,1.0,CG,"[missing, information, , use, , great,..."
4,Home_and_Kitchen_5,5.0,CG,"[ , national, institute, , clinical, excellen..."
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,"[ , read, , reviews, saying, , bra, ran, s..."
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,"[ , was, nt, sure, exactly, , would, , l..."
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"[ , wear, , neighborhood, ghetto, , wear, ..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,"[ , liked, nothing, , dress, , reason, , ..."


In [93]:
# Function for Lemmatization task

def lemmatize_tokens(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_tokens

In [94]:
df['text_'] = df['text_'].apply(lemmatize_tokens)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"[love, , made, sturdy, , comfortable, , ..."
1,Home_and_Kitchen_5,5.0,CG,"[love, , great, upgrade, , original, i, ve..."
2,Home_and_Kitchen_5,5.0,CG,"[ , pillow, saved, , back, , love, , look, ..."
3,Home_and_Kitchen_5,1.0,CG,"[missing, information, , use, , great,..."
4,Home_and_Kitchen_5,5.0,CG,"[ , national, institute, , clinical, excellen..."
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,"[ , read, , review, saying, , bra, ran, sm..."
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,"[ , wa, nt, sure, exactly, , would, , li..."
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"[ , wear, , neighborhood, ghetto, , wear, ..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,"[ , liked, nothing, , dress, , reason, , ..."


In [95]:
# Function to remove space

def remove_spaces(words_list):
    return [word for word in words_list if word.strip()]  # Removes empty or space-only elements

In [96]:
df['text_'] = df['text_'].apply(remove_spaces)

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"[love, made, sturdy, comfortable, love, itvery..."
1,Home_and_Kitchen_5,5.0,CG,"[love, great, upgrade, original, i, ve, mine, ..."
2,Home_and_Kitchen_5,5.0,CG,"[pillow, saved, back, love, look, feel, pillow]"
3,Home_and_Kitchen_5,1.0,CG,"[missing, information, use, great, product, pr..."
4,Home_and_Kitchen_5,5.0,CG,"[national, institute, clinical, excellence, se..."
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,"[read, review, saying, bra, ran, small, ordere..."
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,"[wa, nt, sure, exactly, would, little, large, ..."
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"[wear, neighborhood, ghetto, wear, hood, wear,..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,"[liked, nothing, dress, reason, gave, star, or..."


In [97]:
df["text_"] = df["text_"].apply(lambda x: " ".join(x))

df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love made sturdy comfortable love itvery pretty
1,Home_and_Kitchen_5,5.0,CG,love great upgrade original i ve mine two peop...
2,Home_and_Kitchen_5,5.0,CG,pillow saved back love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,missing information use great product price
4,Home_and_Kitchen_5,5.0,CG,national institute clinical excellence set goo...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,read review saying bra ran small ordered two b...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,wa nt sure exactly would little large small si...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,wear neighborhood ghetto wear hood wear jacket...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,liked nothing dress reason gave star ordered s...


In [98]:
# One-hot encoding on 'category' column
df_encoded = pd.get_dummies(df['category'], columns=['category'])

# Convert boolean values (if any) to integers (0 or 1)
df_encoded = df_encoded.apply(lambda x: x.astype(int))
df_encoded

,Books_5,Clothing_Shoes_and_Jewelry_5,Electronics_5,Home_and_Kitchen_5,Kindle_Store_5,Movies_and_TV_5,Pet_Supplies_5,Sports_and_Outdoors_5,Tools_and_Home_Improvement_5,Toys_and_Games_5
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
40427,0,1,0,0,0,0,0,0,0,0
40428,0,1,0,0,0,0,0,0,0,0
40429,0,1,0,0,0,0,0,0,0,0
40430,0,1,0,0,0,0,0,0,0,0


In [99]:
df = pd.concat([df, df_encoded], axis=1)
df.drop(['category'], axis=1, inplace=True)

In [100]:
df

,rating,label,text_,Books_5,Clothing_Shoes_and_Jewelry_5,Electronics_5,Home_and_Kitchen_5,Kindle_Store_5,Movies_and_TV_5,Pet_Supplies_5,Sports_and_Outdoors_5,Tools_and_Home_Improvement_5,Toys_and_Games_5
0,5.0,CG,love made sturdy comfortable love itvery pretty,0,0,0,1,0,0,0,0,0,0
1,5.0,CG,love great upgrade original i ve mine two peop...,0,0,0,1,0,0,0,0,0,0
2,5.0,CG,pillow saved back love look feel pillow,0,0,0,1,0,0,0,0,0,0
3,1.0,CG,missing information use great product price,0,0,0,1,0,0,0,0,0,0
4,5.0,CG,national institute clinical excellence set goo...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,4.0,OR,read review saying bra ran small ordered two b...,0,1,0,0,0,0,0,0,0,0
40428,5.0,CG,wa nt sure exactly would little large small si...,0,1,0,0,0,0,0,0,0,0
40429,2.0,OR,wear neighborhood ghetto wear hood wear jacket...,0,1,0,0,0,0,0,0,0,0
40430,1.0,CG,liked nothing dress reason gave star ordered s...,0,1,0,0,0,0,0,0,0,0


In [101]:
# Initialize the LabelEncoder
encoder = LabelEncoder()

# Apply label encoding to 'category' column
df['label'] = encoder.fit_transform(df['label'])
df

,rating,label,text_,Books_5,Clothing_Shoes_and_Jewelry_5,Electronics_5,Home_and_Kitchen_5,Kindle_Store_5,Movies_and_TV_5,Pet_Supplies_5,Sports_and_Outdoors_5,Tools_and_Home_Improvement_5,Toys_and_Games_5
0,5.0,0,love made sturdy comfortable love itvery pretty,0,0,0,1,0,0,0,0,0,0
1,5.0,0,love great upgrade original i ve mine two peop...,0,0,0,1,0,0,0,0,0,0
2,5.0,0,pillow saved back love look feel pillow,0,0,0,1,0,0,0,0,0,0
3,1.0,0,missing information use great product price,0,0,0,1,0,0,0,0,0,0
4,5.0,0,national institute clinical excellence set goo...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,4.0,1,read review saying bra ran small ordered two b...,0,1,0,0,0,0,0,0,0,0
40428,5.0,0,wa nt sure exactly would little large small si...,0,1,0,0,0,0,0,0,0,0
40429,2.0,1,wear neighborhood ghetto wear hood wear jacket...,0,1,0,0,0,0,0,0,0,0
40430,1.0,0,liked nothing dress reason gave star ordered s...,0,1,0,0,0,0,0,0,0,0


<h1>Bag Of Words</h1>

In [102]:
# Apply Bag of Words using CountVectorizer
vectorizer = CountVectorizer(max_features=5000)
bow_matrix = vectorizer.fit_transform(df["text_"])

# Convert BoW matrix to DataFrame for better visualization
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [103]:
bow_df

,aaron,abandoned,abbey,ability,able,abrupt,abruptly,absolute,absolutely,absorb,...,youth,youtube,yr,zero,zipper,zippered,zoe,zombie,zone,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
40430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
bow_df = pd.concat([df, bow_df], axis=1)
bow_df.drop(['text_'], axis=1, inplace=True)
bow_df
bow_df.to_csv("Bag_of_words_dataset.csv", index=False)

<h1>TF-IDF</h1>

In [105]:
# Apply TF-IDF using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(df["text_"])

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [106]:
tfidf_df

,aaron,abandoned,abbey,ability,able,abrupt,abruptly,absolute,absolutely,absorb,...,youth,youtube,yr,zero,zipper,zippered,zoe,zombie,zone,zoom
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,0.0,0.0,0.0,0.0,0.087820,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
40428,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
40429,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.052479,0.0,0.0,0.0,0.0,0.0
40430,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [107]:
tfidf_df = pd.concat([df, tfidf_df], axis=1)
tfidf_df.drop(['text_'], axis=1, inplace=True)
tfidf_df
tfidf_df.to_csv("tf-idf_dataset.csv", index=False)